In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
#fix latitudes that aren't right, +70 not -70

# This is NYC's subway data.

Normally we'd pull it from the internet but Kaggle has locked down their servers so we'll read it as a string into a dataframe. 

In [ ]:
TESTDATA=StringIO("""Division,Line,Station_Name,Station_Latitude,Station_Longitude,Route_1,Route_2,Route_3,Route_4,Route_5,Route_6,Route_7,Route_8,Route_9,Route_10,Route_11,Entrance_Type,Entry,Exit_Only,Vending,Staffing,Staff_Hours,ADA,ADA_Notes,Free_Crossover,North_South_Street,East_West_Street,Corner,Latitude,Longitude
BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,23rd Ave,NW,40.775149,-73.912074
BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,23rd Ave,NE,40.77481,-73.912151
BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,23rd Ave,NE,40.775025,-73.911891
BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,23rd Ave,NW,40.774938,-73.912337
BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Hoyt Ave South,SW,40.770313,-73.917978
BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Hoyt Ave South,SE,40.77017,-73.917755
BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Hoyt Ave North,NW,40.771009,-73.917167
BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Hoyt Ave North,NE,40.77087,-73.916935
BMT,Astoria,30 Av-Grand Av,40.766779,-73.921479,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,30th Ave,SW,40.766787,-73.921629
BMT,Astoria,30 Av-Grand Av,40.766779,-73.921479,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,30th Ave,SE,40.7667,-73.921406
BMT,Astoria,30 Av-Grand Av,40.766779,-73.921479,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,30th Ave,NW,40.767052,-73.921409
BMT,Astoria,30 Av-Grand Av,40.766779,-73.921479,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,30th Ave,NE,40.766963,-73.921192
BMT,Astoria,Broadway,40.76182,-73.925508,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Broadway,SW,40.76188,-73.925638
BMT,Astoria,Broadway,40.76182,-73.925508,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Broadway,SE,40.761773,-73.925399
BMT,Astoria,Broadway,40.76182,-73.925508,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,Broadway,NW,40.762217,-73.925349
BMT,Astoria,36 Av-Washington Av,40.756804,-73.929575,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,36th Ave,SW,40.756478,-73.929993
BMT,Astoria,36 Av-Washington Av,40.756804,-73.929575,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,36th Ave,SE,40.756379,-73.929766
BMT,Astoria,36 Av-Washington Av,40.756804,-73.929575,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,36th Ave,NW,40.756796,-73.92974
BMT,Astoria,39 Av-Beebe Av,40.752882,-73.932755,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,39th Ave,NW,40.752901,-73.932903
BMT,Astoria,39 Av-Beebe Av,40.752882,-73.932755,N,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,31st St,39th Ave,SE,40.752491,-73.932936
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,60th St,NW,40.765156,-73.972501
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,60th St,SE,40.764728,-73.972396
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,60th St,NE,40.764849,-73.972306
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,60th St,SE,40.764657,-73.972229
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,57th St,SW,40.765365,-73.980341
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,57th St,SE,40.765298,-73.980033
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,57th St,NW,40.765796,-73.980012
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,57th St,NE,40.765639,-73.979764
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Northbound Only,FALSE,7th Ave,49th St,SW,40.760486,-73.983974
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Northbound Only,FALSE,7th Ave,49th St,NW,40.760621,-73.983904
BMT,Broadway,28th St,40.745494,-73.988691,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,28th St,SE,40.745089,-73.988472
BMT,Broadway,28th St,40.745494,-73.988691,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,28th St,NE,40.745299,-73.988579
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,23rd St,SE,40.74108,-73.989083
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,23rd St,SE,40.741111,-73.98892
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,23rd St,NE,40.741378,-73.988895
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,23rd St,NE,40.741539,-73.988989
BMT,Broadway,Union Square,40.735736,-73.990568,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,FULL,,TRUE,Lex not ADA,TRUE,Broadway,14th St,NE,40.735278,-73.991058
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,8th Ave,SE,40.730477,-73.992286
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,8th Ave,NE,40.730578,-73.992216
BMT,Broadway,Prince St,40.724329,-73.997702,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,Prince St,SE,40.724124,-73.997686
BMT,Broadway,Prince St,40.724329,-73.997702,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,Prince St,NE,40.72422,-73.997633
BMT,Broadway,City Hall,40.713282,-74.006978,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Warren St,SE,40.71311,-74.007045
BMT,Broadway,City Hall,40.713282,-74.006978,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Warren St,NE,40.713437,-74.00676
BMT,Broadway,City Hall,40.713282,-74.006978,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Warren St,SE,40.713521,-74.006697
BMT,Broadway,Cortlandt St,40.710668,-74.011029,R,,,,,,,,,,,Stair,NO,,NO,FULL,,FALSE,,TRUE,Church St,Dey St,NE,40.710871,-74.010767
BMT,Broadway,Cortlandt St,40.710668,-74.011029,R,,,,,,,,,,,Stair,NO,,NO,FULL,,FALSE,,TRUE,Church St,Dey St,SE,40.711065,-74.010644
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Trinity Pl,Rector St,SE,40.707647,-74.012924
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Trinity Pl,Rector St,NE,40.707902,-74.012774
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Trinity Pl,Rector St,NE,40.708028,-74.012696
BMT,Broadway,Whitehall St,40.703087,-74.012994,R,1,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,,,,40.701938,-74.012588
BMT,Broadway,Whitehall St,40.703087,-74.012994,R,1,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,,,,40.702108,-74.012642
BMT,Broadway,Whitehall St,40.703087,-74.012994,R,1,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,,,,40.702108,-74.012642
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lawrence St,Willoughby St,SE,40.692114,-73.986107
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lawrence St,Willoughby St,SE,40.692258,-73.986114
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lawrence St,Willoughby St,NE,40.692052,-73.986191
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lawrence St,Willoughby St,NE,40.692359,-73.986181
BMT,4 Avenue,DeKalb Av,40.690635,-73.981824,B,Q,R,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Flatbush Ave,De Kalb Ave,SE,40.689804,-73.981164
BMT,4 Avenue,DeKalb Av,40.690635,-73.981824,B,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Flatbush Ave,De Kalb Ave,SW,40.68981,-73.981651
BMT,4 Avenue,DeKalb Av,40.690635,-73.981824,B,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Flatbush Ave,De Kalb Ave,SE,40.689817,-73.981082
BMT,4 Avenue,DeKalb Av,40.690635,-73.981824,B,Q,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Flatbush Ave,De Kalb Ave,SE,40.68974,-73.981156
BMT,4 Avenue,Atlantic Av-Barclays Ctr,40.683666,-73.97881,B,Q,D,N,R,2,3,4,5,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,4th Ave,Pacific St,NE,40.683805,-73.978487
BMT,4 Avenue,Pacific St,40.683666,-73.97881,B,Q,D,N,R,2,3,4,5,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,4th Ave,Pacific St,NW,40.683905,-73.978879
BMT,4 Avenue,Pacific St,40.683666,-73.97881,B,Q,D,N,R,2,3,4,5,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,4th Ave,Pacific St,NE,40.683928,-73.978412
BMT,4 Avenue,Union St,40.677316,-73.98311,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,Union St,SE,40.677001,-73.983133
BMT,4 Avenue,Union St,40.677316,-73.98311,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,Union St,SW,40.677279,-73.983327
BMT,4 Avenue,Union St,40.677316,-73.98311,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,Union St,SE,40.677171,-73.98302
BMT,4 Avenue,9th St,40.670847,-73.988302,F,G,R,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,9th St,NE,40.670387,-73.98848
BMT,4 Avenue,Prospect Av,40.665414,-73.992872,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,Prospect Ave,NE,40.665299,-73.992729
BMT,4 Avenue,Prospect Av,40.665414,-73.992872,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,Prospect Ave,SW,40.665058,-73.993002
BMT,4 Avenue,25th St,40.660397,-73.998091,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,25th St,SE,40.660323,-73.997952
BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,36th St,NW,40.65449,-74.004499
BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,36th St,NE,40.654365,-74.004113
BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,36th St,NW,40.654676,-74.004306
BMT,4 Avenue,45th St,40.648939,-74.010006,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,45th St,NE,40.649389,-74.009333
BMT,4 Avenue,45th St,40.648939,-74.010006,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,45th St,NW,40.649424,-74.009728
BMT,4 Avenue,45th St,40.648939,-74.010006,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,45th St,NE,40.649265,-74.009457
BMT,4 Avenue,45th St,40.648939,-74.010006,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,45th St,NW,40.649546,-74.009602
BMT,4 Avenue,53rd St,40.645069,-74.014034,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,53rd St,SW,40.644653,-74.01469
BMT,4 Avenue,53rd St,40.645069,-74.014034,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,53rd St,SE,40.644493,-74.014419
BMT,4 Avenue,53rd St,40.645069,-74.014034,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,53rd St,SW,40.644756,-74.014586
BMT,4 Avenue,53rd St,40.645069,-74.014034,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,53rd St,SE,40.644601,-74.014309
BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,59th St,NW,40.641606,-74.017897
BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,59th St,NE,40.641435,-74.017561
BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,59th St,NW,40.641778,-74.017722
BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,59th St,NE,40.641633,-74.017362
BMT,4 Avenue,Bay Ridge Av,40.634967,-74.023377,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,Bay Ridge Ave,NW,40.634827,-74.02358
BMT,4 Avenue,Bay Ridge Av,40.634967,-74.023377,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,Bay Ridge Ave,NE,40.634768,-74.023339
BMT,4 Avenue,77th St,40.629742,-74.02551,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,77th St,NW,40.62955,-74.025731
BMT,4 Avenue,77th St,40.629742,-74.02551,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,77th St,NE,40.629503,-74.025452
BMT,4 Avenue,86th St,40.622687,-74.028398,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,86th St,SW,40.622656,-74.028547
BMT,4 Avenue,86th St,40.622687,-74.028398,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,86th St,SE,40.622583,-74.028315
BMT,4 Avenue,95th St,40.616622,-74.030876,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,95th St,SW,40.616021,-74.031383
BMT,4 Avenue,95th St,40.616622,-74.030876,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,95th St,NE,40.615893,-74.03086
BMT,4 Avenue,95th St,40.616622,-74.030876,R,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,95th St,NW,40.616189,-74.031234
BMT,Brighton,7th Av,40.67705,-73.972367,B,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Flatbush Ave,Park Pl,SW,40.677365,-73.972788
BMT,Brighton,7th Av,40.67705,-73.972367,B,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Carlton Ave,Park Pl,SE,40.677324,-73.972212
BMT,Brighton,Prospect Park,40.661614,-73.962246,B,Q,FS,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Ocean Ave,Lincoln Rd,NE,40.660792,-73.962109
BMT,Brighton,Parkside Av,40.655292,-73.961495,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Ocean Ave,Parkside Ave,NE,40.655221,-73.961516
BMT,Brighton,Church Av,40.650527,-73.962982,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,18th St,Church Ave,NW,40.649427,-73.963356
BMT,Brighton,Beverly Rd,40.644031,-73.964492,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Marlborough Rd,Beverly Rd,SE,40.644272,-73.964528
BMT,Brighton,Cortelyou Rd,40.640927,-73.963891,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Marlborough Rd,Cortelyou Rd,SE,40.641314,-73.963996
BMT,Brighton,Newkirk Av,40.635082,-73.962793,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Marlborough Rd,Newkirk Ave,SE,40.635034,-73.962736
BMT,Brighton,Av H,40.62927,-73.961639,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,16th St,Avenue H,SW,40.62993,-73.961477
BMT,Brighton,Av J,40.625039,-73.960803,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,15th St,Avenue J,SE,40.62513,-73.960792
BMT,Brighton,Av M,40.617618,-73.959399,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,15th St,Avenue M,SE,40.617928,-73.959454
BMT,Brighton,Kings Highway,40.60867,-73.957734,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,15th St,Kings Hwy,SE,40.608835,-73.957769
BMT,Brighton,Av U,40.5993,-73.955929,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,15th St,Avenue U,SE,40.598991,-73.955828
BMT,Brighton,Neck Rd,40.595246,-73.955161,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,15th St,Gravesend Neck Rd,NE,40.59502,-73.95512
BMT,Brighton,Sheepshead Bay,40.586896,-73.954155,B,Q,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,15th St,Sheepshead Bay Rd,SE,40.587302,-73.954224
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,,,,40.577986,-73.960258
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Branch 7 St,Brighton Beach Rd,NE,40.577931,-73.960524
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Branch 7 St,Brighton Beach Rd,SE,40.577746,-73.960231
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Branch 7 St,Brighton Beach Rd,SE,40.577697,-73.960438
BMT,Brighton,Ocean Parkway,40.576312,-73.968501,Q,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ocean Pkwy,Brighton Beach Rd,NE,40.576423,-73.967894
BMT,Brighton,Ocean Parkway,40.576312,-73.968501,Q,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ocean Pkwy,Brighton Beach Rd,NE,40.576445,-73.967591
BMT,Brighton,Ocean Parkway,40.576312,-73.968501,Q,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ocean Pkwy,Brighton Beach Rd,SE,40.576209,-73.967875
BMT,Brighton,Ocean Parkway,40.576312,-73.968501,Q,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ocean Pkwy,Brighton Beach Rd,SE,40.576234,-73.967561
BMT,Coney Island,West 8th St,40.576127,-73.975939,F,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,8th St,Surf Ave,NW,40.575499,-73.976873
BMT,Coney Island,West 8th St,40.576127,-73.975939,F,Q,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,8th St,Surf Ave,NW,40.575929,-73.976747
BMT,Coney Island,Stillwell Av,40.577422,-73.981233,D,F,N,Q,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Stillwell Ave,Mermaid Ave,SE,40.5769,-73.980846
BMT,West End,9th Av,40.646292,-73.994324,D,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,9th Ave,New Utrecht Ave,NE,40.646337,-73.994489
BMT,West End,Fort Hamilton Parkway,40.640914,-73.994304,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,45th St,NW,40.640185,-73.994423
BMT,West End,Fort Hamilton Parkway,40.640914,-73.994304,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,45th St,NE,40.64018,-73.99422
BMT,West End,Fort Hamilton Parkway,40.640914,-73.994304,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,44th St,NW,40.64136,-73.994411
BMT,West End,Fort Hamilton Parkway,40.640914,-73.994304,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,44th St,NE,40.641379,-73.994188
BMT,West End,50th St,40.63626,-73.994791,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,50th St,NW,40.636133,-73.994915
BMT,West End,50th St,40.63626,-73.994791,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,50th St,NE,40.636139,-73.994702
BMT,West End,50th St,40.63626,-73.994791,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,49th St,SW,40.636415,-73.994879
BMT,West End,50th St,40.63626,-73.994791,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,49th St,SE,40.636405,-73.994673
BMT,West End,55th St,40.631435,-73.995476,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,55th St,SE,40.631389,-73.995262
BMT,West End,55th St,40.631435,-73.995476,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,55th St,SW,40.631538,-73.995643
BMT,West End,55th St,40.631435,-73.995476,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,55th St,NE,40.631709,-73.995283
BMT,West End,71st St,40.619589,-73.998864,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,71st St,SW,40.618834,-73.999195
BMT,West End,71st St,40.619589,-73.998864,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,71st St,NW,40.619133,-73.99924
BMT,West End,71st St,40.619589,-73.998864,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,71st St,SE,40.61877,-73.998995
BMT,West End,71st St,40.619589,-73.998864,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,71st St,NE,40.618889,-73.998832
BMT,West End,79th St,40.613501,-74.00061,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,79th St,NW,40.612855,-74.000892
BMT,West End,79th St,40.613501,-74.00061,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,79th St,NE,40.612847,-74.000721
BMT,West End,18th Av,40.607954,-74.001736,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,18th Ave,85th St,SE,40.607602,-74.001491
BMT,West End,18th Av,40.607954,-74.001736,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,18th Ave,85th St,SE,40.607723,-74.001894
BMT,West End,18th Av,40.607954,-74.001736,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,18th Ave,85th St,NE,40.607897,-74.001628
BMT,West End,20th Av,40.604556,-73.998168,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,20th Ave,86th St,SE,40.604439,-73.998164
BMT,West End,20th Av,40.604556,-73.998168,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,20th Ave,86th St,NE,40.604587,-73.998007
BMT,West End,20th Av,40.604556,-73.998168,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,20th Ave,86th St,SW,40.604631,-73.998498
BMT,West End,20th Av,40.604556,-73.998168,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,20th Ave,86th St,NW,40.604789,-73.998349
BMT,West End,Bay Parkway,40.601875,-73.993728,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bay Pkwy,86th St,SE,40.601726,-73.993696
BMT,West End,Bay Parkway,40.601875,-73.993728,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bay Pkwy,86th St,NE,40.601898,-73.993538
BMT,West End,Bay Parkway,40.601875,-73.993728,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bay Pkwy,86th St,SW,40.601993,-73.99413
BMT,West End,Bay Parkway,40.601875,-73.993728,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bay Pkwy,86th St,NW,40.60215,-73.993948
BMT,West End,25th Av,40.597704,-73.986829,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,25th Ave,86th St,NE,40.597889,-73.986958
BMT,West End,25th Av,40.597704,-73.986829,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,25th Ave,86th St,NW,40.5981,-73.987279
BMT,West End,25th Av,40.597704,-73.986829,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,25th Ave,86th St,SE,40.597751,-73.987112
BMT,West End,25th Av,40.597704,-73.986829,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,25th Ave,86th St,SW,40.597952,-73.987442
BMT,West End,Bay 50th St,40.588841,-73.983765,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Stillwell Ave,Bay 50th St,SW,40.588022,-73.983735
BMT,West End,Bay 50th St,40.588841,-73.983765,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Stillwell Ave,Bay 50th St,SE,40.588066,-73.983498
BMT,West End,Bay 50th St,40.588841,-73.983765,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Stillwell Ave,Bay 50th St,SW,40.588305,-73.983791
BMT,West End,Bay 50th St,40.588841,-73.983765,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Stillwell Ave,Bay 50th St,SE,40.588348,-73.983543
BMT,Sea Beach,8th Av,40.635064,-74.011719,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,8th Ave,62nd St,NW,40.634552,-74.010726
BMT,Sea Beach,Fort Hamilton Parkway,40.631386,-74.005351,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Fort Hamilton Pkwy,62nd St,NE,40.632179,-74.006658
BMT,Sea Beach,New Utrecht Av,40.624842,-73.996353,D,N,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,New Utrecht Ave,62nd St,SE,40.625606,-73.99683
BMT,Sea Beach,18th Av,40.620671,-73.990414,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,18th Ave,64th St,NW,40.620062,-73.98941
BMT,Sea Beach,20th Av,40.61741,-73.985026,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,20th Ave,64th St,NE,40.617341,-73.98484
BMT,Sea Beach,Bay Parkway-22nd Av,40.611815,-73.981848,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Bay Pkwy,66th St,SE,40.612989,-73.982084
BMT,Sea Beach,Kings Highway,40.603923,-73.980353,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,7th St,Kings Hwy,NW,40.605107,-73.980578
BMT,Sea Beach,Av U,40.597473,-73.979137,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,8th St,Avenue U,NE,40.596459,-73.978943
BMT,Sea Beach,86th St,40.592721,-73.97823,N,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,8th St,86th St,SE,40.592724,-73.978227
BMT,Broadway Jamaica,121st St,40.700492,-73.828294,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,121st St,Jamaica Ave,SW,40.700254,-73.829332
BMT,Broadway Jamaica,121st St,40.700492,-73.828294,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,121st St,Jamaica Ave,NW,40.70052,-73.829231
BMT,Broadway Jamaica,111th St,40.697418,-73.836345,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Jamaica Blvd,NE,40.696771,-73.837042
BMT,Broadway Jamaica,111th St,40.697418,-73.836345,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Jamaica Blvd,SE,40.697055,-73.837156
BMT,Broadway Jamaica,104th St-102nd St,40.695178,-73.84433,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,104th St,Jamaica Ave,SE,40.695033,-73.843298
BMT,Broadway Jamaica,104th St-102nd St,40.695178,-73.84433,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,104th St,Jamaica Ave,NE,40.695267,-73.843223
BMT,Broadway Jamaica,Woodhaven Blvd,40.693879,-73.851576,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Woodhaven Blvd,Jamaica Ave,SW,40.693451,-73.852556
BMT,Broadway Jamaica,Woodhaven Blvd,40.693879,-73.851576,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Woodhaven Blvd,Jamaica Ave,NW,40.693563,-73.852608
BMT,Broadway Jamaica,Forest Parkway-85th St,40.692435,-73.86001,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,85th St,Jamaica Ave,SE,40.692381,-73.858975
BMT,Broadway Jamaica,Forest Parkway-85th St,40.692435,-73.86001,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,85th St,Jamaica Ave,NE,40.692666,-73.859098
BMT,Broadway Jamaica,Elderts Lane-75th St,40.691324,-73.867139,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,75th St,Jamaica,SE,40.691285,-73.866425
BMT,Broadway Jamaica,Elderts Lane-75th St,40.691324,-73.867139,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,75th St,Jamaica,NW,40.691479,-73.86661
BMT,Broadway Jamaica,Cypress Hills,40.689941,-73.87255,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Autumn Ave,Jamaica Ave,SE,40.690142,-73.87163
BMT,Broadway Jamaica,Cypress Hills,40.689941,-73.87255,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Crescent St,Jamaica Ave,SE,40.68947,-73.873449
BMT,Broadway Jamaica,Cypress Hills,40.689941,-73.87255,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Crescent St,Jamaica Ave,NE,40.689596,-73.873555
BMT,Broadway Jamaica,Crescent St,40.683194,-73.873785,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pine St,Fulton St,SE,40.683478,-73.872548
BMT,Broadway Jamaica,Crescent St,40.683194,-73.873785,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pine St,Fulton St,NE,40.683588,-73.8726
BMT,Broadway Jamaica,Norwood Av,40.68141,-73.880039,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Norwood Ave,Fulton St,NE,40.681757,-73.879007
BMT,Broadway Jamaica,Norwood Av,40.68141,-73.880039,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Norwood,,SE,40.681643,-73.878984
BMT,Broadway Jamaica,Cleveland St,40.679947,-73.884639,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Cleveland St,Fulton St,NW,40.679656,-73.885731
BMT,Broadway Jamaica,Cleveland St,40.679947,-73.884639,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Cleveland,,SW,40.67953,-73.885716
BMT,Broadway Jamaica,Van Siclen Av,40.678024,-73.891688,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Fulton St,SW,40.677847,-73.891711
BMT,Broadway Jamaica,Van Siclen Av,40.678024,-73.891688,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen,,SE,40.677864,-73.891566
BMT,Broadway Jamaica,Alabama Av,40.676992,-73.898654,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,,,,40.676755,-73.899834
BMT,Broadway Jamaica,Alabama Av,40.676992,-73.898654,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,,,,40.676803,-73.89951
BMT,Broadway Jamaica,Chauncey St,40.682893,-73.910456,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Moffat St,Broadway,NW,40.683313,-73.911334
BMT,Broadway Jamaica,Chauncey St,40.682893,-73.910456,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Moffat St,Broadway,SW,40.683417,-73.911242
BMT,Broadway Jamaica,Halsey St,40.68637,-73.916559,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Weirfield St,Broadway,SE,40.685847,-73.915807
BMT,Broadway Jamaica,Halsey St,40.68637,-73.916559,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Weirfield St,Broadway,NE,40.685946,-73.915713
BMT,Broadway Jamaica,Gates Av,40.68963,-73.92227,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Linden St,Broadway,SW,40.69003,-73.923161
BMT,Broadway Jamaica,Gates Av,40.68963,-73.92227,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Linden St,Broadway,NW,40.690142,-73.923062
BMT,Broadway Jamaica,Kosciusko St,40.693342,-73.928814,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Patchen Ave,Broadway,NE,40.69293,-73.928257
BMT,Broadway Jamaica,Kosciusko St,40.693342,-73.928814,J,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Patchen Ave,Broadway,SW,40.692929,-73.92794
BMT,Broadway Jamaica,Myrtle Av,40.697207,-73.935657,J,M,Z,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Myrtle Ave,NW,40.696893,-73.935461
BMT,Broadway Jamaica,Myrtle Av,40.697207,-73.935657,J,M,Z,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Myrtle Ave,SW,40.696987,-73.935559
BMT,Broadway Jamaica,Flushing Av,40.70026,-73.941126,J,M,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Flushing Ave,Broadway,SE,40.700629,-73.941995
BMT,Broadway Jamaica,Flushing Av,40.70026,-73.941126,J,M,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.70072,-73.941672
BMT,Broadway Jamaica,Flushing Av,40.70026,-73.941126,J,M,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Flushing Ave,Broadway,SE,40.700634,-73.942173
BMT,Broadway Jamaica,Flushing Av,40.70026,-73.941126,J,M,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Flushing Ave,Broadway,NW,40.700847,-73.94171
BMT,Broadway Jamaica,Lorimer St,40.703869,-73.947408,J,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Moore St,Broadway,SW,40.704279,-73.948324
BMT,Broadway Jamaica,Lorimer St,40.703869,-73.947408,J,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Moore St,Broadway,NW,40.704396,-73.948223
BMT,Broadway Jamaica,Hewes St,40.70687,-73.953431,J,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hooper St,Broadway,SW,40.707117,-73.95437
BMT,Broadway Jamaica,Hewes St,40.70687,-73.953431,J,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hooper St,Broadway,NW,40.707248,-73.954326
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,FALSE,Marcy Ave,Broadway,NW,40.708565,-73.957893
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Marcy Ave,Broadway,NE,40.708409,-73.957689
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Marcy Ave,Broadway,NW,40.708529,-73.958048
BMT,Nassau,Essex St,40.718315,-73.987437,F,J,M,Z,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Norfolk St,Delancey St,NE,40.71841,-73.987181
BMT,Nassau,Essex St,40.718315,-73.987437,F,J,M,Z,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Norfolk St,Delancey St,NW,40.718551,-73.98765
BMT,Nassau,Bowery,40.72028,-73.993915,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bowery,Delancey St,SE,40.719977,-73.993576
BMT,Nassau,Bowery,40.72028,-73.993915,J,Z,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bowery,Delencey St,NE,40.720291,-73.993411
BMT,Myrtle,Metropolitan Av,40.711396,-73.889601,M,,,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,69th St,Metropolitan Ave,SW,40.712082,-73.88954
BMT,Myrtle,Fresh Pond Rd,40.706186,-73.895877,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Fresh Pond Rd,67th Ave,SE,40.705955,-73.896232
BMT,Myrtle,Fresh Pond Rd,40.706186,-73.895877,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,62nd St,67th Ave,SE,40.706115,-73.895559
BMT,Myrtle,Forest Av,40.704423,-73.903077,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Forest Ave,Myrtle Ave,NW,40.704729,-73.902243
BMT,Myrtle,Forest Av,40.704423,-73.903077,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Forest Ave,Myrtle Ave,SW,40.704443,-73.902389
BMT,Myrtle,Seneca Av,40.702762,-73.90774,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Myrtle Ave,Seneca Ave,NE,40.70278,-73.907515
BMT,Myrtle,Seneca Av,40.702762,-73.90774,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Myrtle Ave,Seneca Ave,SW,40.702835,-73.907894
BMT,Myrtle,Knickerbocker Av,40.698664,-73.919711,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Myrtle Ave,Greene Ave,NW,40.698717,-73.918655
BMT,Myrtle,Knickerbocker Av,40.698664,-73.919711,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Myrtle Ave,Greene Ave,SE,40.698913,-73.918771
BMT,Myrtle,Central Av,40.697857,-73.927397,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Myrtle Ave,Cedar St,NW,40.698033,-73.926371
BMT,Myrtle,Central Av,40.697857,-73.927397,M,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Myrtle Ave,Cedar St,SW,40.697876,-73.926525
BMT,Canarsie,8th Av,40.739777,-74.002578,A,C,E,L,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,14th St,SE,40.73955,-74.002389
BMT,Canarsie,8th Av,40.739777,-74.002578,A,C,E,L,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,14th St,SW,40.739734,-74.002804
BMT,Canarsie,3rd Av,40.732849,-73.986122,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,3rd Ave,14th St,SE,40.732961,-73.986657
BMT,Canarsie,3rd Av,40.732849,-73.986122,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,3rd Ave,14th St,SE,40.733134,-73.987041
BMT,Canarsie,1st Av,40.730953,-73.981628,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,1st Ave,14th St,NE,40.731397,-73.982112
BMT,Canarsie,1st Av,40.730953,-73.981628,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,1st Ave,14th St,NE,40.731483,-73.982145
BMT,Canarsie,Bedford Av,40.717304,-73.956872,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bedford Ave,N 7th St,SE,40.717714,-73.957695
BMT,Canarsie,Bedford Av,40.717304,-73.956872,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bedford Ave,N 7th St,NE,40.717821,-73.957568
BMT,Canarsie,Graham Av,40.714565,-73.944053,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Graham Ave,Metropolitan Ave,NE,40.714661,-73.944287
BMT,Canarsie,Graham Av,40.714565,-73.944053,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Graham Ave,Metropolitan Ave,SW,40.714613,-73.94457
BMT,Canarsie,Grand St,40.711926,-73.94067,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Grand St,Bushwick Ave,SW,40.711912,-73.940433
BMT,Canarsie,Grand St,40.711926,-73.94067,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Grand St,Bushwick Ave,SE,40.712182,-73.940668
BMT,Canarsie,Montrose Av,40.707739,-73.93985,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Montrose Ave,Bushwick Ave,SW,40.707723,-73.940044
BMT,Canarsie,Montrose Av,40.707739,-73.93985,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Montrose Ave,Bushwick Ave,NW,40.707819,-73.939749
BMT,Canarsie,Morgan Av,40.706152,-73.933147,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Morgan Ave,Harrison Pl,SE,40.706317,-73.931696
BMT,Canarsie,Morgan Av,40.706152,-73.933147,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Morgan Ave,Harrison Pl,SW,40.70621,-73.93196
BMT,Canarsie,Jefferson St,40.706607,-73.922913,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Starr St,Wyckoff Ave,SE,40.705943,-73.922036
BMT,Canarsie,Jefferson St,40.706607,-73.922913,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Starr St,Wyckoff Ave,NE,40.706228,-73.92191
BMT,Canarsie,Jefferson St,40.706607,-73.922913,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Starr St,Wyckoff Ave,SW,40.706082,-73.9223
BMT,Canarsie,Jefferson St,40.706607,-73.922913,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Starr St,Wyckoff Ave,NW,40.70632,-73.922062
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,DeKalb Ave,Wycoff Ave,SE,40.704153,-73.919273
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,DeKalb Ave,Wycoff Ave,NE,40.704379,-73.919026
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,DeKalb Ave,Wycoff Ave,SW,40.704326,-73.919452
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,DeKalb Ave,Wycoff Ave,NW,40.70448,-73.919184
BMT,Canarsie,Halsey St,40.695602,-73.904084,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,George St,Wycoff Ave,NE,40.696094,-73.90443
BMT,Canarsie,Halsey St,40.695602,-73.904084,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,George St,Wycoff Ave,NW,40.696069,-73.904665
BMT,Canarsie,Wilson Av,40.688764,-73.904046,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,LIRR Bay Ridge Divide,Wilson Ave,NW,40.688805,-73.904741
BMT,Canarsie,Bushwick Av,40.682829,-73.905249,L,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,De Sales Pl,Bushwick Ave,NW,40.6823,-73.905149
BMT,Canarsie,Atlantic Av,40.675345,-73.903097,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Snediker Ave,New York Ave,SW,40.67538,-73.902612
BMT,Canarsie,Atlantic Av,40.675345,-73.903097,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Snediker Ave,New York Ave,SW,40.675609,-73.902767
BMT,Canarsie,Atlantic Av,40.675345,-73.903097,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Snediker Ave,New York Ave,SW,40.675607,-73.902672
BMT,Canarsie,Sutter Av,40.669367,-73.901975,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,LIRR Bay Ridge Divide,Sutter Ave,NW,40.668527,-73.901765
BMT,Canarsie,Livonia Av,40.664038,-73.900571,L,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Van Sinderen Ave,Livonia Ave,SW,40.663742,-73.900522
BMT,Canarsie,New Lots Av,40.658733,-73.899232,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Sinderen Ave,New Lots Ave,SW,40.659041,-73.899409
BMT,Canarsie,East 105th St,40.650573,-73.899485,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Turnbull Ave,E 105th St,SE,40.651462,-73.899191
BMT,Canarsie,East 105th St,40.650573,-73.899485,L,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Turnbull Ave,E 105th St,SE,40.651278,-73.898862
BMT,Canarsie,Canarsie - Rockaway Parkway,40.646654,-73.90185,L,,,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Glenwood Rd,Rockaway Pkwy,NW,40.64556,-73.902524
BMT,Canarsie,Canarsie - Rockaway Parkway,40.645757,-73.9025,L,,,,,,,,,,,Door,YES,,NO,NONE,,TRUE,,TRUE,Glenwood Rd,Rockaway Pkwy,NW,40.64556,-73.902524
BMT,Canarsie,Canarsie - Rockaway Parkway,40.645957,-73.901741,L,,,,,,,,,,,Walkway,YES,,NO,NONE,,TRUE,,TRUE,Glenwood Rd,E 98th St,NW,40.645806,-73.901414
BMT,Franklin,Franklin Av,40.680596,-73.955827,A,FS,,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Franklin Ave,Fulton St,SW,40.681051,-73.95584
BMT,Franklin,Park Place,40.674772,-73.957624,FS,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Franklin Ave,Park Pl,NW,40.674516,-73.957932
BMT,Franklin,Park Place,40.674772,-73.957624,FS,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Franklin Ave,Park Pl,NW,40.675208,-73.957591
BMT,Franklin,Botanic Gardens,40.670343,-73.959245,FS,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Franklin Ave,Eastern Pkwy,SW,40.670588,-73.959093
IND,8 Avenue,Inwood - 207th St,40.868072,-73.919899,A,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Broadway,207th St,NW,40.867892,-73.921234
IND,8 Avenue,Inwood - 207th St,40.868072,-73.919899,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,207th St,SW,40.867799,-73.921431
IND,8 Avenue,Inwood - 207th St,40.868072,-73.919899,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,207th St,SE,40.867512,-73.92133
IND,8 Avenue,Inwood - 207th St,40.868072,-73.919899,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,207th St,SW,40.867653,-73.920949
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Dyckman St,NW,40.865758,-73.927266
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Dyckman St,NW,40.865744,-73.927172
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Dyckman St,SW,40.865362,-73.928059
IND,8 Avenue,190th St,40.859022,-73.93418,A,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Fort Washington Ave,192nd St,NW,40.858899,-73.932702
IND,8 Avenue,190th St,40.859022,-73.93418,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Fort Washington Ave,Margaret Corbin Dr,SE,40.858886,-73.934226
IND,8 Avenue,181st St,40.851695,-73.937969,A,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Fort Washington Ave,185th St,SE,40.852759,-73.936636
IND,8 Avenue,181st St,40.851695,-73.937969,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Fort Washington Ave,181st St,SE,40.850752,-73.93813
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Fort Washington Ave,177th St,NE,40.847946,-73.939375
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Fort Washington Ave,177th St,SW,40.847809,-73.939807
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Fort Washington Ave,177th St,NW,40.847944,-73.939679
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Fort Washington Ave,177th St,NE,40.847859,-73.939303
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Fort Washington Ave,177th St,NW,40.847992,-73.939564
IND,8 Avenue,168th St - Washington Heights,40.840719,-73.939561,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Saint Nicholas Ave,168th St,SE,40.840741,-73.939252
IND,8 Avenue,163rd St - Amsterdam Av,40.836013,-73.939892,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Amsterdam Ave,161st St,SE,40.835192,-73.940146
IND,8 Avenue,163rd St - Amsterdam Av,40.836013,-73.939892,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Saint Nicholas Ave,161st St,NE,40.835278,-73.939625
IND,8 Avenue,163rd St - Amsterdam Av,40.836013,-73.939892,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Saint Nicholas Ave,162nd St,SW,40.835936,-73.940137
IND,8 Avenue,155th St,40.830518,-73.941514,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,155th St,SW,40.830744,-73.941597
IND,8 Avenue,155th St,40.830518,-73.941514,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,155th St,SW,40.830854,-73.941638
IND,8 Avenue,155th St,40.830518,-73.941514,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,155th St,NW,40.831106,-73.94155
IND,8 Avenue,155th St,40.830518,-73.941514,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,155th St,NW,40.831139,-73.941447
IND,8 Avenue,155th St,40.830518,-73.941514,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,155th St,NE,40.831031,-73.941149
IND,8 Avenue,155th St,40.830518,-73.941514,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,155th St,NE,40.830921,-73.9411
IND,8 Avenue,145th St,40.824783,-73.944216,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,145th St,SE,40.823791,-73.944726
IND,8 Avenue,145th St,40.824783,-73.944216,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,145th St,SW,40.823915,-73.945027
IND,8 Avenue,145th St,40.824783,-73.944216,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,145th St,NW,40.824244,-73.944931
IND,8 Avenue,145th St,40.824783,-73.944216,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,145th St,NE,40.824028,-73.944409
IND,8 Avenue,135th St,40.817894,-73.947649,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Saint Nicholas Ave,135th St,SW,40.817125,-73.948225
IND,8 Avenue,125th St,40.811109,-73.952343,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,125th St,NE,40.810879,-73.952318
IND,8 Avenue,125th St,40.811109,-73.952343,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,125th St,SE,40.810526,-73.952583
IND,8 Avenue,125th St,40.811109,-73.952343,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,125th St,NW,40.811007,-73.952615
IND,8 Avenue,125th St,40.811109,-73.952343,A,B,C,D,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,125th St,SW,40.810636,-73.952886
IND,8 Avenue,116th St,40.805085,-73.954882,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Frederick Douglass Blvd,116th St,SW,40.804312,-73.955643
IND,8 Avenue,116th St,40.805085,-73.954882,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Frederick Douglass Blvd,116th St,NW,40.804639,-73.955559
IND,8 Avenue,Cathedral Parkway-110th St,40.800605,-73.958158,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Frederick Douglass Blvd,109th St,SW,40.800002,-73.958882
IND,8 Avenue,Cathedral Parkway-110th St,40.800605,-73.958158,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Frederick Douglass Blvd,109th St,NE,40.800002,-73.958448
IND,8 Avenue,103rd St,40.796092,-73.961454,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,103rd St,NW,40.796263,-73.96161
IND,8 Avenue,96th St,40.791646,-73.964699,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,96th St,SW,40.791642,-73.965009
IND,8 Avenue,86th St,40.785868,-73.968916,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,86th St,SW,40.785195,-73.969596
IND,8 Avenue,86th St,40.785868,-73.968916,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,86th St,SW,40.785282,-73.969662
IND,8 Avenue,86th St,40.785868,-73.968916,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,86th St,NW,40.78548,-73.969458
IND,8 Avenue,81st St - Museum of Natural History,40.781433,-73.972143,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,81st St,SW,40.781976,-73.972027
IND,8 Avenue,81st St - Museum of Natural History,40.781433,-73.972143,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,81st St,NW,40.782254,-73.971741
IND,8 Avenue,72nd St,40.775594,-73.97641,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,72nd St,SW,40.776217,-73.976247
IND,8 Avenue,72nd St,40.775594,-73.97641,B,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Central Park West,72nd St,NW,40.776431,-73.975977
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,57th St,NW,40.766944,-73.982941
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,57th St,SE,40.766488,-73.982708
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NW,40.767667,-73.982408
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NW,40.767677,-73.982526
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NW,40.767659,-73.982542
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,57th St,NE,40.766725,-73.982563
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NW,40.767643,-73.982563
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NW,40.766745,-73.983408
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NW,40.766416,-73.983301
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,58th St,NE,40.767367,-73.982137
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Easement,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,49th St,NW,40.761968,-73.986662
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Easement,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,50th St,SW,40.762312,-73.986378
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Easement,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,50th St,NW,40.762452,-73.986299
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,49th St,NW,40.761968,-73.986662
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,50th St,SW,40.762312,-73.986378
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,49th St,SW,40.761741,-73.986924
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Southbound Only,FALSE,8th Ave,49th St,SW,40.761691,-73.986805
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Easement,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,42nd St,NE,40.757286,-73.989582
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Easement,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,40th St,SW,40.755989,-73.991049
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,40th St,SE,40.755801,-73.990576
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,40th St,NW,40.756101,-73.990887
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.751996,-73.993327
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,34th St,SE,40.75202,-73.993352
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,34th St,NE,40.752161,-73.993109
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,8th Ave,23rd St,SE,40.745102,-73.998365
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,8th Ave,23rd St,NE,40.745299,-73.998145
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,16th St,SW,40.740938,-74.001919
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,16th St,NW,40.741235,-74.001715
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,16th St,SE,40.740944,-74.001402
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,16th St,NE,40.741046,-74.001284
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,8th Ave,14th St,NW,40.739949,-74.002655
IND,8 Avenue,West 4th St,40.732338,-74.000495,A,B,C,D,E,F,M,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.731086,-74.001209
IND,8 Avenue,West 4th St,40.732338,-74.000495,A,B,C,D,E,F,M,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,West 3rd St,NW,40.731133,-74.001534
IND,8 Avenue,West 4th St,40.732338,-74.000495,A,B,C,D,E,F,M,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,West 3rd St,NE,40.731091,-74.001127
IND,8 Avenue,Spring St,40.726227,-74.003739,C,E,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,Spring St,NE,40.725583,-74.003747
IND,8 Avenue,Canal St,40.720824,-74.005229,A,C,E,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lispenard St,West Broadway,NE,40.720704,-74.004999
IND,8 Avenue,Canal St,40.720824,-74.005229,A,C,E,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,West Broadway,NW,40.720758,-74.005399
IND,8 Avenue,Canal St,40.720824,-74.005229,A,C,E,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,St. John La,Laight St,SE,40.721651,-74.005634
IND,8 Avenue,Canal St,40.720824,-74.005229,A,C,E,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Thompson St,Canal St,NE,40.721967,-74.005133
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Church St,Murray St,SW,40.713753,-74.009046
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,Park Pl,NE,40.713333,-74.009033
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,Barclay St,SW,40.712681,-74.009891
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,Park Pl,SW,40.713286,-74.009511
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,Park Pl,SE,40.713036,-74.009245
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,Park Pl,NW,40.713435,-74.009292
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,,,40.711793,-74.010165
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,,,40.71171,-74.010204
IND,8 Avenue,World Trade Center,40.712582,-74.009781,A,C,E,2,3,,,,,,,Stair,YES,,YES,FULL,,TRUE,Construction,TRUE,Church St,Vesey St,NW,40.712258,-74.010306
IND,8 Avenue,Broadway-Nassau,40.710197,-74.007691,A,C,J,Z,2,3,4,5,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Fulton St,NE,40.710692,-74.008428
IND,8 Avenue,Broadway-Nassau,40.710197,-74.007691,A,C,J,Z,2,3,4,5,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Fulton St,SE,40.710555,-74.008632
IND,8 Avenue,High St,40.699337,-73.990531,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Cadman Plaza West,Cranberry St,NW,40.699405,-73.991413
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Jay St,Willoughby St,NW,40.692335,-73.987446
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Jay St,Willoughby St,NW,40.692393,-73.987443
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Jay St,Willoughby St,NW,40.692336,-73.987494
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Jay St,Willoughby St,NW,40.692394,-73.987491
IND,Fulton,Hoyt & Schermerhorn,40.688484,-73.985001,A,C,G,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hoyt St,Schermerhorn St,NE,40.689085,-73.985941
IND,Fulton,Hoyt & Schermerhorn,40.688484,-73.985001,A,C,G,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bond St,Schermerhorn St,NW,40.688396,-73.984216
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Oxford St,Fulton Ave,NE,40.685789,-73.97276
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Oxford St,Fulton Ave,SW,40.685559,-73.9729
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Oxford St,Fulton Ave,NE,40.685969,-73.973021
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Oxford St,Fulton Ave,SW,40.6857,-73.973526
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Oxford St,Fulton Ave,NE,40.686077,-73.973308
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Portland Ave,Fulton Ave,SE,40.686041,-73.974278
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Portland Ave,Fulton Ave,NE,40.686409,-73.974347
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Portland Ave,Fulton Ave,SW,40.686119,-73.974517
IND,Fulton,Lafayette Av,40.686113,-73.973946,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Portland Ave,Fulton Ave,NW,40.686553,-73.974606
IND,Fulton,Clinton & Washington Avs,40.683263,-73.965838,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Clinton Ave,Fulton Ave,NE,40.683653,-73.966797
IND,Fulton,Clinton & Washington Avs,40.683263,-73.965838,C,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Clinton Ave,Fulton Ave,NW,40.683695,-73.967069
IND,Fulton,Nostrand Av,40.680438,-73.950426,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Nostrand Ave,Fulton St,NE,40.680547,-73.949515
IND,Fulton,Nostrand Av,40.680438,-73.950426,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Nostrand Ave,Fulton St,NW,40.680572,-73.949753
IND,Fulton,Kingston-Throop,40.679921,-73.940858,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Throop Ave,Fulton St,NE,40.68001,-73.940247
IND,Fulton,Kingston-Throop,40.679921,-73.940858,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Throop Ave,Fulton St,NW,40.680057,-73.940483
IND,Fulton,Utica Av,40.679364,-73.930729,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Utica Ave,Fulton St,SW,40.679095,-73.930044
IND,Fulton,Utica Av,40.679364,-73.930729,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Utica Ave,Fulton St,NW,40.679487,-73.929725
IND,Fulton,Ralph Av,40.678822,-73.920786,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ralph Ave,Fulton St,SE,40.678702,-73.921552
IND,Fulton,Ralph Av,40.678822,-73.920786,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ralph Ave,Fulton St,NE,40.678984,-73.921442
IND,Fulton,Ralph Av,40.678822,-73.920786,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ralph Ave,Mac Dougal St,NE,40.679164,-73.92187
IND,Fulton,Rockaway Av,40.67834,-73.911946,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hopkinson Ave,Fulton St,NW,40.678594,-73.913732
IND,Fulton,Rockaway Av,40.67834,-73.911946,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hopkinson Ave,Fulton St,SW,40.678317,-73.913651
IND,Fulton,Broadway Junction-East New York,40.678334,-73.905316,A,C,J,L,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Fulton St,Van Sinderen Ave,SW,40.678334,-73.903922
IND,Fulton,Liberty Av,40.674542,-73.896548,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pennsylvania Ave,Liberty Ave,SW,40.674242,-73.896645
IND,Fulton,Liberty Av,40.674542,-73.896548,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pennsylvania Ave,Liberty Ave,SE,40.674268,-73.896312
IND,Fulton,Liberty Av,40.674542,-73.896548,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pennsylvania Ave,Liberty Ave,NW,40.67443,-73.896776
IND,Fulton,Liberty Av,40.674542,-73.896548,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pennsylvania Ave,Liberty Ave,NE,40.674567,-73.896364
IND,Fulton,Van Siclen Av,40.67271,-73.890358,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Pitkin Ave,SE,40.672479,-73.890164
IND,Fulton,Van Siclen Av,40.67271,-73.890358,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Pitkin Ave,NE,40.67285,-73.890257
IND,Fulton,Van Siclen Av,40.67271,-73.890358,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Pitkin Ave,SW,40.672545,-73.890381
IND,Fulton,Van Siclen Av,40.67271,-73.890358,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Pitkin Ave,NW,40.672771,-73.890584
IND,Fulton,Shepherd Av,40.67413,-73.88075,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Shepherd Ave,Pitkin Ave,SE,40.67397,-73.880667
IND,Fulton,Shepherd Av,40.67413,-73.88075,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Shepherd Ave,Pitkin Ave,NE,40.674303,-73.880749
IND,Fulton,Shepherd Av,40.67413,-73.88075,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Shepherd Ave,Pitkin Ave,SW,40.674234,-73.880927
IND,Fulton,Shepherd Av,40.67413,-73.88075,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Shepherd Ave,Pitkin Ave,NW,40.673989,-73.880869
IND,Fulton,Euclid Av,40.675377,-73.872106,A,C,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Pitkin,,NE,40.675554,-73.871875
IND,Fulton,Euclid Av,40.675377,-73.872106,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Euclid Ave,Pitkin Ave,SE,40.675301,-73.871801
IND,Fulton,Euclid Av,40.675377,-73.872106,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Euclid Ave,Pitkin Ave,NE,40.675533,-73.871742
IND,Fulton,Euclid Av,40.675377,-73.872106,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Euclid Ave,Pitkin Ave,SW,40.675274,-73.872115
IND,Fulton,Euclid Av,40.675377,-73.872106,A,C,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Euclid Ave,Pitkin Ave,NW,40.675478,-73.872109
IND,Liberty,Grant Av,40.677044,-73.86505,A,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Grant Ave,Pitkin Ave,NE,40.676735,-73.865491
IND,Liberty,80th St-Hudson St,40.679371,-73.858992,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,80th St,Liberty Ave,SE,40.679371,-73.858024
IND,Liberty,80th St-Hudson St,40.679371,-73.858992,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,80th St,Liberty Ave,NE,40.679498,-73.858049
IND,Liberty,88th St-Boyd Av,40.679843,-73.85147,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,88th St,Liberty Ave,SE,40.679828,-73.850401
IND,Liberty,88th St-Boyd Av,40.679843,-73.85147,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,88th St,Liberty Ave,NE,40.679973,-73.850413
IND,Liberty,Rockaway Blvd,40.680429,-73.843853,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,94th St,Liberty Ave,SW,40.680158,-73.845131
IND,Liberty,Rockaway Blvd,40.680429,-73.843853,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,94th St,Liberty Ave,NW,40.680305,-73.845159
IND,Liberty,104th St-Oxford Av,40.681711,-73.837683,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,104th St,Liberty Ave,SE,40.681994,-73.836742
IND,Liberty,104th St-Oxford Av,40.681711,-73.837683,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,104th St,Liberty Ave,NE,40.68212,-73.836836
IND,Liberty,111th St-Greenwood Av,40.684331,-73.832163,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Liberty Ave,SE,40.684732,-73.830898
IND,Liberty,111th St-Greenwood Av,40.684331,-73.832163,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Liberty Ave,NE,40.684874,-73.830943
IND,Liberty,Lefferts Blvd,40.685951,-73.825798,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lefferts Blvd,Liberty Ave,SW,40.686103,-73.824314
IND,Liberty,Lefferts Blvd,40.685951,-73.825798,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lefferts Blvd,Liberty Ave,NW,40.686234,-73.824378
IND,Rockaway,Aqueduct-North Conduit Av,40.668234,-73.834058,A,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Cohancy St,North Conduit Ave,NE,40.667018,-73.83348
IND,Rockaway,Howard Beach,40.660476,-73.830301,A,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,103rd St,159th Ave,NE,40.660465,-73.830518
IND,Rockaway,Howard Beach,40.660476,-73.830301,A,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,103rd St,159th Ave,NE,40.660486,-73.830453
IND,Rockaway,Howard Beach,40.660476,-73.830301,A,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,103rd St,159th Ave,SE,40.660372,-73.830369
IND,Rockaway,Broad Channel,40.608382,-73.815925,A,H,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,West Rd,Noel Rd,NE,40.608007,-73.815967
IND,Rockaway,Beach 90th St,40.588034,-73.813641,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 89th St,Rockaway Frwy,NW,40.58808,-73.813907
IND,Rockaway,Beach 90th St,40.588034,-73.813641,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 89th St,Rockaway Frwy,SW,40.587929,-73.813755
IND,Rockaway,Beach 90th St,40.588034,-73.813641,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 90th St,Rockaway Frwy,NE,40.587999,-73.814117
IND,Rockaway,Beach 90th St,40.588034,-73.813641,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 90th St,Rockaway Frwy,SE,40.587814,-73.814037
IND,Rockaway,Beach 98th St,40.585307,-73.820558,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 98th St,Rockaway Frwy,NW,40.58554,-73.820345
IND,Rockaway,Beach 98th St,40.585307,-73.820558,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 98th St,Rockaway Frwy,SW,40.585342,-73.820249
IND,Rockaway,Beach 98th St,40.585307,-73.820558,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 98th St,Rockaway Frwy,SW,40.585208,-73.820595
IND,Rockaway,Beach 105th St,40.583209,-73.827559,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 106th St,Rockaway Frwy,NW,40.583187,-73.827388
IND,Rockaway,Beach 105th St,40.583209,-73.827559,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 106th St,Rockaway Frwy,SW,40.583369,-73.827474
IND,Rockaway,Beach 105th St,40.583209,-73.827559,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 106th St,Rockaway Frwy,SW,40.583248,-73.827163
IND,Rockaway,Beach 105th St,40.583209,-73.827559,H,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 106th St,Rockaway Frwy,NW,40.583435,-73.827232
IND,Rockaway,Rockaway Park-Beach 116th,40.580903,-73.835592,H,,,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Beach 116th St,Wainwright Ct,NE,40.58051,-73.837418
IND,Rockaway,Beach 67th St,40.590927,-73.796924,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 67th St,Rockaway Frwy,NW,40.590744,-73.797789
IND,Rockaway,Beach 67th St,40.590927,-73.796924,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 67th St,Rockaway Frwy,SW,40.590901,-73.796609
IND,Rockaway,Beach 67th St,40.590927,-73.796924,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 67th St,Rockaway Frwy,NW,40.591123,-73.79646
IND,Rockaway,Beach 67th St,40.590927,-73.796924,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 67th St,Rockaway Frwy,SW,40.590951,-73.796176
IND,Rockaway,Beach 60th St,40.592374,-73.788522,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 60th St,Rockaway Frwy,SE,40.592269,-73.789578
IND,Rockaway,Beach 60th St,40.592374,-73.788522,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 59th Sr,Rockaway Frwy,NW,40.59249,-73.789161
IND,Rockaway,Beach 60th St,40.592374,-73.788522,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 59th Sr,Rockaway Frwy,SW,40.592292,-73.789153
IND,Rockaway,Beach 60th St,40.592374,-73.788522,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 60th St,Rockaway Frwy,NE,40.592478,-73.78941
IND,Rockaway,Beach 44th St,40.592943,-73.776013,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 44th St,Edgemere Ave,NW,40.593106,-73.775743
IND,Rockaway,Beach 44th St,40.592943,-73.776013,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 45th St,Edgemere Ave,SW,40.59286,-73.776109
IND,Rockaway,Beach 44th St,40.592943,-73.776013,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 45th St,Edgemere Ave,NE,40.593079,-73.775985
IND,Rockaway,Beach 44th St,40.592943,-73.776013,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 44th St,Edgemere Ave,SW,40.592897,-73.775715
IND,Rockaway,Beach 36th St,40.595398,-73.768175,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 35th,Rockaway Frwy,NW,40.595617,-73.768032
IND,Rockaway,Beach 36th St,40.595398,-73.768175,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 36th,Edgemere Ave,SE,40.595297,-73.768206
IND,Rockaway,Beach 36th St,40.595398,-73.768175,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 36th,Edgemere Ave,NE,40.595445,-73.768313
IND,Rockaway,Beach 36th St,40.595398,-73.768175,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 35th,Rockaway Frwy,SW,40.595469,-73.767919
IND,Rockaway,Beach 25th St,40.600066,-73.761353,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 25th St,Rockaway Frwy,NW,40.599708,-73.761959
IND,Rockaway,Beach 25th St,40.600066,-73.761353,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 25th St,Rockaway Frwy,SW,40.599427,-73.761807
IND,Rockaway,Beach 25th St,40.600066,-73.761353,A,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Beach 25th St,Rockaway Frwy,SW,40.59967,-73.761596
IND,Rockaway,Far Rockaway-Mott Av,40.603995,-73.755405,A,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Beach 22nd St,Mott Ave,SW,40.604657,-73.754178
IND,Concourse,Norwood-205th St,40.874811,-73.878855,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bainbridge Ave,205th St,SE,40.874902,-73.879267
IND,Concourse,Norwood-205th St,40.874811,-73.878855,D,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Bainbridge Ave,205th St,NE,40.875087,-73.87961
IND,Concourse,Bedford Park Blvd,40.873244,-73.887138,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,Bedofrd Park Blvd,NE,40.872135,-73.887459
IND,Concourse,Bedford Park Blvd,40.873244,-73.887138,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,Bedford Park Blvd,SE,40.872198,-73.887788
IND,Concourse,Bedford Park Blvd,40.873244,-73.887138,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,Bedford Park Blvd,NW,40.872468,-73.887894
IND,Concourse,Kingsbridge Rd,40.866978,-73.893509,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,in planning,TRUE,Grand Concourse,196th St,NW,40.868114,-73.893195
IND,Concourse,Kingsbridge Rd,40.866978,-73.893509,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,in planning,TRUE,Grand Concourse,196th St,SE,40.867688,-73.892739
IND,Concourse,Fordham Rd,40.861296,-73.897749,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,188th St,SW,40.861401,-73.898098
IND,Concourse,Fordham Rd,40.861296,-73.897749,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,188th St,SE,40.861147,-73.897424
IND,Concourse,Fordham Rd,40.861296,-73.897749,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,188th St,NW,40.861592,-73.898019
IND,Concourse,Fordham Rd,40.861296,-73.897749,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,188th St,NE,40.861322,-73.897355
IND,Concourse,182nd-183rd Sts,40.856093,-73.900741,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,182nd St,SW,40.856038,-73.90114
IND,Concourse,182nd-183rd Sts,40.856093,-73.900741,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,182nd St,SE,40.855864,-73.900484
IND,Concourse,182nd-183rd Sts,40.856093,-73.900741,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,182nd St,NW,40.856237,-73.901115
IND,Concourse,182nd-183rd Sts,40.856093,-73.900741,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,182nd St,NE,40.856039,-73.900449
IND,Concourse,Tremont Av,40.85041,-73.905227,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,Echo Pl,SW,40.849864,-73.906084
IND,Concourse,Tremont Av,40.85041,-73.905227,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,Echo Pl,NW,40.850124,-73.905841
IND,Concourse,Tremont Av,40.85041,-73.905227,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,Echo Pl,NE,40.849968,-73.905226
IND,Concourse,174-175th Sts,40.8459,-73.910136,B,D,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,174th St,NW,40.844933,-73.911212
IND,Concourse,174-175th Sts,40.8459,-73.910136,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,174th St,NE,40.844939,-73.910885
IND,Concourse,170th St,40.839306,-73.9134,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,171st St,NE,40.839751,-73.912743
IND,Concourse,170th St,40.839306,-73.9134,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,171st St,NW,40.840097,-73.913333
IND,Concourse,167th St,40.833769,-73.918432,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,167th St,SW,40.834466,-73.918155
IND,Concourse,167th St,40.833769,-73.918432,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,167th St,SE,40.834286,-73.917562
IND,Concourse,167th St,40.833769,-73.918432,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,167th St,NW,40.834788,-73.917838
IND,Concourse,167th St,40.833769,-73.918432,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,167th St,NE,40.834609,-73.917245
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,River Ave,161st St,NE,40.82784,-73.925715
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,River Ave,161st St,SE,40.827488,-73.926004
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,River Ave,161st St,SW,40.827471,-73.92625
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,River Ave,161st St,SW,40.827545,-73.926202
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,River Ave,161st St,NE,40.828011,-73.925588
IND,Concourse,155th St,40.830135,-73.938209,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Frederick Douglass Blvd,156th St,NW,40.830313,-73.936687
IND,63rd Street,21st St,40.754203,-73.942836,F,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,21st St,41st Ave,NW,40.754322,-73.942374
IND,63rd Street,21st St,40.754203,-73.942836,F,,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,21st St,41st Ave,NW,40.754131,-73.942491
IND,63rd Street,21st St,40.754203,-73.942836,F,,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,21st St,41st Ave,NW,40.754195,-73.942521
IND,63rd Street,21st St,40.754203,-73.942836,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,21st St,41st Ave,NE,40.754069,-73.942132
IND,63rd Street,21st St,40.754203,-73.942836,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,21st St,41st Ave,NE,40.754255,-73.941935
IND,63rd Street,Roosevelt Island,40.759145,-73.95326,F,,,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,West Rd,Unnamed St,SE,40.759016,-73.953505
IND,63rd Street,Lexington Av,40.764627,-73.96611,F,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,63rd St,NW,40.764884,-73.966514
IND,63rd Street,Lexington Av,40.764627,-73.96611,F,,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,63rd St,NW,40.764874,-73.966431
IND,63rd Street,Lexington Av,40.764627,-73.96611,F,,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,63rd St,NW,40.764881,-73.966411
IND,63rd Street,Lexington Av,40.764627,-73.96611,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,63rd St,SW,40.764721,-73.966552
IND,63rd Street,Lexington Av,40.764627,-73.96611,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,63rd St,NW,40.764866,-73.966422
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,56th St,NE,40.763874,-73.977358
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,57th St,SW,40.764227,-73.97743
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,57th St,SE,40.763998,-73.977264
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,57th St,NW,40.764652,-73.977119
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,57th St,NE,40.764564,-73.97683
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,56th St,SW,40.763482,-73.977974
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,56th St,SE,40.763392,-73.977674
IND,6 Avenue,57th St,40.763972,-73.97745,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,56th St,NW,40.763831,-73.977718
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,49th St,NW,40.759434,-73.981001
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,50th St,SW,40.759896,-73.980706
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,51st St,SW,40.760502,-73.980218
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,50th St,NE,40.759953,-73.980117
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,50th St,NW,40.760125,-73.980499
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,49th St,NE,40.759301,-73.980594
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,49th St,NW,40.759414,-73.981035
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,7,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,6th Ave,49th St,SE,40.758996,-73.980788
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,42nd St,SW,40.754849,-73.984627
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,42nd St,SE,40.754609,-73.983894
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.749615,-73.988159
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,34th St,SW,40.749845,-73.988254
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,34th St,NW,40.750088,-73.988142
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,6th Ave,23rd St,SE,40.742659,-73.992489
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,6th Ave,23rd St,SE,40.742715,-73.992617
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,6th Ave,23rd St,NE,40.74288,-73.992506
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,6th Ave,23rd St,NE,40.742825,-73.992383
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,14th St,SW,40.737423,-73.99722
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,14th St,NW,40.737572,-73.997111
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,14th St,NW,40.737519,-73.99697
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,6th Ave,14th St,SW,40.73737,-73.997079
IND,6 Avenue,Broadway-Lafayette St,40.725297,-73.996204,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Houston St,NE,40.725202,-73.996583
IND,6 Avenue,Broadway-Lafayette St,40.725297,-73.996204,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Houston St,SE,40.725499,-73.996387
IND,6 Avenue,Grand St,40.718267,-73.993753,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Chrystie St,Grand St,NE,40.718155,-73.993452
IND,6 Avenue,Grand St,40.718267,-73.993753,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Chrystie St,Grand St,NE,40.718191,-73.993577
IND,6 Avenue,Grand St,40.718267,-73.993753,B,D,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Chrystie St,Grand St,NW,40.718265,-73.993877
IND,6 Avenue,2nd Av,40.723402,-73.989938,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Allen St,Houston St,SW,40.722807,-73.988942
IND,6 Avenue,2nd Av,40.723402,-73.989938,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,1st Ave,Houston St,NW,40.7232,-73.988863
IND,6 Avenue,Delancey St,40.718611,-73.988114,F,J,M,Z,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Essex St,Delancey St,NW,40.718781,-73.988233
IND,6 Avenue,Delancey St,40.718611,-73.988114,F,J,M,Z,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Essex St,Delancey St,NE,40.718778,-73.987961
IND,6 Avenue,East Broadway,40.713715,-73.990173,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Rutgers St,Madison Ave,NW,40.712727,-73.990126
IND,6 Avenue,York St,40.699743,-73.986885,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jay St,York St,SW,40.70138,-73.986655
IND,6 Avenue,Bergen St,40.686145,-73.990862,F,G,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Smith St,Bergen St,SE,40.686584,-73.990362
IND,6 Avenue,Bergen St,40.686145,-73.990862,F,G,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Smith St,Bergen St,NE,40.686727,-73.990269
IND,6 Avenue,Carroll St,40.680303,-73.995048,F,G,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Smith St,2nd St,NW,40.679299,-73.995698
IND,6 Avenue,Carroll St,40.680303,-73.995048,F,G,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Smith St,2nd St,SE,40.679344,-73.995288
IND,6 Avenue,Smith-9th St,40.67358,-73.995959,F,G,R,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Smith St,9th St,NE,40.674725,-73.997505
IND,6 Avenue,4th Av,40.670272,-73.989779,F,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,10th St,NW,40.669953,-73.989442
IND,6 Avenue,4th Av,40.670272,-73.989779,F,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,10th St,NW,40.67001,-73.989248
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,8th Ave,9th St,NE,40.665835,-73.979007
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,8th Ave,9th St,SE,40.665494,-73.979278
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,8th Ave,9th St,SW,40.665593,-73.979509
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,8th Ave,9th St,NW,40.665993,-73.979181
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,7th Ave,9th St,SE,40.666649,-73.981624
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,7th Ave,9th St,NE,40.667004,-73.981331
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,7th Ave,9th St,SW,40.666751,-73.981851
IND,6 Avenue,7th Av,40.666271,-73.980305,F,,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,TRUE,7th Ave,9th St,NW,40.667112,-73.981716
IND,6 Avenue,Prospect Park-15 St,40.660365,-73.979493,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Howard Pl,Windsor Pl,NE,40.659436,-73.979951
IND,6 Avenue,Prospect Park-15 St,40.660365,-73.979493,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Prospect Park West,16th St,NE,40.660198,-73.979787
IND,6 Avenue,Fort Hamilton Parkway,40.650782,-73.975776,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Prospect Ave,Greenwood Ave,NE,40.651834,-73.976076
IND,6 Avenue,Fort Hamilton Parkway,40.650782,-73.975776,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Prospect Ave,Greenwood Ave,NW,40.651608,-73.975596
IND,6 Avenue,Fort Hamilton Parkway,40.650782,-73.975776,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Prospect Ave,Reeve Pl,SE,40.652665,-73.975901
IND,6 Avenue,Church Av,40.644041,-73.979678,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,McDonald Ave,Church Ave,SW,40.642571,-73.979579
IND,6 Avenue,Church Av,40.644041,-73.979678,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,McDonald Ave,Church Ave,SE,40.642692,-73.97926
IND,6 Avenue,Church Av,40.644041,-73.979678,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,McDonald Ave,Church Ave,NE,40.643147,-73.979342
IND,6 Avenue,Church Av,40.644041,-73.979678,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,McDonald Ave,Church Ave,NW,40.643006,-73.97966
IND,Culver,Ditmas Av,40.636119,-73.978172,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Ditmas Ave,SW,40.635434,-73.978289
IND,Culver,Ditmas Av,40.636119,-73.978172,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Ditmas Ave,SE,40.635389,-73.977914
IND,Culver,18th Av,40.629755,-73.976971,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,18th Ave,SW,40.630531,-73.977251
IND,Culver,18th Av,40.629755,-73.976971,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,18th Ave,SE,40.630562,-73.976995
IND,Culver,Av I,40.625322,-73.976127,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue I,NW,40.626133,-73.976407
IND,Culver,Av I,40.625322,-73.976127,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue I,NE,40.626163,-73.976177
IND,Culver,Bay Parkway-22nd Av,40.620769,-73.975264,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Bay Pkwy,SW,40.620797,-73.975479
IND,Culver,Bay Parkway-22nd Av,40.620769,-73.975264,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Bay Pkwy,SE,40.620825,-73.975138
IND,Culver,Bay Parkway-22nd Av,40.620769,-73.975264,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Bay Pkwy,NW,40.621162,-73.975467
IND,Culver,Av N,40.61514,-73.974197,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue N,NW,40.614279,-73.97418
IND,Culver,Av N,40.61514,-73.974197,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue N,NE,40.614304,-73.97391
IND,Culver,Av P,40.608944,-73.973022,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue P,SW,40.608615,-73.973097
IND,Culver,Av P,40.608944,-73.973022,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue P,NE,40.609041,-73.972904
IND,Culver,Kings Highway,40.603217,-73.972361,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Kings Hwy,SW,40.604007,-73.972393
IND,Culver,Kings Highway,40.603217,-73.972361,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Kings Hwy,SE,40.603999,-73.972124
IND,Culver,Av U,40.596063,-73.973357,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue U,SW,40.596701,-73.973402
IND,Culver,Av U,40.596063,-73.973357,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue U,SE,40.596897,-73.973137
IND,Culver,Av X,40.58962,-73.97425,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue X,NW,40.590521,-73.974242
IND,Culver,Av X,40.58962,-73.97425,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,McDonald Ave,Avenue X,NE,40.590266,-73.974051
IND,Culver,Neptune Av-Van Siclen,40.581011,-73.974574,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,West 6th St,Neptune Ave,NE,40.580576,-73.974443
IND,Culver,Neptune Av-Van Siclen,40.581011,-73.974574,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,West 6th St,Neptune Ave,NW,40.580547,-73.974778
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,179th Pl,Hillside Ave,SW,40.712581,-73.783377
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,SW,40.712865,-73.782418
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Midland Pkwy,Hillside Ave,NE,40.713207,-73.782119
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,SW,40.712723,-73.782637
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,NW,40.713113,-73.782624
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,SW,40.712723,-73.783035
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,NW,40.71294,-73.783179
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,SW,40.712669,-73.783274
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,180th St,Hillside Ave,NW,40.712892,-73.783363
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,179th Pl,Hillside Ave,SW,40.712548,-73.783427
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,169th St,Hillside Ave,SE,40.710563,-73.792853
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,169th St,Hillside Ave,NE,40.710968,-73.792761
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,169th St,Hillside Ave,SW,40.710495,-73.793022
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,169th St,Hillside Ave,NW,40.71083,-73.793248
IND,Queens Boulevard,Parsons Blvd,40.707564,-73.803326,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Parsons Blvd,Hillside Ave,SW,40.707473,-73.802797
IND,Queens Boulevard,Parsons Blvd,40.707564,-73.803326,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Parsons Blvd,Hillside Ave,NW,40.707873,-73.802999
IND,Queens Boulevard,Parsons Blvd,40.707564,-73.803326,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Parsons Blvd,Hillside Ave,SE,40.707563,-73.802611
IND,Queens Boulevard,Sutphin Blvd,40.70546,-73.810708,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Sutphin Blvd,Hillside Ave,SE,40.705591,-73.809481
IND,Queens Boulevard,Sutphin Blvd,40.70546,-73.810708,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Sutphin Blvd,Hillside Ave,NE,40.705945,-73.809489
IND,Queens Boulevard,Sutphin Blvd,40.70546,-73.810708,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Sutphin Blvd,Hillside Ave,SW,40.705538,-73.809717
IND,Queens Boulevard,Briarwood-Van Wyck Blvd,40.709179,-73.820574,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Van Wyck Blvd,Queens Blvd,SW,40.708068,-73.820111
IND,Queens Boulevard,Briarwood-Van Wyck Blvd,40.709179,-73.820574,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Van Wyck Blvd,Queens Blvd,SE,40.709316,-73.819773
IND,Queens Boulevard,Briarwood-Van Wyck Blvd,40.709179,-73.820574,F,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,84th Rd,Queens Blvd,NE,40.709681,-73.819601
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Union Tpke,Queens Blvd,SE,40.714001,-73.831166
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,80th Rd,Queens Blvd,SE,40.713663,-73.830011
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Union Tpke,Queens Blvd,NE,40.714073,-73.829917
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Union Tpke,Queens Blvd,SE,40.713967,-73.831303
IND,Queens Boulevard,75th Av,40.718331,-73.837324,E,F,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,75th Rd,Queens Blvd,SE,40.717895,-73.836743
IND,Queens Boulevard,75th Av,40.718331,-73.837324,E,F,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,75th,Queens Blvd,NW,40.719061,-73.837916
IND,Queens Boulevard,75th Av,40.718331,-73.837324,E,F,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,75th,Queens Blvd,SW,40.718574,-73.838257
IND,Queens Boulevard,Forest Hills-71st Av,40.721691,-73.844521,E,F,M,R,,,,,,,,Stair,YES,,YES,FULL,,FALSE,in planning,TRUE,71st,Queens Blvd,SE,40.7211,-73.844039
IND,Queens Boulevard,Forest Hills-71st Av,40.721691,-73.844521,E,F,M,R,,,,,,,,Stair,YES,,YES,FULL,,FALSE,in planning,TRUE,71st,Queens Blvd,NE,40.72149,-73.843389
IND,Queens Boulevard,67th Av,40.726523,-73.852719,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,67th Ave,Queens Blvd,SE,40.726598,-73.853534
IND,Queens Boulevard,67th Av,40.726523,-73.852719,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,67th Ave,Queens Blvd,NE,40.727073,-73.853226
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,YES,Yes,YES,FULL,,FALSE,,TRUE,63rd Rd,Queens Blvd,SW,40.729953,-73.862699
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,63rd Rd,Queens Blvd,NE,40.729265,-73.860814
IND,Queens Boulevard,Woodhaven Blvd,40.733106,-73.869229,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Woodhaven Blvd,Queens Blvd,SE,40.733041,-73.870334
IND,Queens Boulevard,Woodhaven Blvd,40.733106,-73.869229,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Woodhaven Blvd,Queens Blvd,SW,40.733354,-73.870983
IND,Queens Boulevard,Woodhaven Blvd,40.733106,-73.869229,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,59th Ave,Queens Blvd,NE,40.733555,-73.869588
IND,Queens Boulevard,Grand Av-Newtown,40.737015,-73.877223,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,54th St,Queens Blvd,SE,40.735888,-73.876886
IND,Queens Boulevard,Grand Av-Newtown,40.737015,-73.877223,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,54th St,Queens Blvd,NE,40.736277,-73.876424
IND,Queens Boulevard,Grand Av-Newtown,40.737015,-73.877223,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,54th St,Queens Blvd,SW,40.736075,-73.877167
IND,Queens Boulevard,Grand Av-Newtown,40.737015,-73.877223,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Queens Blvd,SE,40.737354,-73.877191
IND,Queens Boulevard,Grand Av-Newtown,40.737015,-73.877223,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Queens Blvd,SW,40.737477,-73.877424
IND,Queens Boulevard,Elmhurst Av,40.742454,-73.882017,M,R,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,FALSE,Elmhurst Ave,Broadway,SE,40.741729,-73.881135
IND,Queens Boulevard,Elmhurst Av,40.742454,-73.882017,M,R,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,FALSE,Elmhurst Ave,Broadway,NW,40.742066,-73.881039
IND,Queens Boulevard,Elmhurst Av,40.742454,-73.882017,M,R,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,FALSE,Elmhurst Ave,Broadway,SW,40.741861,-73.881457
IND,Queens Boulevard,Elmhurst Av,40.742454,-73.882017,M,R,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,FALSE,Britton Ave,Broadway,NW,40.742937,-73.882708
IND,Queens Boulevard,Elmhurst Av,40.742454,-73.882017,M,R,,,,,,,,,,Stair,YES,,NO,FULL,,FALSE,,FALSE,Britton Ave,Broadway,SW,40.742791,-73.883025
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,75th St,Roosevelt Ave,SW,40.746784,-73.890958
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,75th St,Broadway,NW,40.746597,-73.890955
IND,Queens Boulevard,65th St,40.749669,-73.898453,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,65th St,Broadway,SE,40.749145,-73.897495
IND,Queens Boulevard,65th St,40.749669,-73.898453,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,65th St,Broadway,NE,40.74933,-73.897382
IND,Queens Boulevard,Northern Blvd,40.752885,-73.906006,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,54th St,Northern Blvd,NW,40.753086,-73.907119
IND,Queens Boulevard,46th St,40.756312,-73.913333,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,46th St,Broadway,NW,40.756868,-73.914303
IND,Queens Boulevard,Steinway St,40.756879,-73.92074,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Steinway St,Broadway,SW,40.758143,-73.919842
IND,Queens Boulevard,Steinway St,40.756879,-73.92074,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Steinway St,Broadway,SE,40.758279,-73.919433
IND,Queens Boulevard,36th St,40.752039,-73.928781,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,35th St,38th Ave,NE,40.752232,-73.928775
IND,Queens Boulevard,36th St,40.752039,-73.928781,M,R,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,35th St,38th Ave,SW,40.752198,-73.929263
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jackson Ave,West St,SW,40.748705,-73.938471
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jackson Ave,Queens Blvd,SW,40.749086,-73.937896
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jackson Ave,Queens Blvd,SE,40.748697,-73.937364
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jackson Ave,Queens Blvd,NE,40.748807,-73.937057
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jackson Ave,West St,SE,40.748466,-73.938244
IND,Queens Boulevard,23rd St-Ely Av,40.747846,-73.946,E,G,M,7,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Hunter St,44th Ave,SW,40.747371,-73.944287
IND,Queens Boulevard,23rd St-Ely Av,40.747846,-73.946,E,G,M,7,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,21st St,44th Ave,SE,40.747954,-73.94724
IND,Queens Boulevard,23rd St-Ely Av,40.747846,-73.946,E,G,M,7,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,21st St,44th Ave,NE,40.748195,-73.947163
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,53rd St,NE,40.758329,-73.970567
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,53rd St,SE,40.75812,-73.970958
IND,Queens Boulevard,5th Av-53rd St,40.760167,-73.975224,E,M,,,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,53rd St,SW,40.760422,-73.976146
IND,Queens Boulevard,5th Av-53rd St,40.760167,-73.975224,E,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,53rd St,NE,40.760373,-73.975562
IND,Queens Boulevard,5th Av-53rd St,40.760167,-73.975224,E,M,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,53rd St,SE,40.760267,-73.975638
IND,Queens Boulevard,7th Av,40.762862,-73.981637,B,D,E,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,53rd St,NE,40.762937,-73.981578
IND,Queens Boulevard,7th Av,40.762862,-73.981637,B,D,E,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,53rd St,SE,40.762793,-73.98169
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,NE,40.702353,-73.799784
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,NE,40.701855,-73.80174
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,NE,40.702821,-73.799843
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,SW,40.702311,-73.799991
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,NE,40.701886,-73.801754
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,NE,40.702674,-73.799786
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Parsons Blvd,Archer Ave,SW,40.702324,-73.80001
IND,Archer Av,Sutphin Blvd-Archer Av - JFK,40.700486,-73.807969,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Check,TRUE,Sutphin Blvd,Archer Ave,SW,40.700401,-73.807976
IND,Archer Av,Sutphin Blvd-Archer Av - JFK,40.700486,-73.807969,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Check,TRUE,Sutphin Blvd,Archer Ave,NW,40.70081,-73.807923
IND,Archer Av,Sutphin Blvd-Archer Av - JFK,40.700486,-73.807969,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Check,TRUE,Sutphin Blvd,Archer Ave,SE,40.700554,-73.807353
IND,Archer Av,Sutphin Blvd-Archer Av - JFK,40.700486,-73.807969,E,J,Z,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,Check,TRUE,Sutphin Blvd,Archer Ave,NE,40.700872,-73.807673
IND,Archer Av,Jamaica-Van Wyck,40.702566,-73.816859,E,,,,,,,,,,,Elevator,YES,,YES,FULL,,FALSE,,TRUE,Van Wyck Expwy,89th Ave,SW,40.701263,-73.816372
IND,Archer Av,Jamaica-Van Wyck,40.702566,-73.816859,E,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Wyck Expwy,Jamaica Ave,NW,40.702731,-73.81727
IND,Archer Av,Jamaica-Van Wyck,40.702566,-73.816859,E,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Wyck Expwy,89th Ave,SW,40.701241,-73.816577
IND,Crosstown,Long Island City-Court Square,40.746554,-73.943832,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jackson Ave,45th Rd,NW,40.746214,-73.945115
IND,Crosstown,21st St,40.744065,-73.949724,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jackson Ave,47th Ave,SW,40.744625,-73.949117
IND,Crosstown,21st St,40.744065,-73.949724,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jackson Ave,47th Ave,NW,40.744771,-73.948658
IND,Crosstown,21st St,40.744065,-73.949724,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jackson Ave,47th Ave,SE,40.744552,-73.948425
IND,Crosstown,Greenpoint Av,40.731352,-73.954449,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Greenpoint Ave,SW,40.73007,-73.954377
IND,Crosstown,Greenpoint Av,40.731352,-73.954449,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Greenpoint Ave,SE,40.730122,-73.954049
IND,Crosstown,Greenpoint Av,40.731352,-73.954449,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Greenpoint Ave,NW,40.730278,-73.954488
IND,Crosstown,Nassau Av,40.724635,-73.951277,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Nassau Ave,SW,40.72365,-73.950929
IND,Crosstown,Nassau Av,40.724635,-73.951277,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Nassau Ave,SE,40.723765,-73.950566
IND,Crosstown,Nassau Av,40.724635,-73.951277,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Nassau Ave,NW,40.72378,-73.950985
IND,Crosstown,Nassau Av,40.724635,-73.951277,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Manhattan Ave,Nassau Ave,NE,40.723894,-73.950665
IND,Crosstown,Metropolitan Av,40.712792,-73.951418,G,L,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Union Ave,Metropolitan Ave,SW,40.71414,-73.95135
IND,Crosstown,Metropolitan Av,40.712792,-73.951418,G,L,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Union Ave,Metropolitan Ave,SW,40.713927,-73.951737
IND,Crosstown,Broadway,40.706092,-73.950308,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Union Ave,Broadway,SE,40.705114,-73.950062
IND,Crosstown,Broadway,40.706092,-73.950308,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Union Ave,Heyward St,NW,40.70549,-73.950554
IND,Crosstown,Broadway,40.706092,-73.950308,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Union Ave,Broadway,NE,40.705495,-73.950089
IND,Crosstown,Broadway,40.706092,-73.950308,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Union Ave,Broadway,SW,40.705204,-73.95032
IND,Crosstown,Flushing Av,40.700377,-73.950234,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Union Ave,Flushing Ave,SW,40.699582,-73.950164
IND,Crosstown,Myrtle-Willoughby Avs,40.694568,-73.949046,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Marcy Ave,Myrtle Ave,NW,40.695436,-73.94948
IND,Crosstown,Bedford-Nostrand Avs,40.689627,-73.953522,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,Lafayette Ave,NW,40.689995,-73.951616
IND,Crosstown,Bedford-Nostrand Avs,40.689627,-73.953522,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,Lafayette Ave,SW,40.689757,-73.951612
IND,Crosstown,Classon Av,40.688873,-73.96007,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Classon Ave,Lafayette Ave,NW,40.688942,-73.960215
IND,Crosstown,Classon Av,40.688873,-73.96007,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Classon Ave,Lafayette Ave,SE,40.688781,-73.959871
IND,Crosstown,Classon Av,40.688873,-73.96007,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Classon Ave,Lafayette Ave,SW,40.688753,-73.96019
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Clinton Ave,Lafayette Ave,NE,40.688114,-73.967575
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,NO,,NO,FULL,,FALSE,,TRUE,Clinton Ave,Lafayette Ave,NE,40.688114,-73.967575
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Clinton Ave,Lafayette Ave,SW,40.687875,-73.967931
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,NO,,NO,FULL,,FALSE,,TRUE,Clinton Ave,Lafayette Ave,SW,40.687875,-73.967931
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Washington Ave,Lafayette Ave,NE,40.68846,-73.965749
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,NO,,NO,FULL,,FALSE,,TRUE,Washington Ave,Lafayette Ave,NE,40.68846,-73.965749
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Washington Ave,Lafayette Ave,SE,40.688157,-73.965579
IND,Crosstown,Clinton-Washington Avs,40.688089,-73.966839,G,,,,,,,,,,,Stair,NO,,NO,FULL,,FALSE,,TRUE,Washington Ave,Lafayette Ave,SE,40.688157,-73.965579
IND,Crosstown,Fulton St,40.687119,-73.975375,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Fort Greene Pl,Lafayette Ave,NE,40.687159,-73.97629
IND,Crosstown,Fulton St,40.687119,-73.975375,G,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Fort Greene Pl,Lafayette Ave,NW,40.68716,-73.976865
IRT,Broadway-7th Ave,Van Cortlandt Park-242nd St,40.889248,-73.898583,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,242nd St,NW,40.889783,-73.898432
IRT,Broadway-7th Ave,Van Cortlandt Park-242nd St,40.889248,-73.898583,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,242nd St,NE,40.889683,-73.897925
IRT,Broadway-7th Ave,Van Cortlandt Park-242nd St,40.889248,-73.898583,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,242nd St,NW,40.890136,-73.898257
IRT,Broadway-7th Ave,Van Cortlandt Park-242nd St,40.889248,-73.898583,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,242nd St,NE,40.889946,-73.897798
IRT,Broadway-7th Ave,238th St,40.884667,-73.90087,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,238th St,SW,40.884813,-73.900923
IRT,Broadway-7th Ave,231st St,40.878856,-73.904834,1,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Broadway,231st St,SW,40.878775,-73.905024
IRT,Broadway-7th Ave,231st St,40.878856,-73.904834,1,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Broadway,231st St,NW,40.879168,-73.904751
IRT,Broadway-7th Ave,Marble Hill-225th St,40.874561,-73.909831,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,225th St,NW,40.874437,-73.910529
IRT,Broadway-7th Ave,215th St,40.869444,-73.915279,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,10th Ave,215th St,SW,40.869412,-73.915455
IRT,Broadway-7th Ave,215th St,40.869444,-73.915279,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,10th Ave,215th St,NW,40.869793,-73.91517
IRT,Broadway-7th Ave,207th St,40.864614,-73.918819,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,10th Ave,207th St,NW,40.864745,-73.918864
IRT,Broadway-7th Ave,207th St,40.864614,-73.918819,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,10th Ave,207th St,SW,40.864283,-73.919222
IRT,Broadway-7th Ave,Dyckman St,40.860531,-73.925536,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hillside Ave,Nagle Ave,SW,40.861183,-73.92511
IRT,Broadway-7th Ave,Dyckman St,40.860531,-73.925536,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Hillside Ave,Nagle Ave,SW,40.861187,-73.925078
IRT,Broadway-7th Ave,191st St,40.855225,-73.929412,1,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,191st St,SW,40.855374,-73.929536
IRT,Broadway-7th Ave,191st St,40.855225,-73.929412,1,,,,,,,,,,,Ramp,YES,,YES,FULL,,FALSE,,TRUE,Broadway,190th St,NE,40.856413,-73.932605
IRT,Broadway-7th Ave,181st St,40.849505,-73.933596,1,,,,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,181st St,NE,40.849435,-73.933428
IRT,Broadway-7th Ave,181st St,40.849505,-73.933596,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saint Nicholas Ave,181st St,SE,40.849098,-73.933747
IRT,Broadway-7th Ave,157th St,40.834041,-73.94489,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,157th St,NW,40.834268,-73.945183
IRT,Broadway-7th Ave,157th St,40.834041,-73.94489,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,157th St,NW,40.834221,-73.945073
IRT,Broadway-7th Ave,145th St,40.826551,-73.95036,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,145th St,NE,40.826657,-73.95052
IRT,Broadway-7th Ave,137th St-City College,40.822008,-73.953676,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,137th St,NW,40.821584,-73.954223
IRT,Broadway-7th Ave,137th St-City College,40.822008,-73.953676,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,137th St,SW,40.821348,-73.954408
IRT,Broadway-7th Ave,125th St,40.815581,-73.958372,1,,,,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Tiemann Pl,NE,40.815143,-73.958439
IRT,Broadway-7th Ave,125th St,40.815581,-73.958372,1,,,,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Tiemann Pl,NW,40.815688,-73.958554
IRT,Broadway-7th Ave,125th St,40.815581,-73.958372,1,,,,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Broadway,125th St,SW,40.815307,-73.958826
IRT,Broadway-7th Ave,125th St,40.815581,-73.958372,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,125th St,SW,40.81523,-73.958366
IRT,Broadway-7th Ave,116th St-Columbia University,40.807722,-73.96411,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,116th St,NW,40.808251,-73.963955
IRT,Broadway-7th Ave,116th St-Columbia University,40.807722,-73.96411,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,116th St,NE,40.808099,-73.963603
IRT,Broadway-7th Ave,116th St-Columbia University,40.807722,-73.96411,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,116th St,NW,40.8084,-73.963845
IRT,Broadway-7th Ave,116th St-Columbia University,40.807722,-73.96411,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,116th St,NE,40.808245,-73.963495
IRT,Broadway-7th Ave,Cathedral Parkway-110th St,40.803967,-73.966847,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,Cathedral Pkwy,NW,40.80439,-73.966786
IRT,Broadway-7th Ave,103rd St,40.799446,-73.968379,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,103rd St,NW,40.799282,-73.968679
IRT,Broadway-7th Ave,103rd St,40.799446,-73.968379,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,103rd St,NE,40.799142,-73.968314
IRT,Broadway-7th Ave,103rd St,40.799446,-73.968379,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,103rd St,NW,40.79941,-73.968612
IRT,Broadway-7th Ave,103rd St,40.799446,-73.968379,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,103rd St,NE,40.799223,-73.968271
IRT,Broadway-7th Ave,86th St,40.788644,-73.976218,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,86th St,SW,40.788349,-73.976935
IRT,Broadway-7th Ave,86th St,40.788644,-73.976218,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,86th St,NW,40.788515,-73.976718
IRT,Broadway-7th Ave,79th St,40.783934,-73.979917,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,79th St,NW,40.784066,-73.980051
IRT,Broadway-7th Ave,79th St,40.783934,-73.979917,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,79th St,SW,40.783765,-73.980274
IRT,Broadway-7th Ave,72nd St,40.778453,-73.98197,1,2,3,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Broadway,72nd St,NE,40.779146,-73.981822
IRT,Broadway-7th Ave,72nd St,40.778453,-73.98197,1,2,3,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Broadway,72nd St,NE,40.778875,-73.981877
IRT,Broadway-7th Ave,66th St-Lincoln Center,40.77344,-73.982209,1,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Broadway,66th St,SE,40.773738,-73.982448
IRT,Broadway-7th Ave,66th St-Lincoln Center,40.77344,-73.982209,1,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,66th St,SE,40.773829,-73.982433
IRT,Broadway-7th Ave,66th St-Lincoln Center,40.77344,-73.982209,1,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,66th St,SE,40.773576,-73.982425
IRT,Broadway-7th Ave,50th St,40.761728,-73.983849,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,50th St,SE,40.761111,-73.983436
IRT,Broadway-7th Ave,50th St,40.761728,-73.983849,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,50th St,SE,40.761282,-73.983853
IRT,Broadway-7th Ave,50th St,40.761728,-73.983849,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,50th St,NE,40.761212,-73.983357
IRT,Broadway-7th Ave,50th St,40.761728,-73.983849,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Broadway,50th St,NE,40.761367,-73.983782
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Easement,YES,,YES,FULL,,TRUE,,TRUE,7th Ave,42nd St,NW,40.756241,-73.987133
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,FULL,,TRUE,,TRUE,7th Ave,42nd St,SW,40.756042,-73.987242
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,7th Ave,34th St,SW,40.751046,-73.991271
IRT,Broadway-7th Ave,23rd St,40.744081,-73.995657,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,7th Ave,23rd St,SE,40.74391,-73.995608
IRT,Broadway-7th Ave,23rd St,40.744081,-73.995657,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,7th Ave,23rd St,NE,40.74412,-73.995348
IRT,Broadway-7th Ave,18th St,40.74104,-73.997871,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,7th Ave,18th St,SE,40.74092,-73.997698
IRT,Broadway-7th Ave,18th St,40.74104,-73.997871,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,7th Ave,18th St,NE,40.741015,-73.99758
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,14th St,SW,40.738484,-73.999914
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,14th St,NW,40.738775,-73.999711
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,7th Ave,14th St,NE,40.738585,-73.999405
IRT,Broadway-7th Ave,Christopher St,40.733422,-74.002906,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,7th Ave,West 4th St,SE,40.733402,-74.002748
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Varick St,Houston St,SE,40.728472,-74.005188
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Varick St,Houston St,NE,40.728656,-74.005086
IRT,Broadway-7th Ave,Canal St,40.722854,-74.006277,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Canal St,Varick St,NE,40.722569,-74.00608
IRT,Broadway-7th Ave,Canal St,40.722854,-74.006277,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Canal St,Varick St,NE,40.722497,-74.005994
IRT,Broadway-7th Ave,Franklin St,40.719318,-74.006886,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,West Broadway,Franklin St,SW,40.719001,-74.006675
IRT,Broadway-7th Ave,Chambers St,40.715478,-74.009266,1,2,3,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Chambers St,West Broadway,SW,40.715444,-74.009453
IRT,Broadway-7th Ave,Chambers St,40.715478,-74.009266,1,2,3,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Chambers St,West Broadway,SE,40.715349,-74.009211
IRT,Broadway-7th Ave,Chambers St,40.715478,-74.009266,1,2,3,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Chambers St,West Broadway,NW,40.715736,-74.009328
IRT,Broadway-7th Ave,Chambers St,40.715478,-74.009266,1,2,3,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Chambers St,West Broadway,NE,40.715576,-74.009031
IRT,Broadway-7th Ave,Chambers St,40.715478,-74.009266,1,2,3,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Chambers St,Broadway,SW,40.715539,-74.009427
IRT,Broadway-7th Ave,Rector St,40.707513,-74.013783,1,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Greenwich St,Rector St,NE,40.708107,-74.013445
IRT,Clark,Park Place,40.713051,-74.008811,A,C,E,1,2,3,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Park Pl,Broadway,NW,40.712712,-74.007766
IRT,Clark,Fulton St,40.709416,-74.006571,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,William St,Fulton St,SE,40.709475,-74.006409
IRT,Clark,Wall St,40.706821,-74.0091,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,William St,Wall St,NW,40.706506,-74.009544
IRT,Clark,Wall St,40.706821,-74.0091,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,William St,Wall St,NE,40.706392,-74.009332
IRT,Clark,Clark St,40.697466,-73.993086,2,3,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Clark St,Henry St,NW,40.69759,-73.993275
IRT,Clark,Clark St,40.697466,-73.993086,2,3,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Clark St,Henry St,NW,40.697649,-73.99306
IRT,Clark,Borough Hall,40.693219,-73.989998,R,2,3,4,5,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Court Street,Montague Street,SE,40.693594,-73.990304
IRT,Clark,Borough Hall,40.693219,-73.989998,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Cadman Plaza West,Montague St,NE,40.693838,-73.990119
IRT,Clark,Borough Hall,40.693219,-73.989998,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Court St,Montague St,SW,40.693569,-73.990599
IRT,Clark,Borough Hall,40.693219,-73.989998,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Court St,Montague St,SW,40.693728,-73.99063
IRT,Eastern Parkway,Hoyt St,40.690545,-73.985065,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Hoyt St,Fulton St,SW,40.690388,-73.985299
IRT,Eastern Parkway,Nevins St,40.688246,-73.980492,2,3,4,5,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Flatbush Ave,Livingston St,NW,40.688235,-73.980639
IRT,Eastern Parkway,Nevins St,40.688246,-73.980492,2,3,4,5,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Flatbush Ave,Livingston St,NW,40.688053,-73.980506
IRT,Eastern Parkway,Atlantic Av-Barclays Ctr,40.684359,-73.977666,B,D,N,Q,R,2,3,4,5,,,Easement,YES,,YES,FULL,,TRUE,,TRUE,Ashland Pl,Hanson Pl,SE,40.684976,-73.97788
IRT,Eastern Parkway,Bergen St,40.680829,-73.975098,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Flatbush Ave,Bergen St,SE,40.680695,-73.974748
IRT,Eastern Parkway,Bergen St,40.680829,-73.975098,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Flatbush Ave,Bergen St,NE,40.680911,-73.97496
IRT,Eastern Parkway,Grand Army Plaza,40.675235,-73.971046,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Flatbush Ave,Plaza Street West,SW,40.67495,-73.97099
IRT,Eastern Parkway,Grand Army Plaza,40.675235,-73.971046,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Flatbush Ave,Plaza Street East,SE,40.67511,-73.970822
IRT,Eastern Parkway,Grand Army Plaza,40.675235,-73.971046,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Flatbush Ave,Plaza Street West,NW,40.675142,-73.971216
IRT,Eastern Parkway,Grand Army Plaza,40.675235,-73.971046,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Flatbush Ave,Plaza Street East,SE,40.675286,-73.970825
IRT,Eastern Parkway,Eastern Parkway-Brooklyn Museum,40.671987,-73.964375,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Washington Ave,Eastern Pkwy,NW,40.672122,-73.964154
IRT,Eastern Parkway,Eastern Parkway-Brooklyn Museum,40.671987,-73.964375,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Washington Ave,Eastern Pkwy,SW,40.671789,-73.964301
IRT,Eastern Parkway,Franklin Av,40.670682,-73.958131,FS,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Franklin Ave,Eastern Pkwy,SE,40.670485,-73.957806
IRT,Eastern Parkway,Franklin Av,40.670682,-73.958131,FS,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Franklin Ave,Eastern Pkwy,NE,40.670732,-73.957696
IRT,Eastern Parkway,Franklin Av,40.670682,-73.958131,FS,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Franklin Ave,Eastern Pkwy,SE,40.670337,-73.957973
IRT,Eastern Parkway,Franklin Av,40.670682,-73.958131,FS,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Franklin Ave,Eastern Pkwy,NE,40.670922,-73.957764
IRT,Eastern Parkway,Nostrand Av,40.669847,-73.950466,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,Eastern Pkwy,SE,40.669711,-73.950247
IRT,Eastern Parkway,Nostrand Av,40.669847,-73.950466,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,Eastern Pkwy,SW,40.669741,-73.950798
IRT,Eastern Parkway,Kingston Av,40.669399,-73.942161,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Kingston Ave,Eastern Pkwy,SE,40.669261,-73.941948
IRT,Eastern Parkway,Kingston Av,40.669399,-73.942161,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Kingston Ave,Eastern Pkwy,SW,40.669289,-73.942463
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Utica Ave,Eastern Pkwy,SE,40.668935,-73.931831
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Utica Ave,Eastern Pkwy,SE,40.6687,-73.931702
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Utica Ave,Eastern Pkwy,NW,40.66896,-73.931675
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Utica Ave,Eastern Pkwy,SW,40.668708,-73.931883
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Utica Ave,Eastern Pkwy,NW,40.668969,-73.931874
IRT,New Lots,Sutter Av,40.664717,-73.92261,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Rutland Rd,E. 98th St,NE,40.664891,-73.922675
IRT,New Lots,Sutter Av,40.664717,-73.92261,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Rutland Rd,E. 98th St,SW,40.664887,-73.923109
IRT,New Lots,Sutter Av,40.664717,-73.92261,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Rutland Rd,E. 98th St,NW,40.66516,-73.92297
IRT,New Lots,Saratoga Av,40.661453,-73.916327,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saratoga Ave,Livonia Ave,SE,40.661558,-73.915225
IRT,New Lots,Saratoga Av,40.661453,-73.916327,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saratoga Ave,Livonia Ave,NE,40.661689,-73.91524
IRT,New Lots,Saratoga Av,40.661453,-73.916327,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Saratoga Ave,Livonia Ave,SW,40.661401,-73.915543
IRT,New Lots,Rockaway Av,40.662549,-73.908946,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Rockaway Ave,Livonia Ave,SE,40.662401,-73.908809
IRT,New Lots,Rockaway Av,40.662549,-73.908946,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Rockaway Ave,Livonia Ave,NW,40.662703,-73.909033
IRT,New Lots,Junius St,40.663515,-73.902447,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Powell St,Livonia Ave,SW,40.663426,-73.902636
IRT,New Lots,Junius St,40.663515,-73.902447,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Powell St,Livonia Ave,SW,40.663361,-73.903063
IRT,New Lots,Pennsylvania Av,40.664635,-73.894895,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pennsylvania Ave,Livonia Ave,NE,40.664869,-73.893779
IRT,New Lots,Pennsylvania Av,40.664635,-73.894895,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Pennsylvania Ave,Livonia Ave,SW,40.664575,-73.894117
IRT,New Lots,Van Siclen Av,40.665449,-73.889395,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Livonia Ave,SE,40.665431,-73.888386
IRT,New Lots,Van Siclen Av,40.665449,-73.889395,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Van Siclen Ave,Livonia Ave,NW,40.66574,-73.888613
IRT,New Lots,New Lots Av,40.666235,-73.884079,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ashford St,Livonia Ave,SW,40.666285,-73.883276
IRT,New Lots,New Lots Av,40.666235,-73.884079,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Ashford St,Livonia Ave,NW,40.666413,-73.883307
IRT,Nostrand,President St,40.667883,-73.950683,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,President St,NW,40.668166,-73.950862
IRT,Nostrand,President St,40.667883,-73.950683,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,President St,NE,40.668145,-73.950458
IRT,Nostrand,Sterling St,40.662742,-73.95085,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,Sterling St,NW,40.66329,-73.951044
IRT,Nostrand,Sterling St,40.662742,-73.95085,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Nostrand Ave,Sterling St,NE,40.663226,-73.95067
IRT,Nostrand,Winthrop St,40.656652,-73.9502,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Nostrand Ave,Winthrop St,NE,40.657184,-73.949976
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,FALSE,Nostrand Ave,Church Ave,SE,40.650748,-73.949384
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Nostrand Ave,Church Ave,SE,40.65064,-73.949429
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Nostrand Ave,Church Ave,NE,40.65091,-73.949416
IRT,Nostrand,Beverly Rd,40.645098,-73.948959,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Nostrand Ave,Beverly Rd,SE,40.645046,-73.948684
IRT,Nostrand,Newkirk Av,40.639967,-73.948411,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Nostrand Ave,Newkirk Ave,SE,40.639924,-73.948161
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,NE,40.632463,-73.947518
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,NE,40.632359,-73.947389
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,SE,40.632324,-73.947482
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,NE,40.63275,-73.947375
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,NE,40.632868,-73.947547
IRT,Pelham,Pelham Bay Park,40.852462,-73.828121,6,,,,,,,,,,,Door,YES,,YES,FULL,,TRUE,,TRUE,Burr Ave,Westchester Ave,NE,40.852377,-73.825986
IRT,Pelham,Pelham Bay Park,40.852462,-73.828121,6,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.852803,-73.827195
IRT,Pelham,Pelham Bay Park,40.852462,-73.828121,6,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.853041,-73.826991
IRT,Pelham,Pelham Bay Park,40.852462,-73.828121,6,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Burr Ave,Westchester Ave,NE,40.853012,-73.827172
IRT,Pelham,Pelham Bay Park,40.852462,-73.828121,6,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Burr Ave,Westchester Ave,SE,40.852824,-73.827035
IRT,Pelham,Buhre Av,40.84681,-73.832569,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Edison Ave,Buhre Ave,SW,40.846891,-73.832639
IRT,Pelham,Buhre Av,40.84681,-73.832569,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Edison Ave,Westchester Ave,SE,40.846795,-73.832405
IRT,Pelham,Buhre Av,40.84681,-73.832569,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Edison Ave,Westchester Ave,NE,40.847185,-73.832264
IRT,Pelham,Middletown Rd,40.843863,-73.836322,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Westchester Ave,Middletown Rd,NW,40.843935,-73.836386
IRT,Pelham,Middletown Rd,40.843863,-73.836322,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Westchester Ave,Middletown Rd,NE,40.843609,-73.836478
IRT,Pelham,Westchester Square-East Tremont Av,40.839892,-73.842952,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Westchester Ave,Ferris Pl,SW,40.840128,-73.842875
IRT,Pelham,Zerega Av,40.836488,-73.847036,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Westchester Ave,Zerega Ave,NW,40.836954,-73.846653
IRT,Pelham,Zerega Av,40.836488,-73.847036,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Westchester Ave,Zerega Ave,SE,40.836613,-73.846726
IRT,Pelham,Castle Hill Av,40.834255,-73.851222,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Castle Hill Ave,Westchester Ave,SW,40.834093,-73.851857
IRT,Pelham,Castle Hill Av,40.834255,-73.851222,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Castle Hill Ave,Westchester Ave,NE,40.834321,-73.85144
IRT,Pelham,Parkchester-East 177th St,40.833226,-73.860816,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Metropolitan Ave,Westchester Ave,NW,40.833225,-73.860888
IRT,Pelham,St Lawrence Av,40.831509,-73.867618,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,St Lawrence Ave,Westchester Ave,NE,40.831765,-73.867078
IRT,Pelham,St Lawrence Av,40.831509,-73.867618,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,St Lawrence Ave,Westchester Ave,SW,40.831466,-73.86741
IRT,Pelham,Morrison Av-Soundview Av,40.829521,-73.874516,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Morrison Ave,Westchester Ave,NW,40.829517,-73.87486
IRT,Pelham,Morrison Av-Soundview Av,40.829521,-73.874516,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Morrison Ave,Westchester Ave,SW,40.829371,-73.874731
IRT,Pelham,Morrison Av-Soundview Av,40.829521,-73.874516,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Morrison Ave,Westchester,NE,40.829641,-73.874447
IRT,Pelham,Elder Av,40.828584,-73.879159,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Elder Ave,Westchester Ave,NE,40.828688,-73.879018
IRT,Pelham,Elder Av,40.828584,-73.879159,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Elder Ave,Westchester Ave,SW,40.82846,-73.879362
IRT,Pelham,Whitlock Av,40.826525,-73.886283,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Whitlock Ave,Westchester Ave,SW,40.827298,-73.886256
IRT,Pelham,Whitlock Av,40.826525,-73.886283,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Whitlock Ave,Westchester Ave,SE,40.827295,-73.886082
IRT,Pelham,Hunts Point Av,40.820948,-73.890549,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,in planning,TRUE,Hunts Point Ave,163rd St,SE,40.820591,-73.890836
IRT,Pelham,Hunts Point Av,40.820948,-73.890549,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,in planning,TRUE,Hunts Point Ave,163rd St,SE,40.820819,-73.890442
IRT,Pelham,Longwood Av,40.816104,-73.896435,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,Longwood Ave,SW,40.816263,-73.896506
IRT,Pelham,Longwood Av,40.816104,-73.896435,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,Longwood Ave,NW,40.81652,-73.896224
IRT,Pelham,East 149th St,40.812118,-73.904098,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,149th St,NW,40.812331,-73.904102
IRT,Pelham,East 149th St,40.812118,-73.904098,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,149th St,NW,40.812367,-73.903797
IRT,Pelham,East 143rd St-St Mary's St,40.808719,-73.907657,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,St Mary's St,SW,40.808179,-73.908065
IRT,Pelham,East 143rd St-St Mary's St,40.808719,-73.907657,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,St Mary's St,NW,40.808322,-73.908002
IRT,Pelham,Cypress Av,40.805368,-73.914042,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Cypress Ave,138th St,NE,40.805234,-73.913391
IRT,Pelham,Cypress Av,40.805368,-73.914042,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Cypress Ave,138th St,NE,40.805155,-73.913208
IRT,Pelham,Brook Av,40.807566,-73.91924,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Brook Ave,138th St,NW,40.807747,-73.919347
IRT,Pelham,Brook Av,40.807566,-73.91924,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Brook Ave,138th St,NE,40.80758,-73.918957
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Alexander Ave,138th St,NW,40.810282,-73.925386
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Alexander Ave,138th St,SW,40.810102,-73.925641
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Alexander Ave,138th St,NW,40.810241,-73.925267
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Alexander Ave,138th St,SW,40.810019,-73.925438
IRT,Jerome,Woodlawn,40.886037,-73.878751,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Bainbridge Ave,SW,40.886819,-73.878376
IRT,Jerome,Woodlawn,40.886037,-73.878751,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Bainbridge Ave,SE,40.886737,-73.87812
IRT,Jerome,Mosholu Parkway,40.87975,-73.884655,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Mosholu Pkwy,SW,40.880077,-73.88447
IRT,Jerome,Mosholu Parkway,40.87975,-73.884655,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Mosholu Pkwy,SE,40.879899,-73.884244
IRT,Jerome,Bedford Park Blvd-Lehman College,40.873412,-73.890064,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Bedford Park Blvd,SW,40.873528,-73.889754
IRT,Jerome,Kingsbridge Rd,40.86776,-73.897174,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Kingsbridge Rd,NW,40.867606,-73.897418
IRT,Jerome,Kingsbridge Rd,40.86776,-73.897174,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Kingsbridge Rd,NE,40.86762,-73.897152
IRT,Jerome,Kingsbridge Rd,40.86776,-73.897174,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Kingsbridge Rd,NE,40.867766,-73.897039
IRT,Jerome,Fordham Rd,40.862803,-73.901034,4,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jerome Ave,Fordham Rd,SW,40.862562,-73.901367
IRT,Jerome,Fordham Rd,40.862803,-73.901034,4,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jerome Ave,Fordham Rd,SE,40.862592,-73.90097
IRT,Jerome,Fordham Rd,40.862803,-73.901034,4,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jerome Ave,Fordham Rd,NW,40.86289,-73.901262
IRT,Jerome,Fordham Rd,40.862803,-73.901034,4,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Jerome Ave,Fordham Rd,NE,40.862814,-73.900815
IRT,Jerome,183rd St,40.858407,-73.903879,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,183rd St,SW,40.858267,-73.904099
IRT,Jerome,183rd St,40.858407,-73.903879,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,183rd St,NE,40.858453,-73.90369
IRT,Jerome,183rd St,40.858407,-73.903879,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,183rd St,NW,40.858477,-73.904088
IRT,Jerome,Burnside Av,40.853453,-73.907684,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Burnside Ave,SW,40.853313,-73.907977
IRT,Jerome,Burnside Av,40.853453,-73.907684,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Burnside Ave,SE,40.853181,-73.907782
IRT,Jerome,Burnside Av,40.853453,-73.907684,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Burnside Ave,SW,40.85367,-73.907654
IRT,Jerome,Burnside Av,40.853453,-73.907684,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Burnside Ave,SE,40.853596,-73.907393
IRT,Jerome,176th St,40.84848,-73.911794,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,176th St,NW,40.848385,-73.912003
IRT,Jerome,176th St,40.84848,-73.911794,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,176th St,SE,40.848501,-73.911633
IRT,Jerome,Mt Eden Av,40.844434,-73.914685,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Cross Bronx Expwy,SW,40.844702,-73.91465
IRT,Jerome,Mt Eden Av,40.844434,-73.914685,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,Mt Eden Ave,NE,40.844389,-73.914578
IRT,Jerome,170th St,40.840075,-73.917791,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,170th St,SW,40.83996,-73.918018
IRT,Jerome,170th St,40.840075,-73.917791,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,170th St,SE,40.839877,-73.917791
IRT,Jerome,170th St,40.840075,-73.917791,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Jerome Ave,170th St,SE,40.840076,-73.917648
IRT,Jerome,167th St,40.835537,-73.9214,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,River Ave,167th St,SE,40.8356,-73.921131
IRT,Jerome,167th St,40.835537,-73.9214,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,River Ave,167th St,SW,40.835746,-73.921562
IRT,Jerome,167th St,40.835537,-73.9214,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,River Ave,167th St,NW,40.835726,-73.921059
IRT,Jerome,167th St,40.835537,-73.9214,4,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,River Ave,167th St,NE,40.835873,-73.92149
IRT,Jerome,149th St-Grand Concourse,40.818375,-73.927351,2,4,5,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,149th St,SE,40.818281,-73.927251
IRT,Jerome,149th St-Grand Concourse,40.818375,-73.927351,2,4,5,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Conccourse,149th St,SE,40.818405,-73.927101
IRT,Jerome,138th St,40.813224,-73.929849,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,East 138th Street,,40.813328,-73.930004
IRT,Jerome,138th St,40.813224,-73.929849,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,East 138th Street,NE,40.813091,-73.929689
IRT,Jerome,138th St,40.813224,-73.929849,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Grand Concourse,East 138th Street,,40.813363,-73.929847
IRT,Lexington,125th St,40.804138,-73.937594,4,5,6,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Lexington Avenue,125th Street,NE,40.804407,-73.937234
IRT,Lexington,125th St,40.804138,-73.937594,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,125th St,SW,40.804251,-73.937658
IRT,Lexington,125th St,40.804138,-73.937594,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,125th St,SE,40.804164,-73.937444
IRT,Lexington,125th St,40.804138,-73.937594,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,125th St,NW,40.804597,-73.9374
IRT,Lexington,125th St,40.804138,-73.937594,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,125th St,NE,40.804517,-73.937186
IRT,Lexington,116th St,40.798629,-73.941617,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,116th St,SW,40.798466,-73.941882
IRT,Lexington,116th St,40.798629,-73.941617,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,116th St,NW,40.798852,-73.9416
IRT,Lexington,110th St,40.79502,-73.94425,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,110th St,SW,40.794804,-73.944613
IRT,Lexington,110th St,40.79502,-73.94425,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,110th St,NW,40.794927,-73.944502
IRT,Lexington,103rd St,40.7906,-73.947478,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,103rd St,SW,40.790279,-73.947856
IRT,Lexington,103rd St,40.7906,-73.947478,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,103rd St,SE,40.790193,-73.947649
IRT,Lexington,96th St,40.785672,-73.95107,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,96th St,SW,40.785843,-73.951232
IRT,Lexington,96th St,40.785672,-73.95107,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,96th St,SE,40.78564,-73.95093
IRT,Lexington,96th St,40.785672,-73.95107,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,96th St,NW,40.786055,-73.951057
IRT,Lexington,96th St,40.785672,-73.95107,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,96th St,NE,40.785903,-73.950695
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,86th St,SW,40.779468,-73.955905
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,86th St,NW,40.779718,-73.955719
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,86th St,SW,40.779427,-73.95581
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,86th St,NW,40.77972,-73.955596
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,77th St,SW,40.773669,-73.960135
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,77th St,NW,40.773779,-73.959954
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,77th St,NW,40.773779,-73.960041
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lexington Ave,77th St,SW,40.773636,-73.96006
IRT,Lexington,68th St-Hunter College,40.768141,-73.96387,6,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,68th St,SW,40.76787,-73.964244
IRT,Lexington,68th St-Hunter College,40.768141,-73.96387,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,68th St,SW,40.767817,-73.964355
IRT,Lexington,68th St-Hunter College,40.768141,-73.96387,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,68th St,SE,40.767703,-73.963979
IRT,Lexington,68th St-Hunter College,40.768141,-73.96387,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,68th St,NW,40.767981,-73.964178
IRT,Lexington,68th St-Hunter College,40.768141,-73.96387,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Ave,68th St,NE,40.767863,-73.963898
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Lexington Avenue,East 60th Street,SW,40.762836,-73.96804
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,51st St,SW,40.757123,-73.972117
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,51st St,NW,40.757293,-73.972108
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,51st St,SW,40.757157,-73.972196
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Lexington Ave,51st St,NW,40.757298,-73.971939
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,33rd St,SE,40.746263,-73.981663
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,33rd St,SE,40.746242,-73.9816
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,33rd St,NE,40.74655,-73.981533
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,28th St,SE,40.743128,-73.983806
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,28th St,SE,40.743167,-73.983896
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,28th St,NE,40.743304,-73.983857
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Park Ave,28th St,NE,40.743257,-73.983738
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,In Planning,FALSE,Park Ave,23rd St,SE,40.740008,-73.986233
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,In Planning,FALSE,Park Ave,23rd St,SE,40.739984,-73.986343
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,In Planning,FALSE,Park Ave,23rd St,NE,40.74018,-73.986006
IRT,Lexington,14th St-Union Square,40.734673,-73.989951,L,N,Q,R,4,5,6,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,14th St,NE,40.734406,-73.989437
IRT,Lexington,14th St-Union Square,40.734673,-73.989951,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,14th St,SE,40.73418,-73.989549
IRT,Lexington,14th St-Union Square,40.734673,-73.989951,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,4th Ave,14th St,SE,40.734222,-73.989674
IRT,Lexington,Astor Place,40.730054,-73.99107,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,4th Ave,East 8th St,NE,40.730084,-73.990838
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lafayette St,Mulberry St,SW,40.725726,-73.994579
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lafayette St,Mulberry St,SE,40.725755,-73.994367
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lafayette St,Mulberry St,NE,40.725845,-73.99432
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,,,,40.725137,-73.994938
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,,,,40.724888,-73.995291
IRT,Lexington,Spring St,40.722301,-73.997141,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lafayette St,Spring St,SE,40.722116,-73.996912
IRT,Lexington,Spring St,40.722301,-73.997141,6,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lafayette St,Spring St,NE,40.722235,-73.996818
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,FULL,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,SE,40.718155,-74.000317
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,FULL,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,SE,40.718197,-74.000388
IRT,Lexington,Brooklyn Bridge-City Hall,40.712868,-74.004806,J,Z,4,5,6,,,,,,,Stair,YES,,YES,FULL,,TRUE,J Z not ADA,TRUE,Centre St,Brooklyn Bridge Approach,NW,40.712868,-74.004806
IRT,Lexington,Brooklyn Bridge-City Hall,40.712723,-74.004971,J,Z,4,5,6,,,,,,,Stair,YES,,YES,FULL,,TRUE,J Z not ADA,TRUE,Centre St,Brooklyn Bridge Approach,NW,40.712723,-74.004971
IRT,Lexington,Brooklyn Bridge-City Hall,40.712333,-74.004387,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,J Z not ADA,TRUE,Centre St,Brooklyn Bridge Approach,NE,40.712333,-74.004387
IRT,Lexington,Brooklyn Bridge-City Hall,40.711857,-74.005114,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,J Z not ADA,TRUE,Nassau St,Frankfort St,SE,40.711857,-74.005114
IRT,Lexington,Brooklyn Bridge-City Hall,40.711824,-74.005057,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,J Z not ADA,TRUE,Nassau St,Frankfort St,SE,40.711824,-74.005057
IRT,Lexington,Brooklyn Bridge-City Hall,40.713813,-74.003907,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,J Z not ADA,TRUE,Lafayette St,Reade St,NW,40.713721,-74.003734
IRT,Lexington,Brooklyn Bridge-City Hall,40.713889,-74.003654,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,J Z not ADA,TRUE,Lafayette St,Reade St,NW,40.713721,-74.003734
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Rector St,SE,40.70719,-74.011997
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Wall St,SE,40.707446,-74.011848
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Broadway,Wall St,SE,40.707577,-74.011731
IRT,Lexington,Bowling Green,40.704817,-74.014065,4,5,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,State,Battery Pl,SW,40.704659,-74.01392
IRT,Lexington,Bowling Green,40.704817,-74.014065,4,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Broadway,Bowling Green,NW,40.704629,-74.013502
IRT,Lexington,Borough Hall,40.692404,-73.990151,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Court St,Joralemon St,SE,40.69234,-73.99056
IRT,Lexington,Borough Hall,40.692404,-73.990151,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Court St,Joralemon St,SE,40.692299,-73.990301
IRT,Lexington,Borough Hall,40.692404,-73.990151,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Court St,Joralemon St,NE,40.692548,-73.990099
IRT,Lexington,Borough Hall,40.692404,-73.990151,R,2,3,4,5,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Court St,Joralemon St,NE,40.692588,-73.990572
IRT,White Plains Road,Wakefield-241st St,40.903125,-73.85062,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,241st St,NW,40.903597,-73.85036
IRT,White Plains Road,Wakefield-241st St,40.903125,-73.85062,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,241st St,SE,40.903283,-73.850321
IRT,White Plains Road,Wakefield-241st St,40.903125,-73.85062,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,241st St,NE,40.903509,-73.850124
IRT,White Plains Road,238th St-Nereid Av,40.898379,-73.854376,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,Nereid Ave,NW,40.89862,-73.854316
IRT,White Plains Road,238th St-Nereid Av,40.898379,-73.854376,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,Nereid Ave,SE,40.898323,-73.854288
IRT,White Plains Road,233rd St,40.893193,-73.857473,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,White Plains Rd,233rd St,SE,40.893215,-73.857344
IRT,White Plains Road,233rd St,40.893193,-73.857473,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,White Plains Rd,233rd St,NW,40.893597,-73.857405
IRT,White Plains Road,225th St,40.888022,-73.860341,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,225th St,SW,40.887709,-73.860666
IRT,White Plains Road,225th St,40.888022,-73.860341,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,225th St,SE,40.88766,-73.860444
IRT,White Plains Road,225th St,40.888022,-73.860341,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,225th St,NW,40.887972,-73.860517
IRT,White Plains Road,219th St,40.883895,-73.862633,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,219th St,NW,40.883723,-73.86287
IRT,White Plains Road,219th St,40.883895,-73.862633,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,219th St,SE,40.883439,-73.862743
IRT,White Plains Road,Gun Hill Rd,40.87785,-73.866256,2,5,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,211th St,SW,40.877779,-73.866185
IRT,White Plains Road,Gun Hill Rd,40.87785,-73.866256,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,211th St,SW,40.877839,-73.866421
IRT,White Plains Road,Burke Av,40.871356,-73.867164,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,Burke Ave,NW,40.871631,-73.867283
IRT,White Plains Road,Burke Av,40.871356,-73.867164,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains,Burke Ave,SE,40.871215,-73.867043
IRT,White Plains Road,Allerton Av,40.865462,-73.867352,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,Allerton Ave,NW,40.865605,-73.867476
IRT,White Plains Road,Allerton Av,40.865462,-73.867352,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,White Plains Rd,Allerton Ave,SE,40.865257,-73.867246
IRT,White Plains Road,Pelham Parkway,40.857192,-73.867615,2,5,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,White Plains Rd,Pelham Pkwy,SE,40.856338,-73.867544
IRT,White Plains Road,Pelham Parkway,40.857192,-73.867615,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,White Plains Rd,Pelham Pkwy,SW,40.856403,-73.867771
IRT,White Plains Road,Pelham Parkway,40.857192,-73.867615,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,White Plains Rd,Pelham Pkwy,SE,40.856337,-73.867521
IRT,White Plains Road,Bronx Park East,40.848828,-73.868457,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Birchall Ave,Sagamore St,SE,40.848762,-73.868382
IRT,White Plains Road,Bronx Park East,40.848828,-73.868457,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Birchall Ave,Sagamore St,NW,40.849169,-73.868356
IRT,White Plains Road,Bronx Park East,40.848828,-73.868457,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Birchall Ave,Sagamore St,NE,40.849128,-73.868213
IRT,White Plains Road,East 180th St,40.841894,-73.873488,2,5,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Morris Park Ave,180th St,NW,40.841223,-73.873499
IRT,White Plains Road,East 180th St,40.841894,-73.873488,2,5,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Morris Park Ave,180th St,NW,40.841453,-73.872892
IRT,White Plains Road,East Tremont Av-West Farms Sq,40.840295,-73.880049,2,5,,,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Boston Rd,178th St,SW,40.840815,-73.879623
IRT,White Plains Road,East Tremont Av-West Farms Sq,40.840295,-73.880049,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Boston Rd,E Tremont Ave,NW,40.840434,-73.880005
IRT,White Plains Road,East Tremont Av-West Farms Sq,40.840295,-73.880049,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Boston Rd,E Tremont Ave,NE,40.840354,-73.879833
IRT,White Plains Road,East Tremont Av-West Farms Sq,40.840295,-73.880049,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Boston Rd,178th St,SE,40.840639,-73.879555
IRT,White Plains Road,East Tremont Av-West Farms Sq,40.840295,-73.880049,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Boston Rd,178th St,NW,40.841078,-73.879397
IRT,White Plains Road,174th St,40.837288,-73.887734,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Boston Rd,174th St,SW,40.837325,-73.888048
IRT,White Plains Road,174th St,40.837288,-73.887734,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Boston Rd,174th St,NW,40.837599,-73.887755
IRT,White Plains Road,Freeman St,40.829993,-73.891865,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,Freeman St,SW,40.829903,-73.89202
IRT,White Plains Road,Freeman St,40.829993,-73.891865,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Southern Blvd,Freeman St,NW,40.830369,-73.892015
IRT,White Plains Road,Simpson St,40.824073,-73.893064,2,5,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,FALSE,Westchester Ave,Simpson St,NE,40.824356,-73.892897
IRT,White Plains Road,Simpson St,40.824073,-73.893064,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Westchester Ave,Simpson St,NW,40.824177,-73.893114
IRT,White Plains Road,Simpson St,40.824073,-73.893064,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Westchester Ave,Simpson St,NE,40.824358,-73.892724
IRT,White Plains Road,Intervale Av,40.822181,-73.896736,2,5,,,,,,,,,,Escalator,YES,,YES,FULL,,FALSE,,TRUE,Intervale Ave,Westchester Ave,NE,40.8224,-73.896508
IRT,White Plains Road,Intervale Av,40.822181,-73.896736,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Intervale Ave,Westchester Ave,NE,40.822447,-73.896719
IRT,White Plains Road,Intervale Av,40.822181,-73.896736,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Intervale Ave,Westchester Ave,SE,40.822095,-73.896681
IRT,White Plains Road,Prospect Av,40.819585,-73.90177,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Westchester,160th St,SW,40.819583,-73.901958
IRT,White Plains Road,Prospect Av,40.819585,-73.90177,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Prospect Ave,160th St,NW,40.819887,-73.901666
IRT,White Plains Road,Jackson Av,40.81649,-73.907807,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Jackson Ave,152nd St,NE,40.816201,-73.907942
IRT,White Plains Road,Jackson Av,40.81649,-73.907807,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Jackson Ave,152nd St,NE,40.816349,-73.907836
IRT,White Plains Road,Jackson Av,40.81649,-73.907807,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Jackson Ave,Westchester Ave,NE,40.816589,-73.907893
IRT,White Plains Road,Jackson Av,40.81649,-73.907807,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Jackson Ave,Westchester Ave,NW,40.816873,-73.907379
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,FALSE,Melrose Ave,149th St,NW,40.816247,-73.917798
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Melrose Ave,149th St,NE,40.816315,-73.917543
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Melrose Ave,149th St,NW,40.816242,-73.917934
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,3rd Ave,149th St,NW,40.816265,-73.917456
IRT,Lenox,Harlem-148th St,40.82388,-73.93647,3,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,Adam C. Powell Blvd,149th St,NE,40.824069,-73.936981
IRT,Lenox,145th St,40.820421,-73.936245,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lenox Ave,145th St,SW,40.820343,-73.936508
IRT,Lenox,145th St,40.820421,-73.936245,3,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lenox Ave,145th St,NW,40.820662,-73.936275
IRT,Lenox,135th St,40.814229,-73.94077,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Lenox Ave,135th St,SW,40.814024,-73.941117
IRT,Lenox,135th St,40.814229,-73.94077,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,FALSE,Lenox Ave,135th St,NW,40.814316,-73.94091
IRT,Lenox,125th St,40.807754,-73.945495,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lenox Ave,125th St,SW,40.807681,-73.945752
IRT,Lenox,125th St,40.807754,-73.945495,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lenox Ave,125th St,NW,40.807995,-73.945524
IRT,Lenox,116th St,40.802098,-73.949625,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lenox Ave,116th St,SW,40.801945,-73.949943
IRT,Lenox,116th St,40.802098,-73.949625,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Lenox Ave,116th St,NW,40.802229,-73.949734
IRT,Lenox,110th St-Central Park North,40.799075,-73.951822,2,3,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,St Nicholas Ave,Central Park North,NW,40.798497,-73.952672
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Main St,Roosevelt Ave,SE,40.759542,-73.829885
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Main St,Roosevelt Ave,NE,40.759706,-73.829943
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Main St,Roosevelt Ave,SE,40.759451,-73.829927
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Main St,Roosevelt Ave,NW,40.759587,-73.830408
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Main St,Roosevelt Ave,SW,40.759404,-73.830333
IRT,Flushing,Mets - Willets Point,40.754622,-73.845625,7,,,,,,,,,,,Door,YES,,YES,FULL,,FALSE,,TRUE,126th St,Roosevelt Ave,NW,40.754777,-73.845737
IRT,Flushing,Mets - Willets Point,40.754622,-73.845625,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,126th St,Roosevelt Ave,SW,40.754394,-73.845832
IRT,Flushing,Mets - Willets Point,40.754622,-73.845625,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,126th St,Roosevelt Ave,NW,40.754657,-73.846014
IRT,Flushing,Mets - Willets Point,40.754622,-73.845625,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,126th St,Roosevelt Ave,NW,40.754768,-73.845598
IRT,Flushing,111th St,40.75173,-73.855334,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Roosevelt Ave,NW,40.75161,-73.856065
IRT,Flushing,111th St,40.75173,-73.855334,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Roosevelt Ave,SW,40.751423,-73.855996
IRT,Flushing,111th St,40.75173,-73.855334,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,111th St,Roosevelt Ave,SE,40.751551,-73.855628
IRT,Flushing,103rd St,40.749865,-73.8627,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,103rd St,Roosevelt Ave,SE,40.749787,-73.862389
IRT,Flushing,103rd St,40.749865,-73.8627,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,103rd St,Roosevelt Ave,NE,40.749973,-73.862425
IRT,Flushing,103rd St,40.749865,-73.8627,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,103rd St,Roosevelt Ave,SE,40.749775,-73.862537
IRT,Flushing,103rd St,40.749865,-73.8627,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,103rd St,Roosevelt Ave,NE,40.749964,-73.862546
IRT,Flushing,Junction Blvd,40.749145,-73.869527,7,,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,Junction Boulevard,Roosevelt Avenue,NE,40.749263,-73.869376
IRT,Flushing,Junction Blvd,40.749145,-73.869527,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Junction Blvd,Roosevelt Ave,SE,40.749095,-73.86912
IRT,Flushing,Junction Blvd,40.749145,-73.869527,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Junction Blvd,Roosevelt Ave,NE,40.749266,-73.86922
IRT,Flushing,Junction Blvd,40.749145,-73.869527,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Junction Blvd,Roosevelt Ave,SW,40.749028,-73.869574
IRT,Flushing,Junction Blvd,40.749145,-73.869527,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,Junction Blvd,Roosevelt Ave,NW,40.749213,-73.869696
IRT,Flushing,90th St Elmhurst,40.748408,-73.876613,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,90th St,Roosevelt Ave,SE,40.748318,-73.876432
IRT,Flushing,90th St Elmhurst,40.748408,-73.876613,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,90th St,Roosevelt Ave,NE,40.74859,-73.876461
IRT,Flushing,90th St Elmhurst,40.748408,-73.876613,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,90th St,Roosevelt Ave,NW,40.748582,-73.876591
IRT,Flushing,82nd St-Jackson Heights,40.747659,-73.883697,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,82nd St,Roosevelt Ave,NE,40.747812,-73.883876
IRT,Flushing,82nd St-Jackson Heights,40.747659,-73.883697,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,82nd St,Roosevelt Ave,SW,40.747453,-73.883973
IRT,Flushing,82nd St-Jackson Heights,40.747659,-73.883697,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,82nd St,Roosevelt Ave,NW,40.747797,-73.884028
IRT,Flushing,Fisk Av-69th St,40.746325,-73.896403,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,69th St,Roosevelt Ave,NE,40.746509,-73.896268
IRT,Flushing,Fisk Av-69th St,40.746325,-73.896403,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,69th St,Roosevelt Ave,SW,40.746148,-73.896396
IRT,Flushing,Woodside Av-61st St,40.74563,-73.902984,7,,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,61st St,Roosevelt Ave,SE,40.745531,-73.903248
IRT,Flushing,Woodside Av-61st St,40.74563,-73.902984,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,61st St,Roosevelt Ave,NE,40.745717,-73.903063
IRT,Flushing,Woodside Av-61st St,40.74563,-73.902984,7,,,,,,,,,,,Stair,YES,,YES,FULL,,TRUE,,TRUE,61st St,Roosevelt Ave,NE,40.745692,-73.903234
IRT,Flushing,Lincoln Av-52nd St,40.744149,-73.912549,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,52nd St,Roosevelt Ave,SW,40.743933,-73.913003
IRT,Flushing,Lincoln Av-52nd St,40.744149,-73.912549,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,52nd St,Roosevelt Ave,NW,40.744067,-73.91308
IRT,Flushing,Lincoln Av-52nd St,40.744149,-73.912549,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,53rd St,Roosevelt Ave,SE,40.744453,-73.911282
IRT,Flushing,Lincoln Av-52nd St,40.744149,-73.912549,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,53rd St,Roosevelt Ave,NE,40.74459,-73.911294
IRT,Flushing,Bliss St-46th St,40.743132,-73.918435,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,46th St,Queens Blvd,SE,40.743064,-73.918601
IRT,Flushing,Bliss St-46th St,40.743132,-73.918435,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,46th St,Queens Blvd,NE,40.743217,-73.918564
IRT,Flushing,Bliss St-46th St,40.743132,-73.918435,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,46th St,Queens Blvd,SW,40.743105,-73.918848
IRT,Flushing,Bliss St-46th St,40.743132,-73.918435,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,46th St,Queens Blvd,NW,40.74324,-73.918813
IRT,Flushing,Lowery St-40th St,40.743781,-73.924016,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,40th St,Queens Blvd,SE,40.74371,-73.924161
IRT,Flushing,Lowery St-40th St,40.743781,-73.924016,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,40th St,Queens Blvd,NE,40.743882,-73.924112
IRT,Flushing,Lowery St-40th St,40.743781,-73.924016,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,40th St,Queens Blvd,SW,40.743743,-73.924417
IRT,Flushing,Lowery St-40th St,40.743781,-73.924016,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,40th St,Queens Blvd,NW,40.743911,-73.924385
IRT,Flushing,Rawson St-33rd St,40.744587,-73.930997,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,33rd St,Queens Blvd,SE,40.744568,-73.93157
IRT,Flushing,Rawson St-33rd St,40.744587,-73.930997,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,33rd St,Queens Blvd,NE,40.744724,-73.931533
IRT,Flushing,Rawson St-33rd St,40.744587,-73.930997,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,33rd St,Queens Blvd,NW,40.744594,-73.931829
IRT,Flushing,Rawson St-33rd St,40.744587,-73.930997,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,33rd St,Queens Blvd,SW,40.744763,-73.931794
IRT,Flushing,Queensboro Plaza,40.750582,-73.940202,N,Q,7,,,,,,,,,Easement,YES,,YES,FULL,,FALSE,,TRUE,Queens Plaza North,27th St,NW,40.750835,-73.939671
IRT,Flushing,Queensboro Plaza,40.750582,-73.940202,N,Q,7,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,Queens Plaza South,27th St,SW,40.750302,-73.940245
IRT,Flushing,45 Rd-Court House Sq,40.747023,-73.945264,E,G,M,7,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,23rd St,45th Rd,NW,40.746308,-73.945713
IRT,Flushing,45 Rd-Court House Sq,40.747023,-73.945264,E,G,M,7,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,23rd St,45th Rd,SE,40.74613,-73.945349
IRT,Flushing,45 Rd-Court House Sq,40.747023,-73.945264,E,G,M,7,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,23rd St,45th Rd,NE,40.746232,-73.945322
IRT,Flushing,Hunters Point,40.742216,-73.948916,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,21st St,49th Ave,NE,40.7425,-73.948609
IRT,Flushing,Hunters Point,40.742216,-73.948916,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,21st St,49th Ave,NE,40.742405,-73.94811
IRT,Flushing,Hunters Point,40.742216,-73.948916,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,21st St,49th Ave,NE,40.742422,-73.948195
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Vernon Blvd,50th Ave,NE,40.742759,-73.953893
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,YES,,YES,FULL,,FALSE,,FALSE,Vernon Blvd,50th Ave,NE,40.742803,-73.953955
IRT,Flushing,5th Av,40.753821,-73.981963,B,D,F,M,7,,,,,,,Stair,YES,,YES,FULL,,FALSE,,TRUE,5th Ave,42nd St,SW,40.753409,-73.98113
IRT,Broadway-7th Ave,South Ferry,40.702068,-74.013664,R,1,,,,,,,,,,Elevator,YES,,NO,FULL,,TRUE,,TRUE,,,,40.702033,-74.013149
IRT,Broadway-7th Ave,South Ferry,40.702068,-74.013664,R,1,,,,,,,,,,Elevator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.702033,-74.013149
IRT,Broadway-7th Ave,South Ferry,40.702068,-74.013664,R,1,,,,,,,,,,Escalator,YES,,NO,FULL,,TRUE,,TRUE,,,,40.701961,-74.013397
IRT,Broadway-7th Ave,South Ferry,40.702068,-74.013664,R,1,,,,,,,,,,Escalator,YES,,YES,FULL,,TRUE,,TRUE,,,,40.701961,-74.013397
BMT,Broadway,Lexington Av,40.76266,-73.967258,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,3rd Ave,60th St,SW,40.762092,-73.96634
BMT,Broadway,Lexington Av,40.76266,-73.967258,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,3rd Ave,60th St,NW,40.762353,-73.966155
BMT,Broadway,Lexington Av,40.76266,-73.967258,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,3rd Ave,60th St,SE,40.761996,-73.966105
BMT,Broadway,Lexington Av,40.76266,-73.967258,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,3rd Ave,60th St,NE,40.762255,-73.965918
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Grand Army Plaza,Central Park South,SW,40.764599,-73.974228
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Army Plaza,Central Park South,NE,40.764935,-73.974191
BMT,Broadway,5th Av,40.764811,-73.973347,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Army Plaza,Central Park South,NE,40.764859,-73.973998
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,7th Ave,55th St,SW,40.764258,-73.981278
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,7th Ave,55th St,SE,40.764057,-73.980803
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,7th Ave,55th St,NW,40.764377,-73.981167
BMT,Broadway,57th St,40.764664,-73.980658,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,7th Ave,55th St,NE,40.764179,-73.980683
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,Northbound Only,FALSE,7th Ave,49th St,NE,40.760454,-73.983388
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,Northbound Only,FALSE,7th Ave,47th St,SE,40.75903,-73.984412
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,Northbound Only,FALSE,Broadway,47th St,NE,40.759513,-73.984884
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,Northbound Only,FALSE,7th Ave,47th St,NW,40.759384,-73.984709
BMT,Broadway,49th St,40.759901,-73.984139,N,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,Northbound Only,FALSE,7th Ave,49th St,SE,40.760297,-73.983567
BMT,Broadway,Times Square-42nd St,40.754672,-73.986754,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,Shuttle not ADA,TRUE,Broadway,40th St,SE,40.754172,-73.98668
BMT,Broadway,Times Square-42nd St,40.754672,-73.986754,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,Shuttle not ADA,TRUE,Broadway,40th St,SW,40.754335,-73.987097
BMT,Broadway,Times Square-42nd St,40.754672,-73.986754,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,Shuttle not ADA,TRUE,Broadway,40th St,NE,40.754301,-73.986623
BMT,Broadway,Times Square-42nd St,40.754672,-73.986754,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,Shuttle not ADA,TRUE,Broadway,40th St,NW,40.754507,-73.987108
BMT,Broadway,34th St,40.749567,-73.98795,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,34th St,SE,40.74959,-73.98762
BMT,Broadway,34th St,40.749567,-73.98795,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,34th St,NE,40.749891,-73.987495
BMT,Broadway,34th St,40.749567,-73.98795,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,32nd St,NE,40.748294,-73.987976
BMT,Broadway,34th St,40.749567,-73.98795,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,32nd St,NW,40.748477,-73.988288
BMT,Broadway,28th St,40.745494,-73.988691,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,28th St,SW,40.745281,-73.988923
BMT,Broadway,28th St,40.745494,-73.988691,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,28th St,NW,40.745488,-73.98901
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Broadway,22nd St,NE,40.740643,-73.989217
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,5th Ave,22nd St,NE,40.741208,-73.989639
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,23rd St,NW,40.741913,-73.989428
BMT,Broadway,23rd St,40.741303,-73.989344,N,R,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Broadway,22nd St,NW,40.740758,-73.989632
BMT,Broadway,Union Square,40.735736,-73.990568,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,NONE,,TRUE,Lex not ADA,TRUE,Broadway,16th St,SE,40.736339,-73.990603
BMT,Broadway,Union Square,40.735736,-73.990568,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,NONE,,TRUE,Lex not ADA,TRUE,Broadway,16th St,NE,40.736523,-73.990447
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Broadway,Waverly Pl,NE,40.729678,-73.993051
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,8th Ave,SW,40.730642,-73.992737
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,8th Ave,SW,40.730592,-73.99262
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,8th Ave,NW,40.730716,-73.992517
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,8th Ave,NW,40.730773,-73.992641
BMT,Broadway,Prince St,40.724329,-73.997702,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,Prince St,SW,40.72426,-73.997957
BMT,Broadway,Prince St,40.724329,-73.997702,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,Prince St,NW,40.724344,-73.997875
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Canal St,NW,40.719585,-74.00193
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Canal St,NW,40.71966,-74.002019
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Canal St,SW,40.719408,-74.002158
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Canal St,SW,40.71946,-74.002236
BMT,Broadway,Cortlandt St,40.710668,-74.011029,R,,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Church St,Cortlandt St,SE,40.710201,-74.011198
BMT,Broadway,Cortlandt St,40.710668,-74.011029,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Cortlandt St,NE,40.710435,-74.011066
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Greenwich St,Morris St,NE,40.706345,-74.014
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Trinity Pl,Rector St,SW,40.707495,-74.013273
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Trinity Pl,Rector St,SW,40.707659,-74.013166
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Trinity Pl,Rector St,NW,40.70795,-74.012963
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Trinity Pl,Rector St,NW,40.708083,-74.012863
BMT,Broadway,Rector St,40.70722,-74.013342,R,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Trinity Pl,Exchange Alley,NW,40.707225,-74.013454
BMT,Broadway,Whitehall St,40.703087,-74.012994,R,1,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,,,,40.704015,-74.013284
BMT,Broadway,Whitehall St,40.703087,-74.012994,R,1,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,,,,40.704114,-74.013
BMT,Broadway,Whitehall St,40.703087,-74.012994,R,1,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,,,,40.704096,-74.013293
BMT,Broadway,Court St,40.6941,-73.991777,R,2,3,4,5,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Clinton St,Montague St,NW,40.694416,-73.992619
BMT,Broadway,Court St,40.6941,-73.991777,R,2,3,4,5,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Clinton St,Montague St,SW,40.694232,-73.992635
BMT,Broadway,Court St,40.6941,-73.991777,R,2,3,4,5,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Clinton St,Montague St,NW,40.694475,-73.99283
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bridge St,Willoughby St,SE,40.69198,-73.98519
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bridge St,Willoughby St,NE,40.692315,-73.985172
BMT,Broadway,Lawrence St,40.69218,-73.985942,A,C,F,R,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bridge St,Willoughby St,SW,40.692031,-73.985395
BMT,4 Avenue,DeKalb Av,40.690635,-73.981824,B,Q,R,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Flatbush Ave,Fleet St,NW,40.69154,-73.982565
BMT,4 Avenue,DeKalb Av,40.690635,-73.981824,B,Q,R,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Flatbush Ave,Fleet St,NE,40.691562,-73.982048
BMT,4 Avenue,Union St,40.677316,-73.98311,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,4th Ave,Union St,SW,40.677135,-73.983416
BMT,4 Avenue,9th St,40.670847,-73.988302,F,G,R,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,4th Ave,9th St,NW,40.670538,-73.988778
BMT,4 Avenue,Prospect Av,40.665414,-73.992872,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,4th Ave,Prospect Ave,SE,40.665269,-73.993245
BMT,4 Avenue,25th St,40.660397,-73.998091,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,4th Ave,25th St,SW,40.660489,-73.99822
BMT,4 Avenue,53rd St,40.645069,-74.014034,R,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,4th Ave,52nd St,NW,40.645619,-74.013688
BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,4th Ave,60th St,SW,40.640682,-74.018857
BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,4th Ave,60th St,SE,40.640501,-74.018531
BMT,4 Avenue,Bay Ridge Av,40.634967,-74.023377,R,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,4th Ave,68th St,NW,40.635468,-74.023313
BMT,4 Avenue,77th St,40.629742,-74.02551,R,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,4th Ave,76th St,NW,40.630221,-74.025459
BMT,4 Avenue,86th St,40.622687,-74.028398,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,4th Ave,85th St,SW,40.623498,-74.028204
BMT,4 Avenue,95th St,40.616622,-74.030876,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,4th Ave,93th St,SW,40.617587,-74.030696
BMT,4 Avenue,95th St,40.616622,-74.030876,R,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,4th Ave,93th St,NE,40.617363,-74.030404
BMT,Brighton,Atlantic Av,40.68446,-73.97689,B,Q,D,N,R,2,3,4,5,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Ashland Pl,Hanson Pl,NE,40.685222,-73.977628
BMT,Brighton,Prospect Park,40.661614,-73.962246,B,Q,FS,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Flatbush Ave,Empire Blvd,SW,40.662482,-73.962285
BMT,Brighton,Prospect Park,40.661614,-73.962246,B,Q,FS,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Flatbush Ave,Empire Blvd,SW,40.662727,-73.962249
BMT,Brighton,Parkside Av,40.655292,-73.961495,B,Q,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Ocean Ave,Woodruff Ave,NW,40.654251,-73.961909
BMT,Brighton,Church Av,40.650527,-73.962982,B,Q,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,St. Pauls Pl,Caton Ave,SE,40.651482,-73.962732
BMT,Brighton,Av J,40.625039,-73.960803,B,Q,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,15th St,Avenue J,NE,40.625307,-73.960969
BMT,Brighton,Av J,40.625039,-73.960803,B,Q,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,East 16 St,Avenue J,NW,40.625363,-73.960654
BMT,Brighton,Av M,40.617618,-73.959399,B,Q,,,,,,,,,,Door,NO,,NO,NONE,,FALSE,,TRUE,East 16 St,Avenue M,NW,40.61814,-73.95946
BMT,Brighton,Kings Highway,40.60867,-73.957734,B,Q,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,15th St,Kings Hwy,NE,40.609062,-73.957808
BMT,Brighton,Sheepshead Bay,40.586896,-73.954155,B,Q,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,14th St,Voorhies Ave,NE,40.585734,-73.954371
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Branch 6 St,Brighton Beach Rd,NW,40.577618,-73.961994
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Branch 6 St,Brighton Beach Rd,NW,40.577656,-73.961792
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Branch 6 St,Brighton Beach Rd,SW,40.577387,-73.961912
BMT,Brighton,Brighton Beach,40.577621,-73.961376,B,Q,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Branch 6 St,Brighton Beach Rd,SW,40.577425,-73.961719
BMT,Brighton,Ocean Parkway,40.576312,-73.968501,Q,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Ocean Pkwy,Brighton Beach Rd,NW,40.576507,-73.969445
BMT,Brighton,Ocean Parkway,40.576312,-73.968501,Q,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Ocean Pkwy,Brighton Beach Rd,NW,40.576487,-73.969243
BMT,Brighton,West 8th St,40.576127,-73.975939,F,Q,,,,,,,,,,Ramp,YES,,NO,NONE,,FALSE,,TRUE,,,,40.575752,-73.97674
BMT,Brighton,West 8th St,40.576127,-73.975939,F,Q,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,,,,40.576464,-73.974862
BMT,Brighton,Stillwell Av,40.577422,-73.981233,D,F,N,Q,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Stillwell Ave,Surf Ave,NE,40.576241,-73.981077
BMT,West End,62nd St,40.626472,-73.996895,D,N,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,14th Ave,61st St,NE,40.627299,-73.996465
BMT,West End,62nd St,40.626472,-73.996895,D,N,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,14th Ave,61st St,NW,40.62711,-73.99687
BMT,West End,62nd St,40.626472,-73.996895,D,N,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,14th Ave,61st St,NW,40.627367,-73.996729
BMT,Sea Beach,Fort Hamilton Parkway,40.631386,-74.005351,N,,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,11th Ave,62nd St,NW,40.630609,-74.004067
BMT,Sea Beach,New Utrecht Av,40.624842,-73.996353,D,N,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,63rd St,15th Ave,SE,40.624246,-73.995962
BMT,Sea Beach,18th Av,40.620671,-73.990414,N,,,,,,,,,,,Door,YES,,NO,NONE,,FALSE,,TRUE,17th Ave,64th St,NE,40.621306,-73.991469
BMT,Sea Beach,Bay Parkway-22nd Av,40.611815,-73.981848,N,,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,8th St,Avenue O,NE,40.610572,-73.981613
BMT,Sea Beach,Kings Highway,40.603923,-73.980353,N,,,,,,,,,,,Door,YES,,NO,NONE,,FALSE,,TRUE,7th St,Highlawn Ave,SW,40.603212,-73.980199
BMT,Sea Beach,Av U,40.597473,-73.979137,N,,,,,,,,,,,Door,YES,,NO,NONE,,FALSE,,TRUE,8th St,Avenue T,SE,40.598512,-73.979335
BMT,Broadway Jamaica,121st St,40.700492,-73.828294,J,Z,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,123rd St,Jamaica Ave,NW,40.700808,-73.82739
BMT,Broadway Jamaica,Woodhaven Blvd,40.693879,-73.851576,J,Z,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,95th St,Jamaica Ave,SW,40.694077,-73.850476
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,FALSE,Marcy Ave,Broadway,SW,40.708293,-73.958044
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,FALSE,Havemeyer St,Broadway,NE,40.708788,-73.958826
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Marcy Ave,Broadway,SE,40.708271,-73.957749
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Marcy Ave,Broadway,SW,40.708403,-73.958124
BMT,Broadway Jamaica,Marcy Av,40.708359,-73.957757,J,M,Z,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,FALSE,Havemeyer St,Broadway,SE,40.708681,-73.958927
BMT,Nassau,Canal St,40.718092,-73.999892,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Centre St,Canal St,SW,40.717823,-74.000326
BMT,Nassau,Chambers St,40.713243,-74.003401,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Centre St,Chambers St,SE,40.712699,-74.004099
BMT,Nassau,Chambers St,40.71419,-74.003199,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lafayette St,Duane St,NE,40.714199,-74.003187
BMT,Nassau,Fulton St,40.710374,-74.007582,A,C,J,Z,2,3,4,5,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Fulton St,Nassau St,SW,40.710241,-74.008022
BMT,Nassau,Fulton St,40.710374,-74.007582,A,C,J,Z,2,3,4,5,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Fulton St,Nassau St,SE,40.710106,-74.007648
BMT,Nassau,Fulton St,40.710374,-74.007582,A,C,J,Z,2,3,4,5,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,John St,Nassau St,SE,40.709376,-74.008375
BMT,Nassau,Fulton St,40.710374,-74.007582,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Fulton St,Nassau St,NW,40.710388,-74.007935
BMT,Nassau,Fulton St,40.710374,-74.007582,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Fulton St,Nassau St,NE,40.710211,-74.007584
BMT,Nassau,Fulton St,40.710374,-74.007582,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,John St,Nassau St,NE,40.709507,-74.008295
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broad St,Exchange Pl,SE,40.706142,-74.011128
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broad St,Exchange Pl,SE,40.705961,-74.011229
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,Yes,YES,NONE,,FALSE,,TRUE,Broad St,Exchange Pl,SW,40.706001,-74.011497
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,Yes,YES,NONE,,FALSE,,TRUE,Broad St,Exchange Pl,NW,40.706439,-74.011263
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,Yes,YES,NONE,,FALSE,,TRUE,Broad St,Exchange Pl,SW,40.705747,-74.011554
BMT,Canarsie,6th Av,40.737335,-73.996786,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,14th St,SE,40.737197,-73.996704
BMT,Canarsie,6th Av,40.737335,-73.996786,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,14th St,SE,40.737149,-73.996578
BMT,Canarsie,Union Square,40.734789,-73.99073,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,NONE,,TRUE,Lex not ADA,TRUE,Broadway,14th St,SW,40.734755,-73.990991
BMT,Canarsie,Union Square,40.734789,-73.99073,L,N,Q,R,4,5,6,,,,,Stair,YES,,YES,NONE,,TRUE,Lex not ADA,TRUE,Broadway,14th St,SW,40.734859,-73.991235
BMT,Canarsie,3rd Av,40.732849,-73.986122,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,3rd Ave,14th St,NE,40.733122,-73.986548
BMT,Canarsie,3rd Av,40.732849,-73.986122,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,3rd Ave,14th St,NE,40.733277,-73.986931
BMT,Canarsie,1st Av,40.730953,-73.981628,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,1st Ave,14th St,SE,40.73105,-73.982625
BMT,Canarsie,1st Av,40.730953,-73.981628,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,1st Ave,14th St,SE,40.731118,-73.982563
BMT,Canarsie,Bedford Av,40.717304,-73.956872,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Driggs Ave,N 7th St,SE,40.716888,-73.956367
BMT,Canarsie,Bedford Av,40.717304,-73.956872,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Driggs Ave,N 7th St,NE,40.716991,-73.956257
BMT,Canarsie,Lorimer St,40.714063,-73.950275,G,L,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lorimer St,Metropolitan Ave,SE,40.713908,-73.949262
BMT,Canarsie,Lorimer St,40.714063,-73.950275,G,L,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lorimer St,Metropolitan Ave,NE,40.71421,-73.949313
BMT,Canarsie,Graham Av,40.714565,-73.944053,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Graham Ave,Metropolitan Ave,SE,40.714439,-73.94419
BMT,Canarsie,Graham Av,40.714565,-73.944053,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Graham Ave,Metropolitan Ave,NW,40.714433,-73.944546
BMT,Canarsie,Grand St,40.711926,-73.94067,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Grand St,Bushwick Ave,NW,40.711791,-73.940742
BMT,Canarsie,Grand St,40.711926,-73.94067,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Grand St,Bushwick Ave,NE,40.712017,-73.940976
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Stanhope St,Wycoff Ave,NE,40.703461,-73.917606
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Stanhope St,Wycoff Ave,NW,40.70354,-73.917771
BMT,Canarsie,Myrtle Av,40.699814,-73.911586,L,M,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Palmetto St,Wycoff Ave,NW,40.699814,-73.911295
BMT,Canarsie,Myrtle Av,40.699814,-73.911586,L,M,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Palmetto St,Wycoff Ave,NE,40.699468,-73.910648
BMT,Canarsie,Myrtle Av,40.699814,-73.911586,L,M,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Palmetto St,Wycoff Ave,SE,40.699807,-73.910999
BMT,Canarsie,Halsey St,40.695602,-73.904084,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Halsey St,Wycoff Ave,SW,40.695871,-73.905143
BMT,Canarsie,Halsey St,40.695602,-73.904084,L,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Halsey St,Wycoff Ave,SE,40.69599,-73.905265
BMT,Canarsie,Livonia Av,40.664038,-73.900571,L,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Van Sinderen Ave,Livonia Ave,SW,40.663749,-73.900482
IND,8 Avenue,Inwood - 207th St,40.868072,-73.919899,A,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,Isham St,NW,40.868284,-73.919607
IND,8 Avenue,Inwood - 207th St,40.868072,-73.919899,A,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,211th St,NE,40.868036,-73.919284
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,Dyckman St,SE,40.865296,-73.927411
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,Dyckman St,SE,40.865248,-73.927279
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,Dyckman St,NE,40.865342,-73.926964
IND,8 Avenue,Dyckman St-200th St,40.865491,-73.927271,A,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,Dyckman St,NE,40.865446,-73.927035
IND,8 Avenue,181st St,40.851695,-73.937969,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Fort Washington Ave,181st St,SW,40.850778,-73.938472
IND,8 Avenue,181st St,40.851695,-73.937969,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Fort Washington Ave,181st St,SE,40.850717,-73.938214
IND,8 Avenue,181st St,40.851695,-73.937969,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Fort Washington Ave,181st St,SW,40.85087,-73.938537
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Fort Washington Ave,175th St,SW,40.846352,-73.940287
IND,8 Avenue,175th St,40.847391,-73.939704,A,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Fort Washington Ave,175th St,SE,40.846407,-73.939861
IND,8 Avenue,168th St - Washington Heights,40.840719,-73.939561,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Saint Nicholas Ave,169th St,NW,40.84186,-73.939569
IND,8 Avenue,168th St - Washington Heights,40.840719,-73.939561,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Saint Nicholas Ave,169th St,NW,40.841928,-73.939455
IND,8 Avenue,168th St - Washington Heights,40.840719,-73.939561,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,169th St,NE,40.84176,-73.939067
IND,8 Avenue,145th St,40.824783,-73.944216,A,B,C,D,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Saint Nicholas Ave,147th St,NW,40.825622,-73.943796
IND,8 Avenue,145th St,40.824783,-73.944216,A,B,C,D,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Saint Nicholas Ave,147th St,NE,40.825581,-73.943444
IND,8 Avenue,135th St,40.817894,-73.947649,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Saint Nicholas Ave,137th St,SW,40.818558,-73.947543
IND,8 Avenue,135th St,40.817894,-73.947649,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Edgecombe Ave,135th St,NE,40.817188,-73.947665
IND,8 Avenue,135th St,40.817894,-73.947649,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Saint Nicholas Ave,135th St,SW,40.816892,-73.947818
IND,8 Avenue,135th St,40.817894,-73.947649,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Saint Nicholas Ave,135th St,SE,40.816867,-73.947953
IND,8 Avenue,135th St,40.817894,-73.947649,B,C,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Saint Nicholas Ave,137th St,SE,40.818354,-73.947213
IND,8 Avenue,125th St,40.811109,-73.952343,A,B,C,D,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Saint Nicholas Ave,127th St,SE,40.811858,-73.951351
IND,8 Avenue,125th St,40.811109,-73.952343,A,B,C,D,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Saint Nicholas Ave,127th St,SW,40.81199,-73.951745
IND,8 Avenue,116th St,40.805085,-73.954882,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Frederick Douglass Blvd,116th St,NE,40.804428,-73.955024
IND,8 Avenue,116th St,40.805085,-73.954882,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Frederick Douglass Blvd,116th St,SE,40.804172,-73.955131
IND,8 Avenue,Cathedral Parkway-110th St,40.800605,-73.958158,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Frederick Douglass Blvd,Cathedral Pkwy,SW,40.800866,-73.958405
IND,8 Avenue,96th St,40.791646,-73.964699,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Central Park West,97th St,SW,40.792285,-73.964423
IND,8 Avenue,96th St,40.791646,-73.964699,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Central Park West,97th St,NW,40.792591,-73.964316
IND,8 Avenue,86th St,40.785868,-73.968916,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Central Park West,88th St,NW,40.786769,-73.968448
IND,8 Avenue,86th St,40.785868,-73.968916,B,C,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Central Park West,87th St,NW,40.786115,-73.969011
IND,8 Avenue,81st St - Museum of Natural History,40.781433,-73.972143,B,C,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,Central Park West,77th St,NW,40.780895,-73.972918
IND,8 Avenue,81st St - Museum of Natural History,40.781433,-73.972143,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Central Park West,77th St,NW,40.780319,-73.973179
IND,8 Avenue,72nd St,40.775594,-73.97641,B,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Central Park West,70th St,SW,40.77494,-73.977144
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,NO,NONE,,TRUE,,TRUE,Central Park West,60th St,NE,40.768649,-73.981184
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,Southbound Only,FALSE,8th Ave,50th St,SE,40.762085,-73.985829
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,Southbound Only,FALSE,8th Ave,50th St,NE,40.762216,-73.985757
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,44th St,SW,40.758567,-73.989053
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,44th St,SE,40.758401,-73.988673
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,44th St,NW,40.758685,-73.98899
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,35th St,SE,40.752689,-73.992742
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,35th St,NE,40.752796,-73.992669
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,35th St,NE,40.752766,-73.992596
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,35th St,SW,40.752877,-73.993244
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,35th St,NW,40.753053,-73.993256
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,35th St,NW,40.753032,-73.993188
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,34th St,SW,40.752178,-73.993701
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,8th Ave,34th St,NW,40.752427,-73.993679
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,25th St,NW,40.746732,-73.997711
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,25th St,SW,40.746603,-73.997801
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,23rd St,SW,40.745278,-73.998771
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,23rd St,NW,40.745505,-73.998642
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,25th St,NE,40.746535,-73.997271
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,25th St,NE,40.746483,-73.997142
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,8th Ave,24th St,NW,40.74611,-73.998145
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,8th Ave,24th St,SE,40.745814,-73.997852
IND,8 Avenue,23rd St,40.745906,-73.998041,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,8th Ave,25th St,SE,40.746427,-73.99735
IND,8 Avenue,West 4th St,40.732338,-74.000495,A,B,C,D,E,F,M,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,Waverly Pl,NE,40.733067,-73.999728
IND,8 Avenue,West 4th St,40.732338,-74.000495,A,B,C,D,E,F,M,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,Waverly Pl,NE,40.733259,-73.999615
IND,8 Avenue,West 4th St,40.732338,-74.000495,A,B,C,D,E,F,M,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,Waverly Pl,NW,40.733072,-74.000192
IND,8 Avenue,Spring St,40.726227,-74.003739,C,E,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,Spring St,NW,40.725595,-74.004326
IND,8 Avenue,Spring St,40.726227,-74.003739,C,E,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,6th Ave,Vandam St,SW,40.726208,-74.00399
IND,8 Avenue,Canal St,40.720824,-74.005229,A,C,E,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,Walker St,NE,40.71987,-74.005038
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Chambers St,SW,40.714873,-74.008162
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Chambers St,SE,40.714777,-74.007903
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Chambers St,NW,40.715099,-74.007989
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Chambers St,NE,40.714992,-74.007739
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Murray St,NW,40.713949,-74.00896
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Warren St,SW,40.714313,-74.008584
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Church St,Warren St,NW,40.714539,-74.008428
IND,8 Avenue,High St,40.699337,-73.990531,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Red Cross Pl,Adams St,SW,40.699112,-73.98892
IND,8 Avenue,High St,40.699337,-73.990531,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Sands St,Adams St,SE,40.699212,-73.988274
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Jay St,Metrotech Walk,SW,40.693243,-73.987402
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Jay St,Metrotech Walk,SE,40.693491,-73.987089
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Jay St,Metrotech Walk,SW,40.693153,-73.987408
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Escalator,YES,,YES,NONE,,FALSE,,TRUE,Jay St,Metrotech Walk,SW,40.693153,-73.987484
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Escalator,YES,,YES,NONE,,FALSE,,TRUE,Jay St,Metrotech Walk,SW,40.693243,-73.987481
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Jay St,Willoughby St,NE,40.692397,-73.987162
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Jay St,Willoughby St,NE,40.692329,-73.987167
IND,Fulton,Clinton & Washington Avs,40.683263,-73.965838,C,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Washington Ave,Fulton Ave,SE,40.682868,-73.96463
IND,Fulton,Clinton & Washington Avs,40.683263,-73.965838,C,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Washington Ave,Fulton Ave,NE,40.683172,-73.964689
IND,Fulton,Franklin Av,40.68138,-73.956848,A,C,FS,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Franklin Ave,Fulton St,NE,40.681256,-73.95559
IND,Fulton,Nostrand Av,40.680438,-73.950426,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Nostrand Ave,Fulton St,SE,40.680228,-73.949414
IND,Fulton,Nostrand Av,40.680438,-73.950426,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Nostrand Ave,Fulton St,SW,40.680262,-73.949635
IND,Fulton,Kingston-Throop,40.679921,-73.940858,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Kingston Ave,Fulton St,SE,40.679804,-73.941119
IND,Fulton,Kingston-Throop,40.679921,-73.940858,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Kingston Ave,Fulton St,SW,40.679728,-73.941362
IND,Fulton,Utica Av,40.679364,-73.930729,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Stuyvesant Ave,Fulton St,NW,40.67953,-73.932021
IND,Fulton,Utica Av,40.679364,-73.930729,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Stuyvesant Ave,Fulton St,SW,40.679346,-73.931987
IND,Fulton,Rockaway Av,40.67834,-73.911946,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Rockaway Ave,Fulton St,NE,40.678513,-73.910773
IND,Fulton,Rockaway Av,40.67834,-73.911946,A,C,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Rockaway Ave,Fulton St,NW,40.67846,-73.910971
IND,Liberty,80th St-Hudson St,40.679371,-73.858992,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,77th St,Liberty Ave,SE,40.679215,-73.860291
IND,Liberty,80th St-Hudson St,40.679371,-73.858992,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,77th St,Liberty Ave,NE,40.679361,-73.860284
IND,Liberty,Rockaway Blvd,40.680429,-73.843853,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,97th St,Liberty Ave,NW,40.680611,-73.842537
IND,Liberty,Rockaway Blvd,40.680429,-73.843853,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,97th St,Liberty Ave,SW,40.680774,-73.842558
IND,Liberty,111th St-Greenwood Av,40.684331,-73.832163,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,109th St,Liberty Ave,SE,40.683972,-73.832816
IND,Liberty,111th St-Greenwood Av,40.684331,-73.832163,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,109th St,Liberty Ave,NW,40.684111,-73.832889
IND,Liberty,Lefferts Blvd,40.685951,-73.825798,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,116th St,Liberty Ave,SW,40.68572,-73.827092
IND,Liberty,Lefferts Blvd,40.685951,-73.825798,A,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,116th St,Liberty Ave,NW,40.685868,-73.827107
IND,Rockaway,Aqueduct-North Conduit Av,40.668234,-73.834058,A,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,99th Pl,Hawtree St,NE,40.669339,-73.834721
IND,Rockaway,Howard Beach,40.660476,-73.830301,A,,,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,TRUE,103rd St,159th Ave,SE,40.66016,-73.830372
IND,Concourse,Norwood-205th St,40.874811,-73.878855,D,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Perry Ave,205th St,SE,40.874021,-73.877501
IND,Concourse,Norwood-205th St,40.874811,-73.878855,D,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Perry Ave,205th St,NW,40.874262,-73.877533
IND,Concourse,Bedford Park Blvd,40.873244,-73.887138,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,203rd St,NW,40.874084,-73.887112
IND,Concourse,Bedford Park Blvd,40.873244,-73.887138,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,203rd St,NE,40.874025,-73.886466
IND,Concourse,Kingsbridge Rd,40.866978,-73.893509,B,D,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,in planning,TRUE,Grand Concourse,Kingsbridge Rd,NE,40.866181,-73.894306
IND,Concourse,Kingsbridge Rd,40.866978,-73.893509,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,in planning,TRUE,Grand Concourse,Kingsbridge Rd,NW,40.866369,-73.894521
IND,Concourse,Kingsbridge Rd,40.866978,-73.893509,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,in planning,TRUE,Grand Concourse,Kingsbridge Rd,NE,40.866127,-73.893982
IND,Concourse,Fordham Rd,40.861296,-73.897749,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,Fordham Rd,SE,40.862296,-73.896885
IND,Concourse,Fordham Rd,40.861296,-73.897749,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,Fordham Rd,NE,40.862608,-73.896662
IND,Concourse,Tremont Av,40.85041,-73.905227,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,179th St,SW,40.851052,-73.905093
IND,Concourse,Tremont Av,40.85041,-73.905227,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,179th St,NE,40.851176,-73.904263
IND,Concourse,174-175th Sts,40.8459,-73.910136,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,175th St,SW,40.846461,-73.909796
IND,Concourse,174-175th Sts,40.8459,-73.910136,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,175th St,NE,40.846354,-73.909047
IND,Concourse,170th St,40.839306,-73.9134,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,170th St,SW,40.838618,-73.914191
IND,Concourse,170th St,40.839306,-73.9134,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,170th St,NW,40.838991,-73.914037
IND,Concourse,170th St,40.839306,-73.9134,B,D,,,,,,,,,,Stair,YES,Yes,YES,NONE,,FALSE,,TRUE,Grand Concourse,170th St,SE,40.838421,-73.913626
IND,Concourse,167th St,40.833769,-73.918432,B,D,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,McClellan St,SW,40.833111,-73.919399
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Door,YES,,YES,NONE,,TRUE,,TRUE,Walton Ave,161st St,NW,40.827372,-73.924217
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Walton Ave,161st St,NW,40.827572,-73.924028
IND,Concourse,Yankee Stadium-161st St,40.827905,-73.925651,B,D,4,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Walton Ave,161st St,NW,40.827539,-73.924156
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,48th St,SW,40.758623,-73.981566
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,48-49th Sts,W,40.758977,-73.981352
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,48-49th Sts,W,40.759105,-73.981252
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,47th St,SE,40.757801,-73.981788
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,47th St,SW,40.758024,-73.982079
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,47th St,NE,40.757981,-73.98154
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,47th St,SE,40.757862,-73.981735
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,48th St,SE,40.758486,-73.981158
IND,6 Avenue,47-50th Sts Rockefeller Center,40.758663,-73.981329,B,D,F,M,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,48th St,NE,40.758615,-73.981095
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,40th St,NW,40.753709,-73.985084
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,42nd St,NW,40.755333,-73.983934
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,42nd St,NW,40.755008,-73.984249
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,42nd St,NE,40.754832,-73.983657
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,40th St,SW,40.753473,-73.985259
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,40th St,SE,40.753408,-73.984892
IND,6 Avenue,42nd St,40.754222,-73.984569,B,D,F,M,7,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,40th St,NE,40.753546,-73.984779
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,35th St,SW,40.750674,-73.988031
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,35th St,SE,40.750271,-73.987066
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,35th St,NW,40.75088,-73.988096
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,35th St,NW,40.750647,-73.987346
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,6th Ave,35th St,NE,40.750404,-73.986989
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,6th Ave,23rd St,NW,40.743121,-73.993099
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,6th Ave,23rd St,NW,40.743066,-73.992971
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,6th Ave,23rd St,SW,40.742977,-73.993188
IND,6 Avenue,23rd St,40.742878,-73.992821,F,M,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,6th Ave,23rd St,SW,40.742922,-73.993066
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,16th St,NE,40.738678,-73.995632
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,16th St,SE,40.738529,-73.995685
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,16th St,NW,40.738838,-73.996023
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,16th St,SW,40.738718,-73.996121
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,14th St,NE,40.737286,-73.996437
IND,6 Avenue,14th St,40.738228,-73.996209,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,6th Ave,14th St,NE,40.737316,-73.996508
IND,6 Avenue,Broadway-Lafayette St,40.725297,-73.996204,B,D,F,M,6,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Crosby St,Houston St,NE,40.725344,-73.995612
IND,6 Avenue,2nd Av,40.723402,-73.989938,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,2nd Ave,Houston St,NW,40.72395,-73.991298
IND,6 Avenue,2nd Av,40.723402,-73.989938,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Chrystie St,Houston St,SW,40.723665,-73.991651
IND,6 Avenue,Delancey St,40.718611,-73.988114,F,J,M,Z,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Essex St,Delancey St,SW,40.718372,-73.988445
IND,6 Avenue,Delancey St,40.718611,-73.988114,F,J,M,Z,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Essex St,Delancey St,SE,40.718207,-73.988273
IND,6 Avenue,East Broadway,40.713715,-73.990173,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Rutgers St,Broadway,SE,40.713906,-73.990014
IND,6 Avenue,East Broadway,40.713715,-73.990173,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Rutgers St,Broadway,NW,40.714194,-73.990352
IND,6 Avenue,East Broadway,40.713715,-73.990173,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Essex St,Canal St,NE,40.714404,-73.989918
IND,6 Avenue,Bergen St,40.686145,-73.990862,F,G,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Smith St,Bergen St,SW,40.686708,-73.990629
IND,6 Avenue,Bergen St,40.686145,-73.990862,F,G,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Smith St,Bergen St,NW,40.686868,-73.990526
IND,6 Avenue,Bergen St,40.686145,-73.990862,F,G,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Smith St,Warren St,NW,40.685579,-73.991465
IND,6 Avenue,Bergen St,40.686145,-73.990862,F,G,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Smith St,Warren St,NE,40.685445,-73.991131
IND,6 Avenue,Carroll St,40.680303,-73.995048,F,G,,,,,,,,,,Stair,YES,,YES,NONE,Staffed During Construction,FALSE,,TRUE,Smith St,President St,SE,40.680806,-73.99435
IND,6 Avenue,Carroll St,40.680303,-73.995048,F,G,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Smith St,President St,NE,40.680944,-73.994247
IND,6 Avenue,Carroll St,40.680303,-73.995048,F,G,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Smith St,President St,NW,40.681115,-73.994434
IND,6 Avenue,Prospect Park-15 St,40.660365,-73.979493,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Prospect Park West,Prospect Park Southwest,SE,40.660705,-73.979522
IND,6 Avenue,Prospect Park-15 St,40.660365,-73.979493,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Prospect Park West,Prospect Park Southwest,NE,40.660763,-73.978992
IND,6 Avenue,Prospect Park-15 St,40.660365,-73.979493,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Prospect Park West,14th St,SE,40.661287,-73.979395
IND,6 Avenue,Prospect Park-15 St,40.660365,-73.979493,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Prospect Park West,14th St,SW,40.6614,-73.979762
IND,6 Avenue,Fort Hamilton Parkway,40.650782,-73.975776,F,,,,,,,,,,,Door,YES,,YES,NONE,,FALSE,,TRUE,Prospect Expwy,Fort Hamilton Pkwy,NW,40.649854,-73.975815
IND,6 Avenue,Church Av,40.644041,-73.979678,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,McDonald Ave,Albemarle Rd,SE,40.645042,-73.979573
IND,6 Avenue,Church Av,40.644041,-73.979678,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,McDonald Ave,Albemarle Rd,SW,40.644908,-73.980019
IND,Culver,Ditmas Av,40.636119,-73.978172,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Cortelyou Rd,SW,40.637093,-73.978463
IND,Culver,Ditmas Av,40.636119,-73.978172,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Cortelyou Rd,SE,40.637107,-73.978257
IND,Culver,18th Av,40.629755,-73.976971,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Parkville Ave,NW,40.628674,-73.976892
IND,Culver,18th Av,40.629755,-73.976971,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Parkville Ave,NE,40.628708,-73.976646
IND,Culver,Av N,40.61514,-73.974197,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Avenue M,SW,40.616121,-73.974507
IND,Culver,Av N,40.61514,-73.974197,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Avenue M,SE,40.61616,-73.97427
IND,Culver,Kings Highway,40.603217,-73.972361,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Avenue S,NW,40.602197,-73.97264
IND,Culver,Kings Highway,40.603217,-73.972361,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,McDonald Ave,Avenue S,NE,40.602178,-73.972379
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,179th St,Hillside Ave,NE,40.71264,-73.784411
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,179th St,Hillside Ave,SE,40.712295,-73.78428
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,179th St,Hillside Ave,NE,40.712575,-73.784669
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,179th St,Hillside Ave,SW,40.71224,-73.784456
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,179th St,Hillside Ave,NW,40.712483,-73.785064
IND,Queens Boulevard,Jamaica-179th St,40.712646,-73.783817,F,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,179th St,Hillside Ave,SW,40.712242,-73.784968
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,168th St,Hillside Ave,SW,40.709812,-73.794712
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,168th St,Hillside Ave,NW,40.710154,-73.794922
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,168th St,Hillside Ave,SE,40.709852,-73.794564
IND,Queens Boulevard,169th St,40.71047,-73.793604,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,168th St,Hillside Ave,NE,40.710184,-73.79473
IND,Queens Boulevard,Parsons Blvd,40.707564,-73.803326,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,153rd St,Hillside Ave,SE,40.707062,-73.804332
IND,Queens Boulevard,Parsons Blvd,40.707564,-73.803326,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,153rd St,Hillside Ave,NE,40.707358,-73.804498
IND,Queens Boulevard,Parsons Blvd,40.707564,-73.803326,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,153rd St,Hillside Ave,SW,40.707063,-73.80462
IND,Queens Boulevard,Sutphin Blvd,40.70546,-73.810708,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,144th St,Hillside Ave,NE,40.705341,-73.81186
IND,Queens Boulevard,Sutphin Blvd,40.70546,-73.810708,F,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,144th St,Hillside Ave,SW,40.704906,-73.811956
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Union Tpke,Queens Blvd,SE,40.714145,-73.830886
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Union Tpke,Queens Blvd,SW,40.715019,-73.83162
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Union Tpke,Queens Blvd,NW,40.714673,-73.832151
IND,Queens Boulevard,Kew Gardens-Union Turnpike,40.714441,-73.831008,E,F,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Union Tpke,Queens Blvd,NW,40.714929,-73.831501
IND,Queens Boulevard,Forest Hills-71st Av,40.721691,-73.844521,e,F,M,R,,,,,,,,Stair,YES,,YES,NONE,,FALSE,in planning,TRUE,70th Rd,Queens Blvd,NW,40.722269,-73.845153
IND,Queens Boulevard,Forest Hills-71st Av,40.721691,-73.844521,E,F,M,R,,,,,,,,Stair,YES,,YES,NONE,,FALSE,in planning,TRUE,71st,Queens Blvd,SW,40.721409,-73.844632
IND,Queens Boulevard,Forest Hills-71st Av,40.721691,-73.844521,E,F,M,R,,,,,,,,Stair,YES,,YES,NONE,,FALSE,in planning,TRUE,108th St,Queens Blvd,NW,40.721916,-73.844352
IND,Queens Boulevard,67th Av,40.726523,-73.852719,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,67th Dr,Queens Blvd,SW,40.72603,-73.852186
IND,Queens Boulevard,67th Av,40.726523,-73.852719,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,67th Dr,Queens Blvd,NW,40.726367,-73.851592
IND,Queens Boulevard,Woodhaven Blvd,40.733106,-73.869229,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Horace Harding Expwy,Queens Blvd,NE,40.733188,-73.868605
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,73rd St,37th Rd,NE,40.747238,-73.892146
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,73rd St,Broadway,SW,40.747034,-73.892555
IND,Queens Boulevard,Northern Blvd,40.752885,-73.906006,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,54th St,Broadway,SW,40.753428,-73.906847
IND,Queens Boulevard,46th St,40.756312,-73.913333,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,48th St,Newtown Rd,NE,40.756029,-73.91218
IND,Queens Boulevard,46th St,40.756312,-73.913333,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,46th St,Broadway,SW,40.756624,-73.914407
IND,Queens Boulevard,46th St,40.756312,-73.913333,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,48th St,Broadway,SE,40.755813,-73.912642
IND,Queens Boulevard,Steinway St,40.756879,-73.92074,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Steinway St,34th Ave,SW,40.756583,-73.921108
IND,Queens Boulevard,Steinway St,40.756879,-73.92074,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Steinway St,34th Ave,NE,40.756673,-73.920622
IND,Queens Boulevard,36th St,40.752039,-73.928781,M,R,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,34th St,Northern Blvd,SW,40.751868,-73.930585
IND,Queens Boulevard,36th St,40.752039,-73.928781,M,R,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,36th St,Northern Blvd,NE,40.752238,-73.927662
IND,Queens Boulevard,36th St,40.752039,-73.928781,M,R,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,36th St,Northern Blvd,SE,40.75193,-73.927141
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Northern Blvd,41st Ave,SE,40.749577,-73.936094
IND,Queens Boulevard,Queens Plaza,40.748973,-73.937243,E,M,R,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Northern Blvd,41st Ave,NW,40.749857,-73.936198
IND,Queens Boulevard,23rd St-Ely Av,40.747846,-73.946,E,G,M,7,,,,,,,,Stair,YES,Yes,YES,NONE,,FALSE,,TRUE,23rd St,44th Ave,NE,40.747706,-73.944626
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,52nd St,NE,40.757784,-73.971098
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,TRUE,Lexington Avenue,52nd Street,NE,40.757804,-73.971141
IND,Queens Boulevard,5th Av-53rd St,40.760167,-73.975224,E,M,,,,,,,,,,Easement,YES,,NO,NONE,,FALSE,,TRUE,Madison Ave,53rd St,NE,40.759721,-73.973858
IND,Queens Boulevard,5th Av-53rd St,40.760167,-73.975224,E,M,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Madison Ave,53rd St,SE,40.759556,-73.974068
IND,Queens Boulevard,7th Av,40.762862,-73.981637,B,D,E,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,53rd St,SE,40.763141,-73.982666
IND,Queens Boulevard,7th Av,40.762862,-73.981637,B,D,E,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,53rd St,NE,40.763388,-73.982495
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,153rd St,Archer Ave,NE,40.702234,-73.801859
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,153rd St,Archer Ave,SW,40.701874,-73.801747
IND,Archer Av,Parsons Blvd-Archer Av - Jamaica Center,40.702147,-73.801109,E,J,Z,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,158th St,Archer Ave,SW,40.70164,-73.802478
IND,Crosstown,Greenpoint Av,40.731352,-73.954449,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Manhattan Ave,India St,SE,40.732258,-73.954379
IND,Crosstown,Greenpoint Av,40.731352,-73.954449,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Manhattan Ave,India St,SW,40.732216,-73.954784
IND,Crosstown,Nassau Av,40.724635,-73.951277,G,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Manhattan Ave,Norman Ave,NW,40.725446,-73.951974
IND,Crosstown,Nassau Av,40.724635,-73.951277,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Manhattan Ave,Norman Ave,NE,40.725573,-73.951603
IND,Crosstown,Metropolitan Av,40.712792,-73.951418,G,L,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Union Ave,Metropolitan Ave,NE,40.713747,-73.951768
IND,Crosstown,Metropolitan Av,40.712792,-73.951418,G,L,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Union Ave,Metropolitan Ave,SE,40.713744,-73.951455
IND,Crosstown,Flushing Av,40.700377,-73.950234,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Union Ave,Flushing Ave,NE,40.699985,-73.949895
IND,Crosstown,Myrtle-Willoughby Avs,40.694568,-73.949046,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Marcy Ave,Myrtle Ave,NE,40.6955,-73.94896
IND,Crosstown,Bedford-Nostrand Avs,40.689627,-73.953522,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bedford Ave,Lafayette Ave,NE,40.689543,-73.954998
IND,Crosstown,Bedford-Nostrand Avs,40.689627,-73.953522,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bedford Ave,Lafayette Ave,SE,40.689367,-73.954957
IND,Crosstown,Bedford-Nostrand Avs,40.689627,-73.953522,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bedford Ave,Lafayette Ave,NW,40.6895,-73.955353
IND,Crosstown,Bedford-Nostrand Avs,40.689627,-73.953522,G,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Bedford Ave,Lafayette Ave,SW,40.68933,-73.955318
IRT,Broadway-7th Ave,238th St,40.884667,-73.90087,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Broadway,238th St,NE,40.884868,-73.900598
IRT,Broadway-7th Ave,231st St,40.878856,-73.904834,1,,,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,FALSE,Broadway,231st St,SE,40.878694,-73.904821
IRT,Broadway-7th Ave,231st St,40.878856,-73.904834,1,,,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,FALSE,Broadway,231st St,NE,40.879091,-73.904538
IRT,Broadway-7th Ave,Marble Hill-225th St,40.874561,-73.909831,1,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Broadway,225th St,SE,40.874308,-73.909795
IRT,Broadway-7th Ave,215th St,40.869444,-73.915279,1,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,10th Ave,215th St,SE,40.869315,-73.915216
IRT,Broadway-7th Ave,207th St,40.864614,-73.918819,1,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,10th Ave,207th St,SE,40.86423,-73.918951
IRT,Broadway-7th Ave,168th St,40.840556,-73.940133,A,C,1,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,168th St,SW,40.840859,-73.940209
IRT,Broadway-7th Ave,168th St,40.840556,-73.940133,A,C,1,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,,,,40.841024,-73.940083
IRT,Broadway-7th Ave,157th St,40.834041,-73.94489,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,157th St,SE,40.83386,-73.94462
IRT,Broadway-7th Ave,157th St,40.834041,-73.94489,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,157th St,SE,40.833832,-73.944753
IRT,Broadway-7th Ave,145th St,40.826551,-73.95036,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,145th St,SW,40.826199,-73.950392
IRT,Broadway-7th Ave,145th St,40.826551,-73.95036,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,145th St,NW,40.826513,-73.950162
IRT,Broadway-7th Ave,137th St-City College,40.822008,-73.953676,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,137th St,NE,40.821454,-73.953856
IRT,Broadway-7th Ave,137th St-City College,40.822008,-73.953676,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,137th St,NE,40.821391,-73.953906
IRT,Broadway-7th Ave,116th St-Columbia University,40.807722,-73.96411,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,115th St,SE,40.807535,-73.964013
IRT,Broadway-7th Ave,Cathedral Parkway-110th St,40.803967,-73.966847,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,Cathedral Pkwy,SE,40.803926,-73.966647
IRT,Broadway-7th Ave,Cathedral Parkway-110th St,40.803967,-73.966847,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,Cathedral Pkwy,NE,40.804257,-73.966422
IRT,Broadway-7th Ave,96th St,40.793919,-73.972323,1,2,3,,,,,,,,,Door,YES,,YES,NONE,,TRUE,,TRUE,Broadway,95th St,N,40.794329,-73.972042
IRT,Broadway-7th Ave,86th St,40.788644,-73.976218,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,86th St,SE,40.788073,-73.976288
IRT,Broadway-7th Ave,86th St,40.788644,-73.976218,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,86th St,NE,40.788272,-73.976083
IRT,Broadway-7th Ave,79th St,40.783934,-73.979917,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,79th St,SE,40.78362,-73.97991
IRT,Broadway-7th Ave,79th St,40.783934,-73.979917,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Broadway,79th St,NE,40.783985,-73.979662
IRT,Broadway-7th Ave,72nd St,40.778453,-73.98197,1,2,3,,,,,,,,,Door,YES,,YES,NONE,,TRUE,,TRUE,Broadway,72nd St,SE,40.778585,-73.981923
IRT,Broadway-7th Ave,66th St-Lincoln Center,40.77344,-73.982209,1,,,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,TRUE,Broadway,66th St,SE,40.773536,-73.982006
IRT,Broadway-7th Ave,66th St-Lincoln Center,40.77344,-73.982209,1,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,66th St,SE,40.773457,-73.982004
IRT,Broadway-7th Ave,66th St-Lincoln Center,40.77344,-73.982209,1,,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,66th St,SE,40.773682,-73.982015
IRT,Broadway-7th Ave,59th St-Columbus Circle,40.768247,-73.981929,A,B,C,D,1,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,60th St,SW,40.768777,-73.982106
IRT,Broadway-7th Ave,59th St-Columbus Circle,40.768247,-73.981929,A,B,C,D,1,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Columbus Circle,59th St,SE,40.769227,-73.9824
IRT,Broadway-7th Ave,59th St-Columbus Circle,40.768247,-73.981929,A,B,C,D,1,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Columbus Circle,59th St,SE,40.769365,-73.982392
IRT,Broadway-7th Ave,50th St,40.761728,-73.983849,1,,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,FALSE,Broadway,50th St,SW,40.761432,-73.98437
IRT,Broadway-7th Ave,50th St,40.761728,-73.983849,1,,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,FALSE,Broadway,50th St,NW,40.761561,-73.984154
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Easement,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,40th St,SW,40.754691,-73.988067
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,40th St,SE,40.754568,-73.987827
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,41st St,NW,40.755554,-73.987679
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,42nd St,SE,40.755575,-73.986311
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,32nd St,NE,40.749656,-73.99113
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,33rd St,SE,40.750077,-73.99082
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,33rd St,NE,40.750283,-73.99065
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,33rd St,NE,40.750313,-73.990854
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,TRUE,7th Ave,33rd St,NW,40.750476,-73.991254
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,34th St,SW,40.750982,-73.990899
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,7th Ave,34th St,NW,40.751187,-73.990791
IRT,Broadway-7th Ave,28th St,40.747215,-73.993365,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,28th St,SW,40.747264,-73.993611
IRT,Broadway-7th Ave,28th St,40.747215,-73.993365,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,28th St,NW,40.747392,-73.99351
IRT,Broadway-7th Ave,28th St,40.747215,-73.993365,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,7th Ave,27th St,NW,40.746787,-73.993955
IRT,Broadway-7th Ave,28th St,40.747215,-73.993365,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,7th Ave,27th St,NE,40.746594,-73.993555
IRT,Broadway-7th Ave,23rd St,40.744081,-73.995657,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,23rd St,SW,40.744017,-73.995901
IRT,Broadway-7th Ave,23rd St,40.744081,-73.995657,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,23rd St,NW,40.744313,-73.995811
IRT,Broadway-7th Ave,18th St,40.74104,-73.997871,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,18th St,SW,40.741081,-73.998105
IRT,Broadway-7th Ave,18th St,40.74104,-73.997871,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,18th St,NW,40.741206,-73.998035
IRT,Broadway-7th Ave,18th St,40.74104,-73.997871,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,7th Ave,18th St,NW,40.741783,-73.997557
IRT,Broadway-7th Ave,18th St,40.74104,-73.997871,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,7th Ave,18th St,NE,40.741617,-73.997197
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,7th Ave,14th St,SE,40.738358,-73.999656
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,Yes,NO,NONE,,FALSE,,TRUE,7th Ave,13th St,NW,40.738013,-74.000329
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,7th Ave,12th St,NW,40.737388,-74.000815
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,7th Ave,12th St,NE,40.737197,-74.0004
IRT,Broadway-7th Ave,Christopher St,40.733422,-74.002906,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,Christopher St,SW,40.733515,-74.003147
IRT,Broadway-7th Ave,Christopher St,40.733422,-74.002906,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,Christopher St,SW,40.733467,-74.003076
IRT,Broadway-7th Ave,Christopher St,40.733422,-74.002906,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,Christopher St,NW,40.733634,-74.003123
IRT,Broadway-7th Ave,Christopher St,40.733422,-74.002906,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,7th Ave,Christopher St,NW,40.733652,-74.003006
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,King St,SE,40.727739,-74.005305
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,King St,NE,40.727989,-74.005258
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,King St,SW,40.72784,-74.005681
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,King St,NW,40.727983,-74.00565
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,Houston St,SW,40.728561,-74.00554
IRT,Broadway-7th Ave,Houston St,40.728251,-74.005367,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,Houston St,NW,40.728686,-74.005517
IRT,Broadway-7th Ave,Canal St,40.722854,-74.006277,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Canal St,Varick St,NW,40.722932,-74.00662
IRT,Broadway-7th Ave,Canal St,40.722854,-74.006277,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Canal St,Varick St,NW,40.722878,-74.006518
IRT,Broadway-7th Ave,Franklin St,40.719318,-74.006886,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Varick St,North Moore St,NE,40.719918,-74.00662
IRT,Broadway-7th Ave,Franklin St,40.719318,-74.006886,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,Franklin St,SW,40.718965,-74.007121
IRT,Broadway-7th Ave,Franklin St,40.719318,-74.006886,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Varick St,Franklin St,NW,40.719251,-74.007058
IRT,Broadway-7th Ave,Franklin St,40.719318,-74.006886,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Varick St,North Moore St,NW,40.719888,-74.006925
IRT,Broadway-7th Ave,Rector St,40.707513,-74.013783,1,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Greenwich St,Morris St,NW,40.706493,-74.014319
IRT,Broadway-7th Ave,Rector St,40.707513,-74.013783,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Greenwich St,Morris St,NE,40.706527,-74.014144
IRT,Broadway-7th Ave,Rector St,40.707513,-74.013783,1,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Greenwich St,Morris St,NE,40.706597,-74.014115
IRT,Broadway-7th Ave,Rector St,40.707513,-74.013783,1,,,,,,,,,,,Stair,YES,Yes,NO,NONE,,FALSE,,FALSE,Greenwich St,Rector St,SW,40.707564,-74.013853
IRT,Clark,Wall St,40.706821,-74.0091,2,3,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,William St,Cedar St,NW,40.707431,-74.008778
IRT,Clark,Wall St,40.706821,-74.0091,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,William St,Wall St,SW,40.706385,-74.009658
IRT,Clark,Wall St,40.706821,-74.0091,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,William St,Wall St,SE,40.706272,-74.009447
IRT,Clark,Wall St,40.706821,-74.0091,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,William St,Pine St,SE,40.706755,-74.008804
IRT,Eastern Parkway,Hoyt St,40.690545,-73.985065,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Bridge St,Fulton St,NE,40.690931,-73.98525
IRT,Eastern Parkway,Hoyt St,40.690545,-73.985065,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Bridge St,Fulton St,NW,40.690972,-73.985448
IRT,Eastern Parkway,Hoyt St,40.690545,-73.985065,2,3,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Elm Pl,Fulton St,SW,40.690025,-73.984248
IRT,Eastern Parkway,Nevins St,40.688246,-73.980492,2,3,4,5,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Flatbush Ave,Livingston St,NE,40.688243,-73.980356
IRT,Eastern Parkway,Nevins St,40.688246,-73.980492,2,3,4,5,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Flatbush Ave,Livingston St,NE,40.688085,-73.980243
IRT,Eastern Parkway,Bergen St,40.680829,-73.975098,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Flatbush Ave,Bergen St,SW,40.680832,-73.975534
IRT,Eastern Parkway,Bergen St,40.680829,-73.975098,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Flatbush Ave,Bergen St,SW,40.680709,-73.975241
IRT,Eastern Parkway,Bergen St,40.680829,-73.975098,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Flatbush Ave,Bergen St,NW,40.680951,-73.975448
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Schenectady Ave,Eastern Pkwy,SE,40.668838,-73.934239
IRT,Eastern Parkway,Utica Av,40.668897,-73.932942,3,4,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Schenectady Ave,Eastern Pkwy,NE,40.669102,-73.934213
IRT,Nostrand,Winthrop St,40.656652,-73.9502,2,5,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Nostrand Ave,Parkside Ave,SW,40.656171,-73.950346
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,FALSE,Nostrand Ave,Church Ave,SW,40.65064,-73.949677
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,FALSE,Nostrand,Church,SW,40.650717,-73.949741
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Nostrand Ave,Church Ave,NW,40.650894,-73.949819
IRT,Nostrand,Church Av,40.650843,-73.949575,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Nostrand Ave,Church Ave,NW,40.650969,-73.949714
IRT,Nostrand,Beverly Rd,40.645098,-73.948959,2,5,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,FALSE,Nostrand Ave,Beverly Rd,SW,40.645016,-73.949213
IRT,Nostrand,Newkirk Av,40.639967,-73.948411,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Nostrand Ave,Newkirk Ave,SW,40.639883,-73.948701
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Nostrand Ave,Avenue H,NE,40.631845,-73.947665
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,NW,40.63327,-73.947929
IRT,Nostrand,Flatbush Av-Brooklyn College,40.632836,-73.947642,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Nostrand Ave,Flatbush Ave,NW,40.633297,-73.947836
IRT,Pelham,Hunts Point Av,40.820948,-73.890549,6,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,in planning,TRUE,Hunts Point Ave,163rd St,SW,40.820614,-73.891519
IRT,Pelham,Longwood Av,40.816104,-73.896435,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,Longwood Ave,SE,40.816137,-73.896133
IRT,Pelham,Longwood Av,40.816104,-73.896435,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,Longwood Ave,NE,40.816367,-73.895924
IRT,Pelham,East 149th St,40.812118,-73.904098,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,149th St,SE,40.811887,-73.904145
IRT,Pelham,East 149th St,40.812118,-73.904098,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,149th St,NE,40.812027,-73.903869
IRT,Pelham,East 143rd St-St Mary's St,40.808719,-73.907657,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,St Mary's St,SE,40.80803,-73.90772
IRT,Pelham,East 143rd St-St Mary's St,40.808719,-73.907657,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,St Mary's St,NE,40.80834,-73.907618
IRT,Pelham,Cypress Av,40.805368,-73.914042,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Cypress Ave,138th St,SW,40.805171,-73.913933
IRT,Pelham,Cypress Av,40.805368,-73.914042,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Cypress Ave,138th St,SE,40.805047,-73.913524
IRT,Pelham,Brook Av,40.807566,-73.91924,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Brook Ave,138th St,SW,40.807467,-73.919448
IRT,Pelham,Brook Av,40.807566,-73.91924,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Brook Ave,138th St,SE,40.807355,-73.919219
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,3rd Ave,138th St,NE,40.810831,-73.926655
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Lincoln Ave,138th St,SE,40.810712,-73.92705
IRT,Pelham,138th St-3rd Ave,40.810476,-73.926138,6,,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Lincoln Ave,138th St,SE,40.810638,-73.926883
IRT,Jerome,Mosholu Parkway,40.87975,-73.884655,4,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Jerome Ave,Mosholu Pkwy,SW,40.879503,-73.885138
IRT,Jerome,Mosholu Parkway,40.87975,-73.884655,4,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Jerome Ave,Mosholu Pkwy,SE,40.879343,-73.88488
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,TRUE,River Ave,161st St,SE,40.827685,-73.925842
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,River Ave,161st St,SW,40.827759,-73.9261
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,River Ave,161st St,NW,40.827942,-73.926
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,River Ave,161st St,NE,40.827854,-73.925745
IRT,Jerome,149th St-Grand Concourse,40.818375,-73.927351,2,4,5,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,149th St,SW,40.818259,-73.927659
IRT,Jerome,149th St-Grand Concourse,40.818375,-73.927351,2,4,5,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Grand Concourse,149th St,SW,40.818373,-73.927609
IRT,Lexington,116th St,40.798629,-73.941617,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,116th St,SE,40.798404,-73.941647
IRT,Lexington,116th St,40.798629,-73.941617,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,116th St,NE,40.798742,-73.941406
IRT,Lexington,110th St,40.79502,-73.94425,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,110th St,SE,40.794682,-73.944319
IRT,Lexington,110th St,40.79502,-73.94425,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,110th St,NE,40.794811,-73.944235
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,86th St,NE,40.779661,-73.955302
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,86th St,NE,40.779577,-73.955304
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,86th St,SE,40.779284,-73.955443
IRT,Lexington,86th St,40.779492,-73.955589,4,5,6,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,86th St,SE,40.77933,-73.955545
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,77th St,SE,40.773526,-73.959734
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,77th St,NE,40.773631,-73.959659
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,77th St,SE,40.773493,-73.959659
IRT,Lexington,77th St,40.77362,-73.959874,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lexington Ave,77th St,NE,40.773603,-73.959578
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,Lexington Ave,60th St,SE,40.76266,-73.967625
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lexington Ave,59th St,NW,40.762414,-73.968371
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lexington Ave,59th St,SW,40.762266,-73.968479
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lexington Ave,60th St,NE,40.762835,-73.967554
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Lexington Ave,59th St,SE,40.762069,-73.968016
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,50th St,NW,40.756686,-73.97251
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,51st St,SE,40.756999,-73.971803
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,51st St,NE,40.757126,-73.971721
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,51st St,SE,40.756964,-73.971712
IRT,Lexington,51st St,40.757107,-73.97192,E,M,6,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,51st St,NE,40.757059,-73.97157
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Park Ave,42nd St,SE,40.751741,-73.977691
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Park Ave,42nd St,SE,40.751653,-73.977753
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Lexington Ave,42nd St,SW,40.751352,-73.976396
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Park Ave,42nd St,SE,40.751647,-73.977382
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,32nd St,SE,40.745585,-73.982198
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,32nd St,SE,40.745641,-73.982103
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,33rd St,SW,40.746447,-73.982147
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,33rd St,NW,40.746632,-73.981882
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,32nd St,SW,40.745723,-73.982531
IRT,Lexington,33rd St,40.746081,-73.982076,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,32nd St,SW,40.745821,-73.982576
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,28th St,NW,40.743484,-73.98428
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,28th St,SW,40.743394,-73.984409
IRT,Lexington,28th St,40.74307,-73.984264,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Park Ave,28th St,SW,40.743351,-73.984302
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,In Planning,FALSE,Park Ave,23rd St,NW,40.740419,-73.986452
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Easement,YES,,YES,NONE,,FALSE,In Planning,FALSE,Park Ave,22nd St,SW,40.739501,-73.987134
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,In Planning,FALSE,Park Ave,23rd St,SW,40.740097,-73.986601
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,In Planning,FALSE,Park Ave,23rd St,SW,40.74018,-73.986617
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,In Planning,FALSE,Park Ave,22nd St,NW,40.739674,-73.98704
IRT,Lexington,23rd St,40.739864,-73.986599,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,In Planning,FALSE,Park Ave,22nd St,NE,40.739477,-73.986562
IRT,Lexington,14th St-Union Square,40.734673,-73.989951,L,N,Q,R,4,5,6,,,,,Easement,YES,,YES,NONE,,FALSE,,TRUE,4th Ave,15th St,SW,40.735184,-73.989793
IRT,Lexington,14th St-Union Square,40.734673,-73.989951,L,N,Q,R,4,5,6,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,4th Ave,14th St,SW,40.734495,-73.990311
IRT,Lexington,14th St-Union Square,40.734673,-73.989951,L,N,Q,R,4,5,6,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,4th Ave,14th St,SW,40.734444,-73.990184
IRT,Lexington,Astor Place,40.730054,-73.99107,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lafayette St,Astor Pl,NW,40.729968,-73.99141
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lafayette St,Bleecker St,SW,40.725958,-73.994829
IRT,Lexington,Bleecker St,40.725915,-73.994659,B,D,F,M,6,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lafayette St,Bleecker St,NW,40.726047,-73.994759
IRT,Lexington,Spring St,40.722301,-73.997141,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lafayette St,Spring St,NW,40.722468,-73.997397
IRT,Lexington,Spring St,40.722301,-73.997141,6,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lafayette St,Spring St,SW,40.72239,-73.997437
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Elevator,YES,,YES,NONE,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,NE,40.718304,-74.000138
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,NONE,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,NE,40.718358,-74.000216
IRT,Lexington,Brooklyn Bridge-City Hall,40.713065,-74.004131,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Centre St,Reade St,NW,40.713839,-74.003864
IRT,Lexington,Brooklyn Bridge-City Hall,40.713065,-74.004131,J,Z,4,5,6,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Centre St,Reade St,NW,40.713899,-74.003642
IRT,Lexington,Fulton St,40.710368,-74.009509,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,John St,SE,40.710166,-74.009424
IRT,Lexington,Fulton St,40.710368,-74.009509,A,C,J,Z,2,3,4,5,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,John St,NE,40.710165,-74.009182
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Rector St,SE,40.708436,-74.011298
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Rector St,SW,40.707315,-74.012161
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Rector St,SW,40.707428,-74.012067
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,Rector St,NW,40.707577,-74.011927
IRT,Lexington,Wall St,40.707557,-74.011862,4,5,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Broadway,Rector St,NW,40.707671,-74.011854
IRT,Lexington,Bowling Green,40.704817,-74.014065,4,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,Battery Pl,NW,40.704825,-74.014572
IRT,Lexington,Bowling Green,40.704817,-74.014065,4,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Broadway,Battery Pl,NW,40.704795,-74.014454
IRT,Lexington,Bowling Green,40.704817,-74.014065,4,5,,,,,,,,,,Stair,YES,,NO,NONE,,TRUE,,TRUE,State,Battery Pl,SW,40.704157,-74.014509
IRT,Lexington,Borough Hall,40.692404,-73.990151,R,2,3,4,5,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,Court St,Joralemon St,SW,40.692489,-73.991193
IRT,White Plains Road,Pelham Parkway,40.857192,-73.867615,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,White Plains Rd,Pelham Pkwy,NW,40.858041,-73.867718
IRT,White Plains Road,Pelham Parkway,40.857192,-73.867615,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,White Plains Rd,Pelham Pkwy,NE,40.858029,-73.867481
IRT,White Plains Road,174th St,40.837288,-73.887734,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Boston Rd,174th St,SE,40.837241,-73.88756
IRT,White Plains Road,174th St,40.837288,-73.887734,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Boston Rd,174th St,NE,40.837306,-73.887287
IRT,White Plains Road,Freeman St,40.829993,-73.891865,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,Freeman St,SE,40.830074,-73.891486
IRT,White Plains Road,Freeman St,40.829993,-73.891865,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Southern Blvd,Freeman St,NE,40.830177,-73.891482
IRT,White Plains Road,Simpson St,40.824073,-73.893064,2,5,,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,FALSE,Westchester Ave,Simpson St,SW,40.823884,-73.893197
IRT,White Plains Road,Simpson St,40.824073,-73.893064,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Westchester Ave,Simpson St,SE,40.824118,-73.892731
IRT,White Plains Road,Prospect Av,40.819585,-73.90177,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,,,,40.819368,-73.901966
IRT,White Plains Road,Prospect Av,40.819585,-73.90177,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Prospect Ave,Longwood Ave,NE,40.819562,-73.901498
IRT,White Plains Road,Prospect Av,40.819585,-73.90177,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Prospect Ave,Longwood Ave,NE,40.819684,-73.901348
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Elevator,YES,,YES,NONE,,TRUE,,FALSE,Melrose Ave,149th St,SW,40.815971,-73.917983
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,3rd Ave,149th St,SE,40.815881,-73.917522
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Melrose Ave,149th St,SW,40.816033,-73.918064
IRT,White Plains Road,149th St-3rd Av,40.816109,-73.917757,2,5,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,3rd Ave,149th St,SE,40.815853,-73.917759
IRT,Lenox,135th St,40.814229,-73.94077,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Lenox Ave,135th St,SE,40.813896,-73.940831
IRT,Lenox,135th St,40.814229,-73.94077,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,TRUE,,FALSE,Lenox Ave,135th St,NE,40.814188,-73.940618
IRT,Lenox,125th St,40.807754,-73.945495,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lenox Ave,125th St,SE,40.807548,-73.945463
IRT,Lenox,125th St,40.807754,-73.945495,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lenox Ave,125th St,NE,40.807875,-73.945229
IRT,Lenox,116th St,40.802098,-73.949625,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lenox Ave,116th St,SE,40.801822,-73.949651
IRT,Lenox,116th St,40.802098,-73.949625,2,3,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Lenox Ave,116th St,NE,40.802111,-73.949437
IRT,Lenox,110th St-Central Park North,40.799075,-73.951822,2,3,,,,,,,,,,Stair,YES,,NO,NONE,,FALSE,,TRUE,Lenox Ave,111th St,NW,40.799033,-73.952058
IRT,Flushing,Broadway-74th St,40.746848,-73.891394,E,F,M,R,7,,,,,,,Stair,YES,,YES,NONE,,TRUE,,TRUE,74th St,Roosevelt Ave,NE,40.747025,-73.891413
IRT,Flushing,Bliss St-46th St,40.743132,-73.918435,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,47th St,Queens Blvd,SW,40.742987,-73.917936
IRT,Flushing,Bliss St-46th St,40.743132,-73.918435,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,47th St,Queens Blvd,NW,40.743134,-73.917895
IRT,Flushing,Rawson St-33rd St,40.744587,-73.930997,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,34th St,Queens Blvd,SW,40.744494,-73.930894
IRT,Flushing,Rawson St-33rd St,40.744587,-73.930997,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,TRUE,34th St,Queens Blvd,NW,40.744658,-73.930865
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Vernon Blvd,50th Ave,SW,40.742588,-73.954252
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Vernon Blvd,50th Ave,SW,40.742654,-73.954232
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Jackson Ave,50th Ave,NE,40.742496,-73.952482
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,YES,,YES,NONE,,FALSE,,FALSE,Jackson Ave,50th Ave,NW,40.742593,-73.95285
IRT,Flushing,Grand Central-42nd St,40.751431,-73.976041,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,3rd Ave,42nd St,SW,40.750851,-73.975078
IRT,42nd St Shuttle,Times Square,40.755983,-73.986229,A,C,E,N,Q,R,GS,1,2,3,7,Stair,YES,,YES,NONE,,FALSE,,TRUE,Broadway,43rd St,SW,40.756476,-73.986311
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Madison Ave,42nd St,NE,40.75283,-73.978917
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Vanderbilt Ave,42nd St,SE,40.752218,-73.978396
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Vanderbilt Ave,42nd St,SE,40.752279,-73.97828
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Easement,YES,,YES,NONE,,TRUE,,TRUE,Vanderbilt Ave,42nd St,NW,40.752717,-73.978487
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Escalator,YES,,YES,NONE,,TRUE,,TRUE,Park Ave,42nd St,SW,40.752184,-73.978048
BMT,Broadway,Union Square,40.735736,-73.990568,L,N,Q,R,4,5,6,,,,,Stair,NO,,NO,NONE,,TRUE,Lex not ADA,TRUE,4th Ave,15th St,SW,40.735311,-73.990265
BMT,Broadway,8th St,40.730328,-73.992629,N,R,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,FALSE,Broadway,Waverly Pl,NW,40.729778,-73.993262
BMT,West End,71st St,40.619589,-73.998864,D,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,New Utrecht Ave,Bay Ridge Ave,SE,40.62036,-73.99855
BMT,West End,79th St,40.613501,-74.00061,D,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,New Utrecht Ave,77th St,NE,40.614018,-74.000378
BMT,Broadway Jamaica,Lorimer St,40.703869,-73.947408,J,M,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Lorimer St,Broadway,NW,40.703454,-73.946536
BMT,Broadway Jamaica,Lorimer St,40.703869,-73.947408,J,M,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Lorimer St,Broadway,SW,40.703321,-73.946613
BMT,Canarsie,Morgan Av,40.706152,-73.933147,L,,,,,,,,,,,Door,NO,,NO,NONE,,FALSE,,TRUE,Bogart St,Harrison Pl,NW,40.706073,-73.933633
BMT,Canarsie,Jefferson St,40.706607,-73.922913,L,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Jefferson St,Wyckoff Ave,NW,40.707229,-73.923725
BMT,Canarsie,Jefferson St,40.706607,-73.922913,L,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Jefferson St,Wyckoff Ave,SW,40.707071,-73.923877
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Stanhope St,Wycoff Ave,SE,40.703248,-73.91772
BMT,Canarsie,DeKalb Av,40.703811,-73.918425,L,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Stanhope St,Wycoff Ave,SW,40.703353,-73.917957
BMT,Canarsie,Halsey St,40.695602,-73.904084,L,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,FALSE,Norman St,Wycoff Ave,NE,40.695186,-73.903065
BMT,Canarsie,Halsey St,40.695602,-73.904084,L,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,FALSE,Covert St,Wycoff Ave,NW,40.695201,-73.903764
IND,8 Avenue,50th St,40.762456,-73.985984,C,E,,,,,,,,,,Stair,NO,,NO,NONE,,TRUE,Southbound Only,FALSE,8th Ave,51st St,NE,40.7629,-73.985412
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,NO,,NO,NONE,,TRUE,,TRUE,8th Ave,43rd St,SW,40.757929,-73.989547
IND,8 Avenue,42nd St,40.757308,-73.989735,A,C,E,N,Q,R,S,1,2,3,7,Stair,NO,,NO,NONE,,TRUE,,TRUE,8th Ave,42nd St,NW,40.757463,-73.989984
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,8th Ave,15th St,NE,40.74041,-74.00175
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,8th Ave,15th St,NW,40.740598,-74.002185
IND,8 Avenue,14th St,40.740893,-74.00169,A,C,E,L,,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,8th Ave,14th St,NE,40.739776,-74.002256
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Jay St,Fulton St,NE,40.691559,-73.987208
IND,Fulton,Jay St - Borough Hall,40.692338,-73.987342,A,C,F,R,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Jay St,Fulton St,NW,40.6917,-73.987461
IRT,Clark,Fulton St,40.709416,-74.006571,A,C,J,Z,2,3,4,5,,,,Easement,NO,,NO,NONE,,FALSE,,TRUE,William St,John St,NE,40.70863,-74.006999
IRT,Clark,Fulton St,40.709416,-74.006571,A,C,J,Z,2,3,4,5,,,,Easement,NO,,NO,NONE,,FALSE,,TRUE,William St,Fulton St,SE,40.7092,-74.006664
IRT,Clark,Fulton St,40.709416,-74.006571,A,C,J,Z,2,3,4,5,,,,Easement,NO,,NO,NONE,,FALSE,,TRUE,William St,Fulton St,SE,40.709046,-74.006814
IRT,Flushing,5th Av,40.753821,-73.981963,B,D,F,M,7,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,5th-6th Aves,42nd St,S,40.754175,-73.982871
IRT,Flushing,5th Av,40.753821,-73.981963,B,D,F,M,7,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,5th-6th Aves,42nd St,S,40.754213,-73.982984
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Easement,NO,,NO,NONE,,TRUE,,TRUE,Madison Ave,42nd St,SW,40.752846,-73.979798
IRT,42nd St Shuttle,Grand Central,40.752769,-73.979189,GS,4,5,6,7,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,Madison Ave,42nd St,NW,40.753087,-73.979537
IRT,Broadway-7th Ave,South Ferry,40.702068,-74.013664,R,1,,,,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,,,,40.701313,-74.013479
BMT,Brighton,Av H,40.62927,-73.961639,B,Q,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,15th St,Avenue H,SE,40.629884,-73.961882
BMT,West End,71st St,40.619589,-73.998864,D,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,New Utrecht Ave,Bay Ridge Ave,SW,40.620379,-73.998764
BMT,West End,79th St,40.613501,-74.00061,D,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,New Utrecht Ave,77th St,NW,40.614054,-74.000585
BMT,Broadway Jamaica,121st St,40.700492,-73.828294,J,Z,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,123rd St,Jamaica Ave,SE,40.700596,-73.827239
IND,8 Avenue,Chambers St,40.714111,-74.008585,A,C,E,2,3,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Church St,Warren St,SE,40.7142,-74.008357
IND,Fulton,Clinton & Washington Avs,40.683263,-73.965838,C,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Clinton Ave,Fulton Ave,SE,40.683317,-73.966741
IND,Fulton,Rockaway Av,40.67834,-73.911946,A,C,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Rockaway Ave,Fulton St,SE,40.678017,-73.910661
IND,Fulton,Rockaway Av,40.67834,-73.911946,A,C,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Rockaway Ave,Fulton St,SW,40.678142,-73.910875
IND,Concourse,167th St,40.833769,-73.918432,B,D,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Grand Concourse,McClellan St,SE,40.832912,-73.918822
IND,Culver,Av I,40.625322,-73.976127,F,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,McDonald Ave,Avenue J,NW,40.624263,-73.976049
IND,Culver,Av U,40.596063,-73.973357,F,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,McDonald Ave,Gravesend Neck Rd,SW,40.595053,-73.973623
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,63rd Rd,Queens Blvd,NE,40.730417,-73.862373
IRT,Broadway-7th Ave,238th St,40.884667,-73.90087,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Broadway,238th St,SE,40.885226,-73.900439
IRT,Broadway-7th Ave,215th St,40.869444,-73.915279,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,10th Ave,215th St,NE,40.869729,-73.914954
IRT,Broadway-7th Ave,168th St,40.840556,-73.940133,A,C,1,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,168th St,SE,40.840717,-73.93972
IRT,Broadway-7th Ave,103rd St,40.799446,-73.968379,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,Broadway,104th St,SE,40.799584,-73.968091
IRT,Broadway-7th Ave,14th St,40.737826,-74.000201,F,L,M,1,2,3,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,TRUE,7th Ave,13th St,NE,40.737846,-73.99993
IRT,Broadway-7th Ave,Rector St,40.707513,-74.013783,1,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Greenwich St,Rector St,NW,40.708191,-74.013582
IRT,Nostrand,Newkirk Av,40.639967,-73.948411,2,5,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Nostrand Ave,Avenue D,SW,40.640743,-73.948597
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Easement,NO,Yes,NO,NONE,,TRUE,,TRUE,Park Ave,42nd St,SE,40.751721,-73.977197
IRT,Lenox,145th St,40.820421,-73.936245,3,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Lenox Ave,145th St,SE,40.820161,-73.936253
IRT,Lenox,145th St,40.820421,-73.936245,3,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Lenox Ave,145th St,NE,40.820602,-73.935933
IRT,Dyre Av,Baychester Av,40.878663,-73.838591,5,,,,,,,,,,,Door,NO,Yes,NO,NONE,,FALSE,,TRUE,Baychester Ave,Tillotson Ave,NW,40.87933,-73.837858
IRT,Dyre Av,Gun Hill Rd,40.869526,-73.846384,5,,,,,,,,,,,Easement,NO,Yes,NO,NONE,,FALSE,,TRUE,,,,40.870039,-73.845724
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Jackson Ave,50th Ave,SE,40.742351,-73.952723
IRT,Flushing,Vernon Blvd-Jackson Av,40.742626,-73.953581,7,,,,,,,,,,,Stair,NO,Yes,NO,NONE,,FALSE,,FALSE,Jackson Ave,50th Ave,SW,40.742432,-73.953156
IND,Rockaway,Aqueduct Racetrack,40.672131,-73.835812,A,,,,,,,,,,,Ramp,NO,,NO,NONE,,FALSE,,FALSE,,,,40.672208,-73.834589
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,PART,6am-9:40pm everyday,FALSE,,TRUE,Broadway,Canal St,NE,40.719498,-74.001606
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,PART,6am-9:40pm everyday,FALSE,,TRUE,Broadway,Canal St,SE,40.719218,-74.00185
BMT,Broadway,Canal St (UL),40.719527,-74.001775,J,N,Q,R,Z,6,,,,,,Stair,YES,,YES,PART,6am-9:40pm everyday,FALSE,,TRUE,Broadway,Canal St,SE,40.719166,-74.001774
BMT,Brighton,Kings Highway,40.60867,-73.957734,B,Q,,,,,,,,,,Door,YES,,YES,PART,,FALSE,,TRUE,,,W,40.608319,-73.957352
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Easement,YES,,YES,PART,,FALSE,,TRUE,Nassau St,Cedar St,SE,40.708027,-74.009814
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,,YES,PART,,FALSE,,TRUE,Broad St,Wall St,SE,40.706911,-74.010494
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,,YES,PART,,FALSE,,TRUE,Broad St,Wall St,SW,40.707117,-74.010821
BMT,Nassau,Broad St,40.706476,-74.011056,J,Z,,,,,,,,,,Stair,YES,,YES,PART,,FALSE,,TRUE,Broad St,Wall St,SE,40.706887,-74.010687
IND,8 Avenue,59th St,40.768296,-73.981736,A,B,C,D,1,,,,,,,Stair,YES,,YES,PART,3:20pm-10:40pm everyday,TRUE,,TRUE,Central Park West,60th St,West,40.768719,-73.981788
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,PART,6am-9:30am everyday,TRUE,,TRUE,8th Ave,33rd St,SE,40.751291,-73.993887
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,PART,6am-9:30am everyday,TRUE,,TRUE,8th Ave,33rd St,NE,40.751432,-73.993538
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,PART,6:30am-10:10pm everyday,TRUE,,TRUE,8th Ave,33rd St,NW,40.751677,-73.994107
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,PART,6:30am-10:10pm everyday,TRUE,,TRUE,8th Ave,33rd St,SW,40.751561,-73.994243
IND,8 Avenue,34th St,40.752287,-73.993391,A,C,E,,,,,,,,,Stair,YES,,YES,PART,6:30am-10:10pm everyday,TRUE,,TRUE,8th Ave,33rd St,SW,40.751539,-73.994181
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Easement,YES,,YES,PART,6:35am-9:40pm everyday,TRUE,,TRUE,6th Ave,33rd St,SW,40.74895,-73.988623
IND,6 Avenue,34th St,40.749719,-73.987823,B,D,F,M,N,Q,R,,,,,Easement,YES,,YES,PART,6:35am-9:40pm everyday,TRUE,,TRUE,6th Ave,32nd St,NW,40.748746,-73.988774
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Easement,YES,,YES,PART,1:45pm-9:10pm everyday,TRUE,,TRUE,3rd Ave,53rd St,SE,40.757475,-73.969255
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Easement,YES,,YES,PART,1:45pm-9:10pm everyday,TRUE,,TRUE,3rd Ave,53rd St,NE,40.757718,-73.969161
IND,Queens Boulevard,Lexington Av-53rd St,40.757552,-73.969055,E,M,6,,,,,,,,,Easement,YES,,YES,PART,1:45pm-9:10pm everyday,TRUE,,TRUE,3rd Ave,53rd St,NE,40.757685,-73.969125
IRT,Broadway-7th Ave,96th St,40.793919,-73.972323,1,2,3,,,,,,,,,Stair,YES,,YES,PART,6:20am-1:40pm everyday,TRUE,,TRUE,Broadway,96th St,SW,40.79307,-73.973174
IRT,Broadway-7th Ave,96th St,40.793919,-73.972323,1,2,3,,,,,,,,,Stair,YES,,YES,PART,6:20am-1:40pm everyday,TRUE,,TRUE,Broadway,96th St,SE,40.792923,-73.972824
IRT,Broadway-7th Ave,96th St,40.793919,-73.972323,1,2,3,,,,,,,,,Stair,YES,,YES,PART,6:20am-1:40pm everyday,TRUE,,TRUE,Broadway,96th St,SW,40.793209,-73.973074
IRT,Broadway-7th Ave,96th St,40.793919,-73.972323,1,2,3,,,,,,,,,Stair,YES,,YES,PART,6:20am-1:40pm everyday,TRUE,,TRUE,Broadway,96th St,SE,40.793065,-73.972722
IRT,Broadway-7th Ave,86th St,40.788644,-73.976218,1,,,,,,,,,,,Stair,YES,,YES,PART,6:30am-10am & 4pm-7:30pm weekdays,FALSE,,FALSE,Broadway,86th St,NE,40.78902,-73.97633
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Easement,YES,,YES,PART,3pm-10:30pm everyday,TRUE,,TRUE,7th Ave,42nd St,SE,40.755747,-73.986875
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,PART,7:30am-11:20pm everyday,TRUE,,TRUE,7th Ave,41st St,SE,40.755248,-73.987362
IRT,Broadway-7th Ave,Times Square,40.75529,-73.987495,A,C,E,N,Q,R,S,1,2,3,7,Stair,YES,,YES,PART,7:30am-11:20pm everyday,TRUE,,TRUE,7th Ave,41st St,SE,40.755334,-73.9872
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Easement,YES,,YES,PART,5:50am-1:10pm everyday,TRUE,,TRUE,7th Ave,32nd St,W,40.74979,-73.991721
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Easement,YES,,YES,PART,3:30pm-11pm everyday,TRUE,,TRUE,7th Ave,34th St,SE,40.75081,-73.990481
IRT,Broadway-7th Ave,34th St,40.750373,-73.991057,1,2,3,,,,,,,,,Stair,YES,,YES,PART,3:30pm-11pm everyday,TRUE,,TRUE,7th Ave,34th St,NE,40.75102,-73.990323
IRT,Broadway-7th Ave,28th St,40.747215,-73.993365,1,,,,,,,,,,,Stair,YES,,YES,PART,5:50am-1:10pm everyday,FALSE,,FALSE,7th Ave,28th St,SE,40.747036,-73.993138
IRT,Broadway-7th Ave,28th St,40.747215,-73.993365,1,,,,,,,,,,,Stair,YES,,YES,PART,5:50am-1:10pm everyday,FALSE,,FALSE,7th Ave,28th St,NE,40.747216,-73.993121
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Easement,YES,,YES,PART,11am-7pm everyday,TRUE,,TRUE,Lexington Ave,43rd St,SE,40.751801,-73.975294
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Easement,YES,,YES,PART,11am-7pm everyday,TRUE,,TRUE,Lexington Ave,42nd St,NE,40.751345,-73.975409
IRT,Lexington,Grand Central-42nd St,40.751776,-73.976848,GS,4,5,6,7,,,,,,,Stair,YES,,YES,PART,11am-7pm everyday,TRUE,,TRUE,Lexington Ave,42nd St,NW,40.751701,-73.976087
IRT,Dyre Av,Eastchester-Dyre Av,40.8883,-73.830834,5,,,,,,,,,,,Door,YES,,YES,PART,,FALSE,,TRUE,Dyre Ave,Light St,NE,40.88798,-73.831195
IRT,Dyre Av,Baychester Av,40.878663,-73.838591,5,,,,,,,,,,,Door,YES,,YES,PART,,FALSE,,TRUE,Baychester Ave,Tillotson Ave,NW,40.879519,-73.837958
IRT,Dyre Av,Gun Hill Rd,40.869526,-73.846384,5,,,,,,,,,,,Door,YES,,YES,PART,,FALSE,,TRUE,Seymour Ave,Gun Hill Rd,SE,40.870117,-73.845869
IRT,Dyre Av,Pelham Parkway,40.858985,-73.855359,5,,,,,,,,,,,Door,YES,,YES,PART,,FALSE,,TRUE,Esplanade,Laconia Ave,NE,40.858804,-73.855515
IRT,Dyre Av,Morris Park,40.854364,-73.860495,5,,,,,,,,,,,Door,YES,,YES,PART,,FALSE,,TRUE,Esplanade,Paulding Ave,W,40.854544,-73.860248
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Elevator,YES,,YES,PART,6am-2pm everyday,TRUE,,TRUE,Main St,Roosevelt Ave,NE,40.759908,-73.829019
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Escalator,YES,,YES,PART,6am-2pm everyday,TRUE,,TRUE,Main St,Roosevelt Ave,NE,40.760004,-73.82863
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Escalator,YES,,YES,PART,6am-2pm everyday,TRUE,,TRUE,Main St,Roosevelt Ave,NE,40.760048,-73.828444
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Escalator,YES,,YES,PART,6am-2pm everyday,TRUE,,TRUE,Main St,Roosevelt Ave,SE,40.759895,-73.828573
IRT,Flushing,Flushing-Main St,40.7596,-73.83003,7,,,,,,,,,,,Stair,YES,,YES,PART,6am-2pm everyday,TRUE,,TRUE,Main St,Roosevelt Ave,SE,40.759946,-73.828414
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,NO,Spc Ev,Used on Game Days,TRUE,,TRUE,River Ave,161st St,SW,40.827605,-73.926282
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,NO,Spc Ev,Used on Game Days,TRUE,,TRUE,River Ave,161st St,SW,40.827529,-73.926322
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,NO,Spc Ev,Used on Game Days,TRUE,,TRUE,River Ave,161st St,SW,40.827461,-73.926362
IRT,Jerome,Yankee Stadium-161st St,40.827994,-73.925831,B ,D,4,,,,,,,,,Stair,YES,,NO,Spc Ev,Used on Game Days,TRUE,,TRUE,River Ave,161st St,SW,40.827399,-73.926394
IND,6 Avenue,Broadway-Lafayette St,40.725297,-73.996204,B,D,F,M,6,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Lafayette St,Houston St,SW,40.724987,-73.995628
IND,6 Avenue,Broadway-Lafayette St,40.725297,-73.996204,B,D,F,M,6,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Crosby St,Houston St,SE,40.725011,-73.995745
IND,Culver,Av I,40.625322,-73.976127,F,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,McDonald Ave,Avenue J,NE,40.624301,-73.975817
IND,Culver,Av U,40.596063,-73.973357,F,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,McDonald Ave,Gravesend Neck Rd,SE,40.595027,-73.973382
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,63rd Dr,Queens Blvd,NW,40.730037,-73.861343
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,63rd Dr,Queens Blvd,SW,40.729574,-73.861751
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,64th Rd,Queens Blvd,SE,40.728847,-73.859668
IND,Queens Boulevard,63rd Drive-Rego Park,40.729846,-73.861604,M,R,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,64th Rd,Queens Blvd,NW,40.729774,-73.860489
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,75th St,Broadway,SE,40.745988,-73.890208
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,75th St,Broadway,NE,40.746391,-73.890285
IND,Queens Boulevard,Jackson Heights-Roosevelt Ave,40.746644,-73.891338,E,F,M,R,7,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,74th St,Roosevelt Av,SW,40.746527,-73.891461
IND,Crosstown,Long Island City-Court Square,40.746554,-73.943832,G,,,,,,,,,,,Easement,NO,,NO,NONE,,FALSE,,TRUE,Jackson Ave,45th Ave,NW,40.746869,-73.943522
IND,Crosstown,Long Island City-Court Square,40.746554,-73.943832,G,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Court Square,Jackson Ave,SW,40.746243,-73.944011
IRT,Broadway-7th Ave,207th St,40.864614,-73.918819,1,,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,FALSE,10th Ave,207th St,NE,40.864599,-73.918681
IRT,Broadway-7th Ave,96th St,40.793919,-73.972323,1,2,3,,,,,,,,,Door,NO,,YES,NONE,,TRUE,,TRUE,Broadway,95th St,N,40.794611,-73.972053
IRT,Broadway-7th Ave,72nd St,40.778453,-73.98197,1,2,3,,,,,,,,,Door,NO,,NO,NONE,,TRUE,,TRUE,Broadway,72nd St,SE,40.778437,-73.981947
IRT,Broadway-7th Ave,59th St-Columbus Circle,40.768247,-73.981929,A,B,C,D,1,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,Columbus Circle,59th St,SE,40.767654,-73.981423
IRT,Broadway-7th Ave,59th St-Columbus Circle,40.768247,-73.981929,A,B,C,D,1,,,,,,,Stair,NO,,NO,NONE,,TRUE,,TRUE,Columbus Circle,59th St,SE,40.767642,-73.981235
IRT,Eastern Parkway,Hoyt St,40.690545,-73.985065,2,3,,,,,,,,,,Stair,NO,,NO,NONE,,FALSE,,FALSE,Duffield St,Fulton St,NE,40.690433,-73.984323
IRT,Lexington,59th St,40.762526,-73.967967,N,Q,R,4,5,6,,,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Lexington Ave,60th St,NW,40.762986,-73.96785
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Elevator,NO,,NO,NONE,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,NW,40.718589,-74.000552
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Stair,NO,,NO,NONE,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,SW,40.718506,-74.000826
IRT,Lexington,Canal St,40.718803,-74.000193,J,N,Q,R,Z,6,,,,,,Stair,NO,,NO,NONE,,TRUE,Bway Nass not ADA,TRUE,Lafayette St,Canal St,NW,40.718638,-74.000623
IRT,Lexington,Fulton St,40.710368,-74.009509,A,C,J,Z,2,3,4,5,,,,Easement,NO,,NO,NONE,,FALSE,,TRUE,Broadway,Fulton St,SW,40.710944,-74.009403
IRT,Lexington,Fulton St,40.710368,-74.009509,A,C,J,Z,2,3,4,5,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Broadway,Fulton St,NW,40.711026,-74.00919
IRT,Lexington,Fulton St,40.710368,-74.009509,A,C,J,Z,2,3,4,5,,,,Stair,NO,,NO,NONE,,FALSE,,TRUE,Broadway,Fulton St,NW,40.711112,-74.009382""")

In [ ]:
#can run in local environments 
#url="http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv"
#subway_df = pd.read_csv(url)

subway_df = pd.read_csv(TESTDATA)

subway_df.head()

In [ ]:
df = pd.read_json(open("../input/train.json", "r"))

df.head().reset_index()

In [ ]:
df_test = df_lat_long.merge(subway_lat_long
                            , left_on='lkey'
                            , right_on='rkey'
                            , how='outer')


In [ ]:
#notice difference in capitalizations

#lat / long of buildings 
#dropping duplicates, we'll find the distance to the nearest subway for each building
df_lat_long = df[["latitude", "longitude"]].drop_duplicates()
df_lat_long["lkey"] = 1 

#just run though this with 5 records... 
#df_lat_long = df_lat_long[:5]
#df_lat_long["building_long"] = df["longitude"]

subway_lat_long = subway_df[["Latitude", "Longitude"]]
subway_lat_long["rkey"] = 1 

In [ ]:
#4 decimal points = accurate to ~8m at this latitude
df_test = df_lat_long.merge(subway_lat_long
                            , left_on='lkey'
                            , right_on='rkey'
                            , how='outer')


In [ ]:
#this is a haversine calculation that's configured to work on numpy arrays... 
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
km = haversine_np(df_test['longitude'],df_test['latitude'],df_test['Longitude'],df_test['Latitude'])

In [ ]:
df_test['distance'] = km

In [ ]:
#creates a column, "distance", in kilometers 
# not used
"""
for index, row in df_test.iterrows():
    df_test.loc[index, 'distance'] = haversine(row['longitude']
                                          , row['latitude']
                                          , row['Longitude']
                                          , row['Latitude']
                                         )
  """

In [ ]:
#overwriting df_test for memory reasons
df_test = df_test[['latitude', 'longitude', 'distance']]
print(df_test.info())


In [ ]:
#http://stackoverflow.com/questions/15705630/python-getting-the-row-which-has-the-max-value-in-groups-using-groupby
df_grouped = df.groupby(['latitude', 'longitude']).agg({'distance':'min'})

df_grouped

In [ ]:
#http://stackoverflow.com/questions/21247992/grouping-pandas-dataframe-by-two-columns-or-more

df_test_slice.groupby(['latitude','longitude'], as_index=False)

In [ ]:
#http://stackoverflow.com/questions/23394476/keep-other-columns-when-using-min-with-groupby

df_distances = df_test.loc[df_test.groupby("latitude")["longitude"].idxmin()]

In [ ]:
#join df_distances to df

df_new = pd.concat([df, df_distances], axis=1, join_axes=[df1.index], )

In [ ]:
df_new = df.join(df_distances, how='left')

df_new[:5]

# read data

In [ ]:
df = pd.read_json(open("../input/train.json", "r"))

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
print(df.shape)

# naive feature engineering

In [ ]:
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day

In [ ]:
num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]

tot_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day", "features"]

X = df[tot_feats]
y = df["interest_level"]

#fill na features with the words "no features"
X["features_clean"] = X["features"].str[0].fillna("No Features")

X[:20]

# split model

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)

In [ ]:
X_train["features"].str[0].head()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer

# Text analysis

In [ ]:
def strings_to_wordlist( strings, remove_stopwords=True):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    
    
    # Remove non-letters
    string_text = re.sub("[^a-zA-Z]"," ", strings)
    #
    # Convert words to lower case and split them
    words = string_text.lower().split()
    #
    # Optionally remove stop words (True by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer #PorterStemmer()
    for word in words:
        b.append(stemmer.stem(word))

    # Return a list of words
    return(b)

In [ ]:
train_features = []
for feature in X_train["features_clean"]:
    train_features.append( " ".join(strings_to_wordlist(feature)))

val_features = []
for feature in X_val["features_clean"]:
    val_features.append( " ".join(strings_to_wordlist(feature))) 

In [ ]:
vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 5000, ngram_range = ( 1, 3 ),
                              sublinear_tf = True )

vectorizer = vectorizer.fit(train_features)
train_features_transformed = vectorizer.transform(X_train["features_clean"])

val_features_transformed = vectorizer.transform(X_val["features_clean"])

#Train Model

In [ ]:
model3 = RandomForestClassifier()
model3.fit(  train_features_transformed, y_train)

In [ ]:
x_train_pred = model3.predict_proba(train_features_transformed) 

In [ ]:
type(x_train_pred)

In [ ]:
x_train_2 = pd.DataFrame(x_train_pred)
x_train_2['index_col'] = y_train.index.values

x_train_vals = x_train_2.set_index('index_col')

In [ ]:
x_train_2.info()

In [ ]:
x_train_vals.head()

In [ ]:
x_train_full = pd.concat([X_train[num_feats], x_train_vals], axis=1, ignore_index=False)

x_train_full.head()

In [ ]:
y_val_pred = model3.predict_proba(val_features_transformed)
x_train_2 = pd.DataFrame(y_val_pred)
x_train_2['index_col'] = X_val.index.values

x_train_vals = x_train_2.set_index('index_col')

x_val_full = pd.concat([X_val[num_feats], x_train_vals], axis=1, ignore_index=False)

In [ ]:
x_val_full.head()

In [ ]:
clf = RandomForestClassifier(n_estimators=1800)
clf.fit(x_train_full, y_train)
y_val_pred = clf.predict_proba(x_val_full)
log_loss(y_val, y_val_pred)

# make prediction

In [ ]:
df = pd.read_json(open("../input/test.json", "r"))
print(df.shape)
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day
X = df[num_feats]

y = clf.predict_proba(X)

In [ ]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_rf.csv", index=False)